In [1]:
import pandas as pd
import os
import ast

## A function used to update Country names cleanly is imported from another file in this project
from aux_fix_country_utils import fix_country_from_text

import warnings
warnings.simplefilter(action='ignore', category=pd.errors.DtypeWarning)

### Imports

In [2]:
DF_input_base = input().strip()

 C:\Users\svalb\OneDrive\Escritorio\Data_40_years_cancer_studies\parsedXMLs_combined_until_Oct2025_base\


In [3]:
DF_input_match_GLOBOCAN = input().strip()

 C:\Users\svalb\OneDrive\Escritorio\Data_40_years_cancer_studies\parsedXMLs_combined_until_Oct2025_match_GLOBOCAN\


In [4]:
# Import list of csvs to parse
list_csvs_base = []

for file in os.listdir(DF_input_base):
    if file[-4:] == ".csv":
        list_csvs_base.append(file)

n_csvs_base = len(list_csvs_base)

In [5]:
# Import list of csvs to parse
list_csvs_match_GLOBOCAN = []

for file in os.listdir(DF_input_match_GLOBOCAN):
    if file[-4:] == ".csv":
        list_csvs_match_GLOBOCAN.append(file)

n_csvs_match_GLOBOCAN = len(list_csvs_match_GLOBOCAN)

## Note
The following corrections are performed afer separating the articles' dataset in two copies, one in which the cancer and country names are untouched (base) and another in which cancer and country names are modified to match those in the Globocan dataset. Here corrections in both copies are performed.

### Global corrections

In [6]:
# Global correction: Replace "Pitcairn" by "Italy" in Country. All assignments to Pitcairn correspond to italian research centers
# Base dataset
print("--Fixing base dataset")

for csv in list_csvs_base:
    print(f"Parsing csv: {csv} ({list_csvs_base.index(csv) + 1}/{n_csvs_base})")
    df = pd.read_csv(DF_input_base + csv)
    df.loc[df["Country"] == "Pitcairn", "Country"] = "Italy"
    df.to_csv(DF_input_base + csv, index = False)
    del df

print("--Fixing done for base dataset")

print("--Fixing match GLobocan dataset")

# Match Globocan dataset
for csv in list_csvs_match_GLOBOCAN:
    print(f"Parsing csv: {csv} ({list_csvs_match_GLOBOCAN.index(csv) + 1}/{n_csvs_match_GLOBOCAN})")
    df = pd.read_csv(DF_input_match_GLOBOCAN + csv)
    df.loc[df["Country"] == "Pitcairn", "Country"] = "Italy"
    df.to_csv(DF_input_match_GLOBOCAN + csv, index = False)
    del df

print("--Fixing done for match GLobocan dataset")

--Fixing base dataset
Parsing csv: parsedX_100000.csv (1/45)
Parsing csv: parsedX_1000000.csv (2/45)
Parsing csv: parsedX_1100000.csv (3/45)
Parsing csv: parsedX_1200000.csv (4/45)
Parsing csv: parsedX_1300000.csv (5/45)
Parsing csv: parsedX_1400000.csv (6/45)
Parsing csv: parsedX_1500000.csv (7/45)
Parsing csv: parsedX_1600000.csv (8/45)
Parsing csv: parsedX_1700000.csv (9/45)
Parsing csv: parsedX_1800000.csv (10/45)
Parsing csv: parsedX_1900000.csv (11/45)
Parsing csv: parsedX_200000.csv (12/45)
Parsing csv: parsedX_2000000.csv (13/45)
Parsing csv: parsedX_2100000.csv (14/45)
Parsing csv: parsedX_2200000.csv (15/45)
Parsing csv: parsedX_2300000.csv (16/45)
Parsing csv: parsedX_2400000.csv (17/45)
Parsing csv: parsedX_2500000.csv (18/45)
Parsing csv: parsedX_2600000.csv (19/45)
Parsing csv: parsedX_2700000.csv (20/45)
Parsing csv: parsedX_2800000.csv (21/45)
Parsing csv: parsedX_2900000.csv (22/45)
Parsing csv: parsedX_300000.csv (23/45)
Parsing csv: parsedX_3000000.csv (24/45)
Parsin

#### Systematic corrections for **Most_common_list_countries**
The following corrections aim to fix wrong Country assigments via the **Most_common_list_countries** *Country_source*. In some cases there were systematic errors found (e.g. in articles with the word "Perugia" in the affiliation, the assigned country was "Peru" instead of "Italy". Fixing is done for bothe the base and the matching Globocan datasets

In [2]:
def fixSystematicMost_common_list_countries(dataset, DF_location, list_csvs, country_wrong, problematic_text, country_right):
    print(f"--Fixing {dataset} dataset")
    for csv in list_csvs:
        print(f"Parsing csv: {csv} ({list_csvs.index(csv) + 1}/{len(list_csvs)})")
        df = pd.read_csv(DF_location + csv)

        # Helper df with only records which have the Most_common_list_countries and the problematic country assignment
        df_short = df.loc[(df["Country_source"] == "Most_common_list_countries") & (df["Country"] == country_wrong)]

        # If any record found
        if len(df_short) > 0:
            for PMID in df_short["PMID"].to_list():

                # NER_lastAuthor is stored as a string, convert to list and iterate over elements
                for el in ast.literal_eval(df_short.loc[df_short["PMID"] == PMID, "NER_lastAuthor"].values[0]):

                    # If problematic text in the list, assign correct Country and Manual_correction
                    if problematic_text in el['text']:
                        df.loc[df["PMID"] == PMID, ["Country", "Country_source"]] = [country_right, "Manual_correction"]
    
            df.to_csv(DF_location + csv, index = False)
        del df, df_short
    print(f"--Fixing done for {dataset} dataset")    

In [8]:
## Fix wrong Country assignment in Most_common_list_countries: Peru instead of Italy in records with Perugia
fixSystematicMost_common_list_countries("base", DF_input_base, list_csvs_base, "Peru", "Perugia", "Italy")
fixSystematicMost_common_list_countries("match GLobocan", DF_input_match_GLOBOCAN, list_csvs_match_GLOBOCAN, "Peru", "Perugia", "Italy")

--Fixing base dataset
Parsing csv: parsedX_100000.csv (1/45)
Parsing csv: parsedX_1000000.csv (2/45)
Parsing csv: parsedX_1100000.csv (3/45)
Parsing csv: parsedX_1200000.csv (4/45)
Parsing csv: parsedX_1300000.csv (5/45)
Parsing csv: parsedX_1400000.csv (6/45)
Parsing csv: parsedX_1500000.csv (7/45)
Parsing csv: parsedX_1600000.csv (8/45)
Parsing csv: parsedX_1700000.csv (9/45)
Parsing csv: parsedX_1800000.csv (10/45)
Parsing csv: parsedX_1900000.csv (11/45)
Parsing csv: parsedX_200000.csv (12/45)
Parsing csv: parsedX_2000000.csv (13/45)
Parsing csv: parsedX_2100000.csv (14/45)
Parsing csv: parsedX_2200000.csv (15/45)
Parsing csv: parsedX_2300000.csv (16/45)
Parsing csv: parsedX_2400000.csv (17/45)
Parsing csv: parsedX_2500000.csv (18/45)
Parsing csv: parsedX_2600000.csv (19/45)
Parsing csv: parsedX_2700000.csv (20/45)
Parsing csv: parsedX_2800000.csv (21/45)
Parsing csv: parsedX_2900000.csv (22/45)
Parsing csv: parsedX_300000.csv (23/45)
Parsing csv: parsedX_3000000.csv (24/45)
Parsin

In [110]:
## Fix wrong Country assignment in Most_common_list_countries: United Kingdom instead of China in records with Ningbo
fixSystematicMost_common_list_countries("base", DF_input_base, list_csvs_base, "United Kingdom", "Ningbo", "China")
fixSystematicMost_common_list_countries("match GLobocan", DF_input_match_GLOBOCAN, list_csvs_match_GLOBOCAN, "United Kingdom", "Ningbo", "China")

--Fixing base dataset
Parsing csv: parsedX_100000.csv (1/45)
Parsing csv: parsedX_1000000.csv (2/45)
Parsing csv: parsedX_1100000.csv (3/45)
Parsing csv: parsedX_1200000.csv (4/45)
Parsing csv: parsedX_1300000.csv (5/45)
Parsing csv: parsedX_1400000.csv (6/45)
Parsing csv: parsedX_1500000.csv (7/45)
Parsing csv: parsedX_1600000.csv (8/45)
Parsing csv: parsedX_1700000.csv (9/45)
Parsing csv: parsedX_1800000.csv (10/45)
Parsing csv: parsedX_1900000.csv (11/45)
Parsing csv: parsedX_200000.csv (12/45)
Parsing csv: parsedX_2000000.csv (13/45)
Parsing csv: parsedX_2100000.csv (14/45)
Parsing csv: parsedX_2200000.csv (15/45)
Parsing csv: parsedX_2300000.csv (16/45)
Parsing csv: parsedX_2400000.csv (17/45)
Parsing csv: parsedX_2500000.csv (18/45)
Parsing csv: parsedX_2600000.csv (19/45)
Parsing csv: parsedX_2700000.csv (20/45)
Parsing csv: parsedX_2800000.csv (21/45)
Parsing csv: parsedX_2900000.csv (22/45)
Parsing csv: parsedX_300000.csv (23/45)
Parsing csv: parsedX_3000000.csv (24/45)
Parsin

In [113]:
## Fix wrong Country assignment in Most_common_list_countries: United Kingdom instead of China in records with Bengbu
fixSystematicMost_common_list_countries("base", DF_input_base, list_csvs_base, "United Kingdom", "Bengbu", "China")
fixSystematicMost_common_list_countries("match GLobocan", DF_input_match_GLOBOCAN, list_csvs_match_GLOBOCAN, "United Kingdom", "Bengbu", "China")

--Fixing base dataset
Parsing csv: parsedX_100000.csv (1/45)
Parsing csv: parsedX_1000000.csv (2/45)
Parsing csv: parsedX_1100000.csv (3/45)
Parsing csv: parsedX_1200000.csv (4/45)
Parsing csv: parsedX_1300000.csv (5/45)
Parsing csv: parsedX_1400000.csv (6/45)
Parsing csv: parsedX_1500000.csv (7/45)
Parsing csv: parsedX_1600000.csv (8/45)
Parsing csv: parsedX_1700000.csv (9/45)
Parsing csv: parsedX_1800000.csv (10/45)
Parsing csv: parsedX_1900000.csv (11/45)
Parsing csv: parsedX_200000.csv (12/45)
Parsing csv: parsedX_2000000.csv (13/45)
Parsing csv: parsedX_2100000.csv (14/45)
Parsing csv: parsedX_2200000.csv (15/45)
Parsing csv: parsedX_2300000.csv (16/45)
Parsing csv: parsedX_2400000.csv (17/45)
Parsing csv: parsedX_2500000.csv (18/45)
Parsing csv: parsedX_2600000.csv (19/45)
Parsing csv: parsedX_2700000.csv (20/45)
Parsing csv: parsedX_2800000.csv (21/45)
Parsing csv: parsedX_2900000.csv (22/45)
Parsing csv: parsedX_300000.csv (23/45)
Parsing csv: parsedX_3000000.csv (24/45)
Parsin

In [114]:
## Fix wrong Country assignment in Most_common_list_countries: Mongolia instead of China in records with Inner Mongolia
fixSystematicMost_common_list_countries("base", DF_input_base, list_csvs_base, "Mongolia", "Inner Mongolia", "China")
fixSystematicMost_common_list_countries("match GLobocan", DF_input_match_GLOBOCAN, list_csvs_match_GLOBOCAN, "Mongolia", "Inner Mongolia", "China")

--Fixing base dataset
Parsing csv: parsedX_100000.csv (1/45)
Parsing csv: parsedX_1000000.csv (2/45)
Parsing csv: parsedX_1100000.csv (3/45)
Parsing csv: parsedX_1200000.csv (4/45)
Parsing csv: parsedX_1300000.csv (5/45)
Parsing csv: parsedX_1400000.csv (6/45)
Parsing csv: parsedX_1500000.csv (7/45)
Parsing csv: parsedX_1600000.csv (8/45)
Parsing csv: parsedX_1700000.csv (9/45)
Parsing csv: parsedX_1800000.csv (10/45)
Parsing csv: parsedX_1900000.csv (11/45)
Parsing csv: parsedX_200000.csv (12/45)
Parsing csv: parsedX_2000000.csv (13/45)
Parsing csv: parsedX_2100000.csv (14/45)
Parsing csv: parsedX_2200000.csv (15/45)
Parsing csv: parsedX_2300000.csv (16/45)
Parsing csv: parsedX_2400000.csv (17/45)
Parsing csv: parsedX_2500000.csv (18/45)
Parsing csv: parsedX_2600000.csv (19/45)
Parsing csv: parsedX_2700000.csv (20/45)
Parsing csv: parsedX_2800000.csv (21/45)
Parsing csv: parsedX_2900000.csv (22/45)
Parsing csv: parsedX_300000.csv (23/45)
Parsing csv: parsedX_3000000.csv (24/45)
Parsin

In [118]:
## Fix wrong Country assignment in Most_common_list_countries: Italy instead of Switzerland in records with Svizzera Italiana
fixSystematicMost_common_list_countries("base", DF_input_base, list_csvs_base, "Italy", "Svizzera Italiana", "Switzerland")
fixSystematicMost_common_list_countries("match GLobocan", DF_input_match_GLOBOCAN, list_csvs_match_GLOBOCAN, "Italy", "Svizzera Italiana", "Switzerland")

--Fixing base dataset
Parsing csv: parsedX_100000.csv (1/45)
Parsing csv: parsedX_1000000.csv (2/45)
Parsing csv: parsedX_1100000.csv (3/45)
Parsing csv: parsedX_1200000.csv (4/45)
Parsing csv: parsedX_1300000.csv (5/45)
Parsing csv: parsedX_1400000.csv (6/45)
Parsing csv: parsedX_1500000.csv (7/45)
Parsing csv: parsedX_1600000.csv (8/45)
Parsing csv: parsedX_1700000.csv (9/45)
Parsing csv: parsedX_1800000.csv (10/45)
Parsing csv: parsedX_1900000.csv (11/45)
Parsing csv: parsedX_200000.csv (12/45)
Parsing csv: parsedX_2000000.csv (13/45)
Parsing csv: parsedX_2100000.csv (14/45)
Parsing csv: parsedX_2200000.csv (15/45)
Parsing csv: parsedX_2300000.csv (16/45)
Parsing csv: parsedX_2400000.csv (17/45)
Parsing csv: parsedX_2500000.csv (18/45)
Parsing csv: parsedX_2600000.csv (19/45)
Parsing csv: parsedX_2700000.csv (20/45)
Parsing csv: parsedX_2800000.csv (21/45)
Parsing csv: parsedX_2900000.csv (22/45)
Parsing csv: parsedX_300000.csv (23/45)
Parsing csv: parsedX_3000000.csv (24/45)
Parsin

In [3]:
# Similar to the previous function but fixing only if the NER term is EXACTLY the problematic text
def fixSystematicMost_common_list_countries_FULL_MATCH(dataset, DF_location, list_csvs, country_wrong, problematic_text, country_right):
    print(f"--Fixing {dataset} dataset")
    for csv in list_csvs:
        print(f"Parsing csv: {csv} ({list_csvs.index(csv) + 1}/{len(list_csvs)})")
        df = pd.read_csv(DF_location + csv)

        # Helper df with only records which have the Most_common_list_countries and the problematic country assignment
        df_short = df.loc[(df["Country_source"] == "Most_common_list_countries") & (df["Country"] == country_wrong)]

        # If any record found
        if len(df_short) > 0:
            for PMID in df_short["PMID"].to_list():

                # NER_lastAuthor is stored as a string, convert to list and iterate over elements
                for el in ast.literal_eval(df_short.loc[df_short["PMID"] == PMID, "NER_lastAuthor"].values[0]):

                    # If problematic text is exaclty (not substring) in the list, assign correct Country and Manual_correction
                    if problematic_text == el['text']:
                        df.loc[df["PMID"] == PMID, ["Country", "Country_source"]] = [country_right, "Manual_correction"]
    
            df.to_csv(DF_location + csv, index = False)
        del df, df_short
    print(f"--Fixing done for {dataset} dataset")

In [10]:
## Fix wrong Country assignment in Most_common_list_countries: Peru instead of Brazil in records with PE

# Important: SUBSTRINGS not taken into consideration, corrected only if el["text"] == "PE"
# Important: some records should be assinged to other countries. These are done manually below

fixSystematicMost_common_list_countries_FULL_MATCH("base", DF_input_base, list_csvs_base, "Peru", "PE", "Brazil")
fixSystematicMost_common_list_countries_FULL_MATCH("match GLobocan", DF_input_match_GLOBOCAN, list_csvs_match_GLOBOCAN, "Peru", "PE", "Brazil")

--Fixing base dataset
Parsing csv: parsedX_100000.csv (1/45)
Parsing csv: parsedX_1000000.csv (2/45)
Parsing csv: parsedX_1100000.csv (3/45)
Parsing csv: parsedX_1200000.csv (4/45)
Parsing csv: parsedX_1300000.csv (5/45)
Parsing csv: parsedX_1400000.csv (6/45)
Parsing csv: parsedX_1500000.csv (7/45)
Parsing csv: parsedX_1600000.csv (8/45)
Parsing csv: parsedX_1700000.csv (9/45)
Parsing csv: parsedX_1800000.csv (10/45)
Parsing csv: parsedX_1900000.csv (11/45)
Parsing csv: parsedX_200000.csv (12/45)
Parsing csv: parsedX_2000000.csv (13/45)
Parsing csv: parsedX_2100000.csv (14/45)
Parsing csv: parsedX_2200000.csv (15/45)
Parsing csv: parsedX_2300000.csv (16/45)
Parsing csv: parsedX_2400000.csv (17/45)
Parsing csv: parsedX_2500000.csv (18/45)
Parsing csv: parsedX_2600000.csv (19/45)
Parsing csv: parsedX_2700000.csv (20/45)
Parsing csv: parsedX_2800000.csv (21/45)
Parsing csv: parsedX_2900000.csv (22/45)
Parsing csv: parsedX_300000.csv (23/45)
Parsing csv: parsedX_3000000.csv (24/45)
Parsin

In [11]:
## Fix wrong Country assignment in Most_common_list_countries: United Kingdom instead of Sweden in records with Sweden

# Important: SUBSTRINGS not taken into consideration, corrected only if el["text"] == "Sweden"

fixSystematicMost_common_list_countries_FULL_MATCH("base", DF_input_base, list_csvs_base, "United Kingdom", "Sweden", "Sweden")
fixSystematicMost_common_list_countries_FULL_MATCH("match GLobocan", DF_input_match_GLOBOCAN, list_csvs_match_GLOBOCAN, "United Kingdom", "Sweden", "Sweden")

--Fixing base dataset
Parsing csv: parsedX_100000.csv (1/45)
Parsing csv: parsedX_1000000.csv (2/45)
Parsing csv: parsedX_1100000.csv (3/45)
Parsing csv: parsedX_1200000.csv (4/45)
Parsing csv: parsedX_1300000.csv (5/45)
Parsing csv: parsedX_1400000.csv (6/45)
Parsing csv: parsedX_1500000.csv (7/45)
Parsing csv: parsedX_1600000.csv (8/45)
Parsing csv: parsedX_1700000.csv (9/45)
Parsing csv: parsedX_1800000.csv (10/45)
Parsing csv: parsedX_1900000.csv (11/45)
Parsing csv: parsedX_200000.csv (12/45)
Parsing csv: parsedX_2000000.csv (13/45)
Parsing csv: parsedX_2100000.csv (14/45)
Parsing csv: parsedX_2200000.csv (15/45)
Parsing csv: parsedX_2300000.csv (16/45)
Parsing csv: parsedX_2400000.csv (17/45)
Parsing csv: parsedX_2500000.csv (18/45)
Parsing csv: parsedX_2600000.csv (19/45)
Parsing csv: parsedX_2700000.csv (20/45)
Parsing csv: parsedX_2800000.csv (21/45)
Parsing csv: parsedX_2900000.csv (22/45)
Parsing csv: parsedX_300000.csv (23/45)
Parsing csv: parsedX_3000000.csv (24/45)
Parsin

In [12]:
## Fix wrong Country assignment in Most_common_list_countries: Denmark instead of United Kingdom in records with Denmark Hill

# Important: SUBSTRINGS not taken into consideration, corrected only if el["text"] == "Denmark Hill"

fixSystematicMost_common_list_countries_FULL_MATCH("base", DF_input_base, list_csvs_base, "Denmark", "Denmark Hill", "United Kingdom")
fixSystematicMost_common_list_countries_FULL_MATCH("match GLobocan", DF_input_match_GLOBOCAN, list_csvs_match_GLOBOCAN, "Denmark", "Denmark Hill", "United Kingdom")

--Fixing base dataset
Parsing csv: parsedX_100000.csv (1/45)
Parsing csv: parsedX_1000000.csv (2/45)
Parsing csv: parsedX_1100000.csv (3/45)
Parsing csv: parsedX_1200000.csv (4/45)
Parsing csv: parsedX_1300000.csv (5/45)
Parsing csv: parsedX_1400000.csv (6/45)
Parsing csv: parsedX_1500000.csv (7/45)
Parsing csv: parsedX_1600000.csv (8/45)
Parsing csv: parsedX_1700000.csv (9/45)
Parsing csv: parsedX_1800000.csv (10/45)
Parsing csv: parsedX_1900000.csv (11/45)
Parsing csv: parsedX_200000.csv (12/45)
Parsing csv: parsedX_2000000.csv (13/45)
Parsing csv: parsedX_2100000.csv (14/45)
Parsing csv: parsedX_2200000.csv (15/45)
Parsing csv: parsedX_2300000.csv (16/45)
Parsing csv: parsedX_2400000.csv (17/45)
Parsing csv: parsedX_2500000.csv (18/45)
Parsing csv: parsedX_2600000.csv (19/45)
Parsing csv: parsedX_2700000.csv (20/45)
Parsing csv: parsedX_2800000.csv (21/45)
Parsing csv: parsedX_2900000.csv (22/45)
Parsing csv: parsedX_300000.csv (23/45)
Parsing csv: parsedX_3000000.csv (24/45)
Parsin

In [8]:
## Fix wrong Country assignment in Most_common_list_countries: Ireland instead of United Kingdom in records with N. Ireland

# Important: SUBSTRINGS not taken into consideration, corrected only if el["text"] == "N. Ireland"

fixSystematicMost_common_list_countries_FULL_MATCH("base", DF_input_base, list_csvs_base, "Ireland", "N. Ireland", "United Kingdom")
fixSystematicMost_common_list_countries_FULL_MATCH("match GLobocan", DF_input_match_GLOBOCAN, list_csvs_match_GLOBOCAN, "Ireland", "N. Ireland", "United Kingdom")

--Fixing base dataset
Parsing csv: parsedX_100000.csv (1/45)
Parsing csv: parsedX_1000000.csv (2/45)
Parsing csv: parsedX_1100000.csv (3/45)
Parsing csv: parsedX_1200000.csv (4/45)
Parsing csv: parsedX_1300000.csv (5/45)
Parsing csv: parsedX_1400000.csv (6/45)
Parsing csv: parsedX_1500000.csv (7/45)
Parsing csv: parsedX_1600000.csv (8/45)
Parsing csv: parsedX_1700000.csv (9/45)
Parsing csv: parsedX_1800000.csv (10/45)
Parsing csv: parsedX_1900000.csv (11/45)
Parsing csv: parsedX_200000.csv (12/45)
Parsing csv: parsedX_2000000.csv (13/45)
Parsing csv: parsedX_2100000.csv (14/45)
Parsing csv: parsedX_2200000.csv (15/45)
Parsing csv: parsedX_2300000.csv (16/45)
Parsing csv: parsedX_2400000.csv (17/45)
Parsing csv: parsedX_2500000.csv (18/45)
Parsing csv: parsedX_2600000.csv (19/45)
Parsing csv: parsedX_2700000.csv (20/45)
Parsing csv: parsedX_2800000.csv (21/45)
Parsing csv: parsedX_2900000.csv (22/45)
Parsing csv: parsedX_300000.csv (23/45)
Parsing csv: parsedX_3000000.csv (24/45)
Parsin

In [21]:
## Fix wrong Country assignment in Most_common_list_countries: Bangladesh instead of France in records with Bd

# Important: SUBSTRINGS not taken into consideration, corrected only if el["text"] == "Bd"
# Important: some records should be assinged to other countries. These are done manually below

fixSystematicMost_common_list_countries_FULL_MATCH("base", DF_input_base, list_csvs_base, "Bangladesh", "Bd", "France")
fixSystematicMost_common_list_countries_FULL_MATCH("match GLobocan", DF_input_match_GLOBOCAN, list_csvs_match_GLOBOCAN, "Bangladesh", "Bd", "France")

--Fixing base dataset
Parsing csv: parsedX_100000.csv (1/45)
Parsing csv: parsedX_1000000.csv (2/45)
Parsing csv: parsedX_1100000.csv (3/45)
Parsing csv: parsedX_1200000.csv (4/45)
Parsing csv: parsedX_1300000.csv (5/45)
Parsing csv: parsedX_1400000.csv (6/45)
Parsing csv: parsedX_1500000.csv (7/45)
Parsing csv: parsedX_1600000.csv (8/45)
Parsing csv: parsedX_1700000.csv (9/45)
Parsing csv: parsedX_1800000.csv (10/45)
Parsing csv: parsedX_1900000.csv (11/45)
Parsing csv: parsedX_200000.csv (12/45)
Parsing csv: parsedX_2000000.csv (13/45)
Parsing csv: parsedX_2100000.csv (14/45)
Parsing csv: parsedX_2200000.csv (15/45)
Parsing csv: parsedX_2300000.csv (16/45)
Parsing csv: parsedX_2400000.csv (17/45)
Parsing csv: parsedX_2500000.csv (18/45)
Parsing csv: parsedX_2600000.csv (19/45)
Parsing csv: parsedX_2700000.csv (20/45)
Parsing csv: parsedX_2800000.csv (21/45)
Parsing csv: parsedX_2900000.csv (22/45)
Parsing csv: parsedX_300000.csv (23/45)
Parsing csv: parsedX_3000000.csv (24/45)
Parsin

In [221]:
## Fix wrong Country assignment in Most_common_list_countries: Madagascar instead of Brazil in records with MG

# Important: SUBSTRINGS not taken into consideration, corrected only if el["text"] == "MG"

fixSystematicMost_common_list_countries_FULL_MATCH("base", DF_input_base, list_csvs_base, "Madagascar", "MG", "Brazil")
fixSystematicMost_common_list_countries_FULL_MATCH("match GLobocan", DF_input_match_GLOBOCAN, list_csvs_match_GLOBOCAN, "Madagascar", "MG", "Brazil")

--Fixing base dataset
Parsing csv: parsedX_100000.csv (1/45)
Parsing csv: parsedX_1000000.csv (2/45)
Parsing csv: parsedX_1100000.csv (3/45)
Parsing csv: parsedX_1200000.csv (4/45)
Parsing csv: parsedX_1300000.csv (5/45)
Parsing csv: parsedX_1400000.csv (6/45)
Parsing csv: parsedX_1500000.csv (7/45)
Parsing csv: parsedX_1600000.csv (8/45)
Parsing csv: parsedX_1700000.csv (9/45)
Parsing csv: parsedX_1800000.csv (10/45)
Parsing csv: parsedX_1900000.csv (11/45)
Parsing csv: parsedX_200000.csv (12/45)
Parsing csv: parsedX_2000000.csv (13/45)
Parsing csv: parsedX_2100000.csv (14/45)
Parsing csv: parsedX_2200000.csv (15/45)
Parsing csv: parsedX_2300000.csv (16/45)
Parsing csv: parsedX_2400000.csv (17/45)
Parsing csv: parsedX_2500000.csv (18/45)
Parsing csv: parsedX_2600000.csv (19/45)
Parsing csv: parsedX_2700000.csv (20/45)
Parsing csv: parsedX_2800000.csv (21/45)
Parsing csv: parsedX_2900000.csv (22/45)
Parsing csv: parsedX_300000.csv (23/45)
Parsing csv: parsedX_3000000.csv (24/45)
Parsin

In [304]:
## Fix wrong Country assignment in Most_common_list_countries: Mexico instead of Colombia in records with Col

# Important: SUBSTRINGS not taken into consideration, corrected only if el["text"] == "Col"

fixSystematicMost_common_list_countries_FULL_MATCH("base", DF_input_base, list_csvs_base, "Colombia", "Col", "Mexico")
fixSystematicMost_common_list_countries_FULL_MATCH("match GLobocan", DF_input_match_GLOBOCAN, list_csvs_match_GLOBOCAN, "Colombia", "Col", "Mexico")

--Fixing base dataset
Parsing csv: parsedX_100000.csv (1/45)
Parsing csv: parsedX_1000000.csv (2/45)
Parsing csv: parsedX_1100000.csv (3/45)
Parsing csv: parsedX_1200000.csv (4/45)
Parsing csv: parsedX_1300000.csv (5/45)
Parsing csv: parsedX_1400000.csv (6/45)
Parsing csv: parsedX_1500000.csv (7/45)
Parsing csv: parsedX_1600000.csv (8/45)
Parsing csv: parsedX_1700000.csv (9/45)
Parsing csv: parsedX_1800000.csv (10/45)
Parsing csv: parsedX_1900000.csv (11/45)
Parsing csv: parsedX_200000.csv (12/45)
Parsing csv: parsedX_2000000.csv (13/45)
Parsing csv: parsedX_2100000.csv (14/45)
Parsing csv: parsedX_2200000.csv (15/45)
Parsing csv: parsedX_2300000.csv (16/45)
Parsing csv: parsedX_2400000.csv (17/45)
Parsing csv: parsedX_2500000.csv (18/45)
Parsing csv: parsedX_2600000.csv (19/45)
Parsing csv: parsedX_2700000.csv (20/45)
Parsing csv: parsedX_2800000.csv (21/45)
Parsing csv: parsedX_2900000.csv (22/45)
Parsing csv: parsedX_300000.csv (23/45)
Parsing csv: parsedX_3000000.csv (24/45)
Parsin

In [388]:
## Fix wrong Country assignment in Most_common_list_countries: Jamaica instead of Colombia in records with Jamaica

# Important: SUBSTRINGS not taken into consideration, corrected only if el["text"] == "Jamaica"
# Important: some records should be assinged to other countries. These are done manually below

fixSystematicMost_common_list_countries_FULL_MATCH("base", DF_input_base, list_csvs_base, "Jamaica", "Jamaica", "United States")
fixSystematicMost_common_list_countries_FULL_MATCH("match GLobocan", DF_input_match_GLOBOCAN, list_csvs_match_GLOBOCAN, "Jamaica", "Jamaica", "United States")

--Fixing base dataset
Parsing csv: parsedX_100000.csv (1/45)
Parsing csv: parsedX_1000000.csv (2/45)
Parsing csv: parsedX_1100000.csv (3/45)
Parsing csv: parsedX_1200000.csv (4/45)
Parsing csv: parsedX_1300000.csv (5/45)
Parsing csv: parsedX_1400000.csv (6/45)
Parsing csv: parsedX_1500000.csv (7/45)
Parsing csv: parsedX_1600000.csv (8/45)
Parsing csv: parsedX_1700000.csv (9/45)
Parsing csv: parsedX_1800000.csv (10/45)
Parsing csv: parsedX_1900000.csv (11/45)
Parsing csv: parsedX_200000.csv (12/45)
Parsing csv: parsedX_2000000.csv (13/45)
Parsing csv: parsedX_2100000.csv (14/45)
Parsing csv: parsedX_2200000.csv (15/45)
Parsing csv: parsedX_2300000.csv (16/45)
Parsing csv: parsedX_2400000.csv (17/45)
Parsing csv: parsedX_2500000.csv (18/45)
Parsing csv: parsedX_2600000.csv (19/45)
Parsing csv: parsedX_2700000.csv (20/45)
Parsing csv: parsedX_2800000.csv (21/45)
Parsing csv: parsedX_2900000.csv (22/45)
Parsing csv: parsedX_300000.csv (23/45)
Parsing csv: parsedX_3000000.csv (24/45)
Parsin

In [399]:
## Fix wrong Country assignment in Most_common_list_countries: Spain instead of Brazil in records with ES

# Important: SUBSTRINGS not taken into consideration, corrected only if el["text"] == "ES"
# Important: some records should be assinged to other countries. These are done manually below

fixSystematicMost_common_list_countries_FULL_MATCH("base", DF_input_base, list_csvs_base, "Spain", "ES", "Brazil")
fixSystematicMost_common_list_countries_FULL_MATCH("match GLobocan", DF_input_match_GLOBOCAN, list_csvs_match_GLOBOCAN, "Spain", "ES", "Brazil")

--Fixing base dataset
Parsing csv: parsedX_100000.csv (1/45)
Parsing csv: parsedX_1000000.csv (2/45)
Parsing csv: parsedX_1100000.csv (3/45)
Parsing csv: parsedX_1200000.csv (4/45)
Parsing csv: parsedX_1300000.csv (5/45)
Parsing csv: parsedX_1400000.csv (6/45)
Parsing csv: parsedX_1500000.csv (7/45)
Parsing csv: parsedX_1600000.csv (8/45)
Parsing csv: parsedX_1700000.csv (9/45)
Parsing csv: parsedX_1800000.csv (10/45)
Parsing csv: parsedX_1900000.csv (11/45)
Parsing csv: parsedX_200000.csv (12/45)
Parsing csv: parsedX_2000000.csv (13/45)
Parsing csv: parsedX_2100000.csv (14/45)
Parsing csv: parsedX_2200000.csv (15/45)
Parsing csv: parsedX_2300000.csv (16/45)
Parsing csv: parsedX_2400000.csv (17/45)
Parsing csv: parsedX_2500000.csv (18/45)
Parsing csv: parsedX_2600000.csv (19/45)
Parsing csv: parsedX_2700000.csv (20/45)
Parsing csv: parsedX_2800000.csv (21/45)
Parsing csv: parsedX_2900000.csv (22/45)
Parsing csv: parsedX_300000.csv (23/45)
Parsing csv: parsedX_3000000.csv (24/45)
Parsin

In [410]:
## Fix wrong Country assignment in Most_common_list_countries: Serbia instead of Brazil in records with RS

# Important: SUBSTRINGS not taken into consideration, corrected only if el["text"] == "RS"

fixSystematicMost_common_list_countries_FULL_MATCH("base", DF_input_base, list_csvs_base, "Serbia", "RS", "Brazil")
fixSystematicMost_common_list_countries_FULL_MATCH("match GLobocan", DF_input_match_GLOBOCAN, list_csvs_match_GLOBOCAN, "Serbia", "RS", "Brazil")

--Fixing base dataset
Parsing csv: parsedX_100000.csv (1/45)
Parsing csv: parsedX_1000000.csv (2/45)
Parsing csv: parsedX_1100000.csv (3/45)
Parsing csv: parsedX_1200000.csv (4/45)
Parsing csv: parsedX_1300000.csv (5/45)
Parsing csv: parsedX_1400000.csv (6/45)
Parsing csv: parsedX_1500000.csv (7/45)
Parsing csv: parsedX_1600000.csv (8/45)
Parsing csv: parsedX_1700000.csv (9/45)
Parsing csv: parsedX_1800000.csv (10/45)
Parsing csv: parsedX_1900000.csv (11/45)
Parsing csv: parsedX_200000.csv (12/45)
Parsing csv: parsedX_2000000.csv (13/45)
Parsing csv: parsedX_2100000.csv (14/45)
Parsing csv: parsedX_2200000.csv (15/45)
Parsing csv: parsedX_2300000.csv (16/45)
Parsing csv: parsedX_2400000.csv (17/45)
Parsing csv: parsedX_2500000.csv (18/45)
Parsing csv: parsedX_2600000.csv (19/45)
Parsing csv: parsedX_2700000.csv (20/45)
Parsing csv: parsedX_2800000.csv (21/45)
Parsing csv: parsedX_2900000.csv (22/45)
Parsing csv: parsedX_300000.csv (23/45)
Parsing csv: parsedX_3000000.csv (24/45)
Parsin

In [412]:
## Fix wrong Country assignment in Most_common_list_countries: Paraguay instead of Argentina in records with Paraguay

# Important: SUBSTRINGS not taken into consideration, corrected only if el["text"] == "Paraguay"
# Important: some records should be assinged to other countries. These are done manually below

fixSystematicMost_common_list_countries_FULL_MATCH("base", DF_input_base, list_csvs_base, "Paraguay", "Paraguay", "Argentina")
fixSystematicMost_common_list_countries_FULL_MATCH("match GLobocan", DF_input_match_GLOBOCAN, list_csvs_match_GLOBOCAN, "Paraguay", "Paraguay", "Argentina")

--Fixing base dataset
Parsing csv: parsedX_100000.csv (1/45)
Parsing csv: parsedX_1000000.csv (2/45)
Parsing csv: parsedX_1100000.csv (3/45)
Parsing csv: parsedX_1200000.csv (4/45)
Parsing csv: parsedX_1300000.csv (5/45)
Parsing csv: parsedX_1400000.csv (6/45)
Parsing csv: parsedX_1500000.csv (7/45)
Parsing csv: parsedX_1600000.csv (8/45)
Parsing csv: parsedX_1700000.csv (9/45)
Parsing csv: parsedX_1800000.csv (10/45)
Parsing csv: parsedX_1900000.csv (11/45)
Parsing csv: parsedX_200000.csv (12/45)
Parsing csv: parsedX_2000000.csv (13/45)
Parsing csv: parsedX_2100000.csv (14/45)
Parsing csv: parsedX_2200000.csv (15/45)
Parsing csv: parsedX_2300000.csv (16/45)
Parsing csv: parsedX_2400000.csv (17/45)
Parsing csv: parsedX_2500000.csv (18/45)
Parsing csv: parsedX_2600000.csv (19/45)
Parsing csv: parsedX_2700000.csv (20/45)
Parsing csv: parsedX_2800000.csv (21/45)
Parsing csv: parsedX_2900000.csv (22/45)
Parsing csv: parsedX_300000.csv (23/45)
Parsing csv: parsedX_3000000.csv (24/45)
Parsin

In [457]:
## Fix wrong Country assignment in Most_common_list_countries: Slovakia instead of Canada in records with SK

# Important: SUBSTRINGS not taken into consideration, corrected only if el["text"] == "SK"
# Important: some records should be assinged to other countries. These are done manually below

fixSystematicMost_common_list_countries_FULL_MATCH("base", DF_input_base, list_csvs_base, "Slovakia", "SK", "Canada")
fixSystematicMost_common_list_countries_FULL_MATCH("match GLobocan", DF_input_match_GLOBOCAN, list_csvs_match_GLOBOCAN, "Slovakia", "SK", "Canada")

--Fixing base dataset
Parsing csv: parsedX_100000.csv (1/45)
Parsing csv: parsedX_1000000.csv (2/45)
Parsing csv: parsedX_1100000.csv (3/45)
Parsing csv: parsedX_1200000.csv (4/45)
Parsing csv: parsedX_1300000.csv (5/45)
Parsing csv: parsedX_1400000.csv (6/45)
Parsing csv: parsedX_1500000.csv (7/45)
Parsing csv: parsedX_1600000.csv (8/45)
Parsing csv: parsedX_1700000.csv (9/45)
Parsing csv: parsedX_1800000.csv (10/45)
Parsing csv: parsedX_1900000.csv (11/45)
Parsing csv: parsedX_200000.csv (12/45)
Parsing csv: parsedX_2000000.csv (13/45)
Parsing csv: parsedX_2100000.csv (14/45)
Parsing csv: parsedX_2200000.csv (15/45)
Parsing csv: parsedX_2300000.csv (16/45)
Parsing csv: parsedX_2400000.csv (17/45)
Parsing csv: parsedX_2500000.csv (18/45)
Parsing csv: parsedX_2600000.csv (19/45)
Parsing csv: parsedX_2700000.csv (20/45)
Parsing csv: parsedX_2800000.csv (21/45)
Parsing csv: parsedX_2900000.csv (22/45)
Parsing csv: parsedX_300000.csv (23/45)
Parsing csv: parsedX_3000000.csv (24/45)
Parsin

In [10]:
## Fix wrong Country assignment in Most_common_list_countries: Bosnia and Herzegovina instead of Brazil in records with BA

# Important: SUBSTRINGS not taken into consideration, corrected only if el["text"] == "BA"
# Important: some records should be assinged to other countries. These are done manually below

fixSystematicMost_common_list_countries_FULL_MATCH("base", DF_input_base, list_csvs_base, "Bosnia and Herzegovina", "BA", "Brazil")
fixSystematicMost_common_list_countries_FULL_MATCH("match GLobocan", DF_input_match_GLOBOCAN, list_csvs_match_GLOBOCAN, "Bosnia and Herzegovina", "BA", "Brazil")

--Fixing base dataset
Parsing csv: parsedX_100000.csv (1/45)
Parsing csv: parsedX_1000000.csv (2/45)
Parsing csv: parsedX_1100000.csv (3/45)
Parsing csv: parsedX_1200000.csv (4/45)
Parsing csv: parsedX_1300000.csv (5/45)
Parsing csv: parsedX_1400000.csv (6/45)
Parsing csv: parsedX_1500000.csv (7/45)
Parsing csv: parsedX_1600000.csv (8/45)
Parsing csv: parsedX_1700000.csv (9/45)
Parsing csv: parsedX_1800000.csv (10/45)
Parsing csv: parsedX_1900000.csv (11/45)
Parsing csv: parsedX_200000.csv (12/45)
Parsing csv: parsedX_2000000.csv (13/45)
Parsing csv: parsedX_2100000.csv (14/45)
Parsing csv: parsedX_2200000.csv (15/45)
Parsing csv: parsedX_2300000.csv (16/45)
Parsing csv: parsedX_2400000.csv (17/45)
Parsing csv: parsedX_2500000.csv (18/45)
Parsing csv: parsedX_2600000.csv (19/45)
Parsing csv: parsedX_2700000.csv (20/45)
Parsing csv: parsedX_2800000.csv (21/45)
Parsing csv: parsedX_2900000.csv (22/45)
Parsing csv: parsedX_300000.csv (23/45)
Parsing csv: parsedX_3000000.csv (24/45)
Parsin

In [101]:
## Fix wrong Country assignment in Most_common_list_countries: Benin instead of Nigeria in records with Benin City

# Important: SUBSTRINGS not taken into consideration, corrected only if el["text"] == "Benin City"

fixSystematicMost_common_list_countries_FULL_MATCH("base", DF_input_base, list_csvs_base, "Benin", "Benin City", "Nigeria")
fixSystematicMost_common_list_countries_FULL_MATCH("match GLobocan", DF_input_match_GLOBOCAN, list_csvs_match_GLOBOCAN, "Benin", "Benin City", "Nigeria")

--Fixing base dataset
Parsing csv: parsedX_100000.csv (1/45)
Parsing csv: parsedX_1000000.csv (2/45)
Parsing csv: parsedX_1100000.csv (3/45)
Parsing csv: parsedX_1200000.csv (4/45)
Parsing csv: parsedX_1300000.csv (5/45)
Parsing csv: parsedX_1400000.csv (6/45)
Parsing csv: parsedX_1500000.csv (7/45)
Parsing csv: parsedX_1600000.csv (8/45)
Parsing csv: parsedX_1700000.csv (9/45)
Parsing csv: parsedX_1800000.csv (10/45)
Parsing csv: parsedX_1900000.csv (11/45)
Parsing csv: parsedX_200000.csv (12/45)
Parsing csv: parsedX_2000000.csv (13/45)
Parsing csv: parsedX_2100000.csv (14/45)
Parsing csv: parsedX_2200000.csv (15/45)
Parsing csv: parsedX_2300000.csv (16/45)
Parsing csv: parsedX_2400000.csv (17/45)
Parsing csv: parsedX_2500000.csv (18/45)
Parsing csv: parsedX_2600000.csv (19/45)
Parsing csv: parsedX_2700000.csv (20/45)
Parsing csv: parsedX_2800000.csv (21/45)
Parsing csv: parsedX_2900000.csv (22/45)
Parsing csv: parsedX_300000.csv (23/45)
Parsing csv: parsedX_3000000.csv (24/45)
Parsin

In [103]:
## Fix wrong Country assignment in Most_common_list_countries: Bolivia instead of Italy in records with BO

# Important: SUBSTRINGS not taken into consideration, corrected only if el["text"] == "BO"

fixSystematicMost_common_list_countries_FULL_MATCH("base", DF_input_base, list_csvs_base, "Bolivia", "BO", "Italy")
fixSystematicMost_common_list_countries_FULL_MATCH("match GLobocan", DF_input_match_GLOBOCAN, list_csvs_match_GLOBOCAN, "Bolivia", "BO", "Italy")

--Fixing base dataset
Parsing csv: parsedX_100000.csv (1/45)
Parsing csv: parsedX_1000000.csv (2/45)
Parsing csv: parsedX_1100000.csv (3/45)
Parsing csv: parsedX_1200000.csv (4/45)
Parsing csv: parsedX_1300000.csv (5/45)
Parsing csv: parsedX_1400000.csv (6/45)
Parsing csv: parsedX_1500000.csv (7/45)
Parsing csv: parsedX_1600000.csv (8/45)
Parsing csv: parsedX_1700000.csv (9/45)
Parsing csv: parsedX_1800000.csv (10/45)
Parsing csv: parsedX_1900000.csv (11/45)
Parsing csv: parsedX_200000.csv (12/45)
Parsing csv: parsedX_2000000.csv (13/45)
Parsing csv: parsedX_2100000.csv (14/45)
Parsing csv: parsedX_2200000.csv (15/45)
Parsing csv: parsedX_2300000.csv (16/45)
Parsing csv: parsedX_2400000.csv (17/45)
Parsing csv: parsedX_2500000.csv (18/45)
Parsing csv: parsedX_2600000.csv (19/45)
Parsing csv: parsedX_2700000.csv (20/45)
Parsing csv: parsedX_2800000.csv (21/45)
Parsing csv: parsedX_2900000.csv (22/45)
Parsing csv: parsedX_300000.csv (23/45)
Parsing csv: parsedX_3000000.csv (24/45)
Parsin

In [108]:
## Fix wrong Country assignment in Most_common_list_countries: United Kingdom instead of China in records with Ningbo

# Important: SUBSTRINGS not taken into consideration, corrected only if el["text"] == "Ningbo"

fixSystematicMost_common_list_countries_FULL_MATCH("base", DF_input_base, list_csvs_base, "United Kingdom", "Ningbo", "China")
fixSystematicMost_common_list_countries_FULL_MATCH("match GLobocan", DF_input_match_GLOBOCAN, list_csvs_match_GLOBOCAN, "United Kingdom", "Ningbo", "China")

--Fixing base dataset
Parsing csv: parsedX_100000.csv (1/45)
Parsing csv: parsedX_1000000.csv (2/45)
Parsing csv: parsedX_1100000.csv (3/45)
Parsing csv: parsedX_1200000.csv (4/45)
Parsing csv: parsedX_1300000.csv (5/45)
Parsing csv: parsedX_1400000.csv (6/45)
Parsing csv: parsedX_1500000.csv (7/45)
Parsing csv: parsedX_1600000.csv (8/45)
Parsing csv: parsedX_1700000.csv (9/45)
Parsing csv: parsedX_1800000.csv (10/45)
Parsing csv: parsedX_1900000.csv (11/45)
Parsing csv: parsedX_200000.csv (12/45)
Parsing csv: parsedX_2000000.csv (13/45)
Parsing csv: parsedX_2100000.csv (14/45)
Parsing csv: parsedX_2200000.csv (15/45)
Parsing csv: parsedX_2300000.csv (16/45)
Parsing csv: parsedX_2400000.csv (17/45)
Parsing csv: parsedX_2500000.csv (18/45)
Parsing csv: parsedX_2600000.csv (19/45)
Parsing csv: parsedX_2700000.csv (20/45)
Parsing csv: parsedX_2800000.csv (21/45)
Parsing csv: parsedX_2900000.csv (22/45)
Parsing csv: parsedX_300000.csv (23/45)
Parsing csv: parsedX_3000000.csv (24/45)
Parsin

### Correcting *parsedX_100000.csv*

In [21]:
# Read csv
df_parsedX_100000_base = pd.read_csv(DF_input_base + "parsedX_100000.csv")
df_parsedX_100000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_100000.csv")

In [23]:
## Replace found errors by true Country values

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1522964, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1522964, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1318271, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1318271, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1525053, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1525053, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1586297, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1586297, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1703193, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1703193, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1768942, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1768942, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1772998, ["Country", "Country_source"]] = fix_country_from_text("Ireland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1772998, ["Country", "Country_source"]] = fix_country_from_text("Ireland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1874576, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1874576, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1573176, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] ==1573176, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1349273, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1349273, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1375495, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1375495, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1707769, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1707769, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1707770, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1707770, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1390303, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1390303, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1295379, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1295379, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1947740, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1947740, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1300849, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1300849, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1356784, ["Country", "Country_source"]] = fix_country_from_text("Sweden")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1356784, ["Country", "Country_source"]] = fix_country_from_text("Sweden")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1370978, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1370978, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1387621, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1387621, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1409313, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1409313, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1423819, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1423819, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1445797, ["Country", "Country_source"]] = fix_country_from_text("Sweden")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1445797, ["Country", "Country_source"]] = fix_country_from_text("Sweden")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1449159, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1449159, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1458906, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1458906, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1467206, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1467206, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1486561, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1486561, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1486713, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1486713, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1501759, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1501759, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1506818, ["Country", "Country_source"]] = fix_country_from_text("Sweden")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1506818, ["Country", "Country_source"]] = fix_country_from_text("Sweden")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1511046, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1511046, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1526868, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1526868, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1537073, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1537073, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1543545, ["Country", "Country_source"]] = fix_country_from_text("Denmark")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1543545, ["Country", "Country_source"]] = fix_country_from_text("Denmark")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1551310, ["Country", "Country_source"]] = fix_country_from_text("Denmark")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1551310, ["Country", "Country_source"]] = fix_country_from_text("Denmark")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1553897, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1553897, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1577173, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1577173, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1584102, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1584102, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1587763, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1587763, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1591123, ["Country", "Country_source"]] = fix_country_from_text("Sweden")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1591123, ["Country", "Country_source"]] = fix_country_from_text("Sweden")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1606565, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1606565, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1622660, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1622660, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1626970, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1626970, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1647021, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1647021, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1648537, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1648537, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1655272, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1655272, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1657755, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1657755, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1673272, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1673272, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1693592, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1693592, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1694580, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1694580, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1695382, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1695382, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1698662, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1698662, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1702309, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1702309, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1702350, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1702350, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1705275, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1705275, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1708257, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1708257, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1710543, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1710543, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1767140, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1767140, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1328906, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1328906, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1371670, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1371670, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1769875, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1769875, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1356784, ["Country", "Country_source"]] = fix_country_from_text("Sweden")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1356784, ["Country", "Country_source"]] = fix_country_from_text("Sweden")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1300459, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1300459, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1300461, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1300461, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1301332, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1301332, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1347577, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1347577, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1402119, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1402119, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1404975, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1404975, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1479704, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1479704, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1495207, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1495207, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1501373, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1501373, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1532743, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1532743, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1534383, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1534383, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1540939, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1540939, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1570879, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1570879, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1593782, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1593782, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1619978, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1619978, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1646677, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1646677, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1662800, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1662800, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1677185, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1677185, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1700040, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1700040, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1710872, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1710872, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1723963, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1723963, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1725629, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1725629, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1753416, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1753416, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1753433, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1753433, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1758054, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1758054, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1759745, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1759745, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1761901, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1761901, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1786946, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1786946, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1808777, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1808777, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1842316, ["Country", "Country_source"]] = fix_country_from_text("Czechia")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1842316, ["Country", "Country_source"]] = fix_country_from_text("Czechia")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1862441, ["Country", "Country_source"]] = fix_country_from_text("South Africa")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1862441, ["Country", "Country_source"]] = fix_country_from_text("South Africa")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1886755, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1886755, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1886756, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1886756, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1930462, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1930462, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1942559, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1942559, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1943408, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1943408, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1966314, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1966314, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1279262, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1279262, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1279493, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1279493, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1279637, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1279637, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1279638, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1279638, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1279642, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1279642, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1279643, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1279643, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1279747, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1279747, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1280773, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1280773, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1281052, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1281052, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1281064, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1281064, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] ==1281250, ["Country", "Country_source"]] = fix_country_from_text("Sweden")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1281250, ["Country", "Country_source"]] = fix_country_from_text("Sweden")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1282711, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1282711, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1282712, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] ==1282712, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1283005, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1283005, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1283006, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1283006, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1283220, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1283220, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1283234, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1283234, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1283247, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1283247, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1283275, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1283275, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1283282, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1283282, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1283479, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1283479, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1283494, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1283494, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1283539, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1283539, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1283547, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1283547, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1283650, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1283650, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1283685, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1283685, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1283694, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1283694, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1283782, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1283782, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1283967, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1283967, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1284033, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1284033, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1284261, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1284261, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1284295, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1284295, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1284501, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1284501, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1284502, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1284502, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1284666, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1284666, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1284825, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1284825, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1284912, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1284912, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1285425, ["Country", "Country_source"]] = fix_country_from_text("Finland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1285425, ["Country", "Country_source"]] = fix_country_from_text("Finland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1286159, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1286159, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1287424, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1287424, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1287547, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1287547, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1287551, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1287551, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1288821, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1288821, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1288889, ["Country", "Country_source"]] = fix_country_from_text("Czechia")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1288889, ["Country", "Country_source"]] = fix_country_from_text("Czechia")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1289668, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1289668, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1289679, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1289679, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1290353, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1290353, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1290974, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1290974, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1290981, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1290981, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1290986, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1290986, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1290993, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1290993, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1291042, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1291042, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1291294, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1291294, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1291329, ["Country", "Country_source"]] = fix_country_from_text("Hungary")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1291329, ["Country", "Country_source"]] = fix_country_from_text("Hungary")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1291756, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1291756, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1291900, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1291900, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1291904, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1291904, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1291911, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1291911, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1291995, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1291995, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1292964, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1292964, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1293477, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1293477, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1293565, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1293565, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1293582, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1293582, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1293586, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1293586, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1293587, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1293587, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1293903, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1293903, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1293907, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1293907, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1293909, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1293909, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1293910, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1293910, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1293989, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1293989, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1294440, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1294440, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1294502, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1294502, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1294933, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1294933, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1294935, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1294935, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1295123, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1295123, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1295169, ["Country", "Country_source"]] = fix_country_from_text("Tunisia")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1295169, ["Country", "Country_source"]] = fix_country_from_text("Tunisia")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1295234, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1295234, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1295236, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1295236, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1295240, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1295240, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1295242, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1295242, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1296171, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1296171, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1296389, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1296389, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1296855, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1296855, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1297149, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1297149, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1297545, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1297545, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1297619, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1297619, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1298954, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1298954, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1298955, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1298955, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1298956, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1298956, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1298958, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1298958, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1298959, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1298959, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1299042, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1299042, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1299044, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1299044, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1299045, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1299045, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1299046, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1299046, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1299047, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1299047, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1299049, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1299049, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1299406, ["Country", "Country_source"]] = fix_country_from_text("Romania")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1299406, ["Country", "Country_source"]] = fix_country_from_text("Romania")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1300550, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1300550, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1300595, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1300595, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1301111, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1301111, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1301507, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1301507, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1301511, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1301511, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1301512, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1301512, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1301513, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1301513, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1301518, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1301518, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1301519, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1301519, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1301533, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1301533, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1301534, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1301534, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1301535, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1301535, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1301536, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1301536, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1301537, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1301537, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1301538, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1301538, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1301543, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1301543, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1301545, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1301545, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1301555, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1301555, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1301557, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1301557, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1301558, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1301558, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1301718, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1301718, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1302055, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1302055, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1302066, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1302066, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1302112, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1302112, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1302352, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1302352, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1302542, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1302542, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1302721, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1302721, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1303055, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1303055, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1303310, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1303310, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1303883, ["Country", "Country_source"]] = fix_country_from_text("Hungary")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1303883, ["Country", "Country_source"]] = fix_country_from_text("Hungary")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1303919, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1303919, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1303925, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1303925, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1303926, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1303926, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1303927, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1303927, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1303930, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1303930, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1304101, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1304101, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1304503, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1304503, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1304506, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1304506, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1304508, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1304508, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1304510, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1304510, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1304514, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1304514, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1304520, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1304520, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1304530, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1304530, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1304531, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1304531, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1304532, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1304532, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1304533, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1304533, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1304534, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1304534, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1305130, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1305130, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1305132, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1305132, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1305134, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1305134, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1305141, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1305141, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1305145, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1305145, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1305500, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1305500, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1305504, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1305504, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1305505, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1305505, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1305562, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1305562, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1305565, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1305565, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1305574, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1305574, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1305592, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1305592, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1305593, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1305593, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1305602, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1305602, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1305719, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1305719, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1305789, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1305789, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1305790, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1305790, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1306573, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1306573, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1306574, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1306574, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1306894, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1306894, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1307110, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1307110, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1307111, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1307111, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1307984, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1307984, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1307987, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1307987, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1308738, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1308738, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1310884, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1310884, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1311548, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1311548, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1311784, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1311784, ["Country", "Country_source"]] = fix_country_from_text("Japan")

# Taiwan in base, China in match_GLOBOCAN
df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1312381, ["Country", "Country_source"]] = fix_country_from_text("Taiwan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1312381, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1313454, ["Country", "Country_source"]] = fix_country_from_text("New Zealand")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1313454, ["Country", "Country_source"]] = fix_country_from_text("New Zealand")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1313666, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1313666, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1313668, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1313668, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1314060, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1314060, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

# Taiwan in base, China in match_GLOBOCAN
df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1315191, ["Country", "Country_source"]] = fix_country_from_text("Taiwan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1315191, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1315201, ["Country", "Country_source"]] = fix_country_from_text("Taiwan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1315201, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1315602, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1315602, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1315687, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1315687, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1315723, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1315723, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1315724, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1315724, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1315920, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1315920, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1316092, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1316092, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1316759, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1316759, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1317667, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1317667, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1317834, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1317834, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1318179, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1318179, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1318524, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1318524, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1318620, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1318620, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1318697, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1318697, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1318977, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1318977, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1319374, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1319374, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1320006, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1320006, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1320145, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1320145, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1320707, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1320707, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1323327, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1323327, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1323452, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1323452, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1323502, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1323502, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1324467, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1324467, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1325342, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1325342, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1325569, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1325569, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1326101, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1326101, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1326378, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1326378, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1326912, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1326912, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1326926, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1326926, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1327254, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1327254, ["Country", "Country_source"]] = fix_country_from_text("United States")

# Taiwan in base, China in match_GLOBOCAN
df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1327475, ["Country", "Country_source"]] = fix_country_from_text("Taiwan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1327475, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1327672, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1327672, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1327673, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1327673, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1328741, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1328741, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1328809, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1328809, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1328810, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1328810, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1328908, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1328908, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1329093, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1329093, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1329225, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1329225, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1329298, ["Country", "Country_source"]] = fix_country_from_text("Denmark")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1329298, ["Country", "Country_source"]] = fix_country_from_text("Denmark")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1330255, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1330255, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1330864, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1330864, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1331002, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1331002, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1331352, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1331352, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1331562, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1331562, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1331599, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1331599, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1331775, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1331775, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1331930, ["Country", "Country_source"]] = fix_country_from_text("Hungary")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1331930, ["Country", "Country_source"]] = fix_country_from_text("Hungary")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1332130, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1332130, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1332131, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1332131, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1332173, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1332173, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1332267, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1332267, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1333016, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1333016, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1333570, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1333570, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1333591, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1333591, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1334052, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1334052, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1334165, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1334165, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1334167, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1334167, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1334224, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1334224, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1335166, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1335166, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1335198, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1335198, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1335446, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1335446, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1335447, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1335447, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1335525, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1335525, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1336255, ["Country", "Country_source"]] = fix_country_from_text("Serbia")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1336255, ["Country", "Country_source"]] = fix_country_from_text("Serbia")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1336256, ["Country", "Country_source"]] = fix_country_from_text("Serbia")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1336256, ["Country", "Country_source"]] = fix_country_from_text("Serbia")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1336257, ["Country", "Country_source"]] = fix_country_from_text("Serbia")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1336257, ["Country", "Country_source"]] = fix_country_from_text("Serbia")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1336730, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1336730, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1336856, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1336856, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1336858, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1336858, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1336912, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1336912, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1337299, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1337299, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1337381, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1337381, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1337641, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1337641, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1337851, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1337851, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1337852, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1337852, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1337854, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1337854, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1337869, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1337869, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1337949, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1337949, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1338194, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1338194, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1338309, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1338309, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1338349, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1338349, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1338655, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1338655, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1338735, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1338735, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1338736, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1338736, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1339054, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1339054, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1339095, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1339095, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1339109, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1339109, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1339121, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1339121, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1339212, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1339212, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1339275, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1339275, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1340175, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1340175, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1340400, ["Country", "Country_source"]] = fix_country_from_text("Czechia")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1340400, ["Country", "Country_source"]] = fix_country_from_text("Czechia")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1341762, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1341762, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1341870, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1341870, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1342647, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1342647, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1342818, ["Country", "Country_source"]] = fix_country_from_text("Romania")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1342818, ["Country", "Country_source"]] = fix_country_from_text("Romania")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1343017, ["Country", "Country_source"]] = fix_country_from_text("Croatia")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1343017, ["Country", "Country_source"]] = fix_country_from_text("Croatia")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1343029, ["Country", "Country_source"]] = fix_country_from_text("Croatia")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1343029, ["Country", "Country_source"]] = fix_country_from_text("Croatia")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1343054, ["Country", "Country_source"]] = fix_country_from_text("Croatia")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1343054, ["Country", "Country_source"]] = fix_country_from_text("Croatia")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1344349, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1344349, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1344672, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1344672, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1344799, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1344799, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1344875, ["Country", "Country_source"]] = fix_country_from_text("Romania")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1344875, ["Country", "Country_source"]] = fix_country_from_text("Romania")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1345087, ["Country", "Country_source"]] = fix_country_from_text("Senegal")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1345087, ["Country", "Country_source"]] = fix_country_from_text("Senegal")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1345232, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1345232, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1345235, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1345235, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1345242, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1345242, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1345363, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1345363, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1345364, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1345364, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1345530, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1345530, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1345538, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1345538, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1345562, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1345562, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1345570, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1345570, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1346018, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1346018, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1346789, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1346789, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1347638, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1347638, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1347643, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1347643, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1347764, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1347764, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1349659, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1349659, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1350789, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1350789, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1351873, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1351873, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1352440, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1352440, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1352907, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1352907, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1354187, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1354187, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1355008, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1355008, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1355156, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1355156, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1355902, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1355902, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1355948, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1355948, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1357218, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1357218, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1358499, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1358499, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1362307, ["Country", "Country_source"]] = fix_country_from_text("Serbia")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1362307, ["Country", "Country_source"]] = fix_country_from_text("Serbia")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1362852, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1362852, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1363442, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1363442, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1363454, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1363454, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1363520, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1363520, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1363575, ["Country", "Country_source"]] = fix_country_from_text("Hungary")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1363575, ["Country", "Country_source"]] = fix_country_from_text("Hungary")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1363586, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1363586, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1364050, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1364050, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1364127, ["Country", "Country_source"]] = fix_country_from_text("Luxembourg")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1364127, ["Country", "Country_source"]] = fix_country_from_text("Luxembourg")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1364262, ["Country", "Country_source"]] = fix_country_from_text("Romania")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1364262, ["Country", "Country_source"]] = fix_country_from_text("Romania")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1364479, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1364479, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1364498, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1364498, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1364510, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1364510, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1364995, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1364995, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1365007, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1365007, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1365059, ["Country", "Country_source"]] = fix_country_from_text("Serbia")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1365059, ["Country", "Country_source"]] = fix_country_from_text("Serbia")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1365785, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1365785, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1365808, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1365808, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1366156, ["Country", "Country_source"]] = fix_country_from_text("Finland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1366156, ["Country", "Country_source"]] = fix_country_from_text("Finland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1366194, ["Country", "Country_source"]] = fix_country_from_text("Finland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1366194, ["Country", "Country_source"]] = fix_country_from_text("Finland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1366274, ["Country", "Country_source"]] = fix_country_from_text("Finland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1366274, ["Country", "Country_source"]] = fix_country_from_text("Finland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1366628, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1366628, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1367127, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1367127, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1369626, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1369626, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1369630, ["Country", "Country_source"]] = fix_country_from_text("Denmark")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1369630, ["Country", "Country_source"]] = fix_country_from_text("Denmark")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1369859, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1369859, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1369860, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1369860, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1369876, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1369876, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1369922, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1369922, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1369936, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1369936, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1369953, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1369953, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1369954, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1369954, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1369964, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1369964, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1371852, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1371852, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1371986, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1371986, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1373423, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1373423, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1373593, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1373593, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1373677, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1373677, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1373913, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1373913, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1374492, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1374492, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1374728, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1374728, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1374729, ["Country", "Country_source"]] = fix_country_from_text("Israel")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1374729, ["Country", "Country_source"]] = fix_country_from_text("Israel")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1375115, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1375115, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1375412, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1375412, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1375800, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1375800, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1376356, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1376356, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1377337, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1377337, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1377671, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1377671, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1377761, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1377761, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1378584, ["Country", "Country_source"]] = fix_country_from_text("Hungary")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1378584, ["Country", "Country_source"]] = fix_country_from_text("Hungary")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1378655, ["Country", "Country_source"]] = fix_country_from_text("Norway")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1378655, ["Country", "Country_source"]] = fix_country_from_text("Norway")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1378656, ["Country", "Country_source"]] = fix_country_from_text("Norway")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1378656, ["Country", "Country_source"]] = fix_country_from_text("Norway")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1378657, ["Country", "Country_source"]] = fix_country_from_text("Norway")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1378657, ["Country", "Country_source"]] = fix_country_from_text("Norway")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1379120, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1379120, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1379365, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1379365, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1379651, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1379651, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1379653, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1379653, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1379768, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1379768, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1380676, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1380676, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1380743, ["Country", "Country_source"]] = fix_country_from_text("Denmark")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1380743, ["Country", "Country_source"]] = fix_country_from_text("Denmark")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1381064, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1381064, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1381821, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1381821, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1381823, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1381823, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1381864, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1381864, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1381945, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1381945, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1382396, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1382396, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1383259, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1383259, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1383585, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1383585, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1383652, ["Country", "Country_source"]] = fix_country_from_text("Sweden")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1383652, ["Country", "Country_source"]] = fix_country_from_text("Sweden")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1383904, ["Country", "Country_source"]] = fix_country_from_text("Hungary")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1383904, ["Country", "Country_source"]] = fix_country_from_text("Hungary")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1384296, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1384296, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1384832, ["Country", "Country_source"]] = fix_country_from_text("Croatia")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1384832, ["Country", "Country_source"]] = fix_country_from_text("Croatia")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1385050, ["Country", "Country_source"]] = fix_country_from_text("Denmark")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1385050, ["Country", "Country_source"]] = fix_country_from_text("Denmark")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1385800, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1385800, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1388039, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1388039, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1388459, ["Country", "Country_source"]] = fix_country_from_text("Puerto Rico")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1388459, ["Country", "Country_source"]] = fix_country_from_text("Puerto Rico")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1388469, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1388469, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1390421, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1390421, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1390848, ["Country", "Country_source"]] = fix_country_from_text("Denmark")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1390848, ["Country", "Country_source"]] = fix_country_from_text("Denmark")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1393642, ["Country", "Country_source"]] = fix_country_from_text("Czechia")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1393642, ["Country", "Country_source"]] = fix_country_from_text("Czechia")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1393705, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1393705, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1394114, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1394114, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1394484, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1394484, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1394992, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1394992, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1395034, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1395034, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1395863, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1395863, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1398423, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1398423, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1401763, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1401763, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1401992, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1401992, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1403359, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1403359, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1404866, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1404866, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1404964, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1404964, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1404971, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1404971, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1405166, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1405166, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1405400, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1405400, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1406237, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1406237, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1407281, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1407281, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1407282, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1407282, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1407283, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1407283, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1407284, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1407284, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1407285, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1407285, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1407287, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1407287, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1407288, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1407288, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1407289, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1407289, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1407304, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1407304, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1407315, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1407315, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1407318, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1407318, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1407319, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1407319, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1407323, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1407323, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1407329, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1407329, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1407334, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1407334, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1407336, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1407336, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1407341, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1407341, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1407342, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1407342, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1408202, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1408202, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1408206, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1408206, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1408213, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1408213, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1408214, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1408214, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1408691, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1408691, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1408730, ["Country", "Country_source"]] = fix_country_from_text("Norway")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1408730, ["Country", "Country_source"]] = fix_country_from_text("Norway")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1408996, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1408996, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1409032, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1409032, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1409039, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1409039, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1410913, ["Country", "Country_source"]] = fix_country_from_text("Romania")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1410913, ["Country", "Country_source"]] = fix_country_from_text("Romania")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1411925, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1411925, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1412051, ["Country", "Country_source"]] = fix_country_from_text("Denmark")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1412051, ["Country", "Country_source"]] = fix_country_from_text("Denmark")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1412128, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1412128, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1412285, ["Country", "Country_source"]] = fix_country_from_text("Norway")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1412285, ["Country", "Country_source"]] = fix_country_from_text("Norway")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1412286, ["Country", "Country_source"]] = fix_country_from_text("Norway")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1412286, ["Country", "Country_source"]] = fix_country_from_text("Norway")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1413180, ["Country", "Country_source"]] = fix_country_from_text("Denmark")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1413180, ["Country", "Country_source"]] = fix_country_from_text("Denmark")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1413194, ["Country", "Country_source"]] = fix_country_from_text("Denmark")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1413194, ["Country", "Country_source"]] = fix_country_from_text("Denmark")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1413228, ["Country", "Country_source"]] = fix_country_from_text("Denmark")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1413228, ["Country", "Country_source"]] = fix_country_from_text("Denmark")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1413789, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1413789, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1413797, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1413797, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1413803, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1413803, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1413859, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1413859, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1413969, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1413969, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1414316, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1414316, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1414553, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1414553, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1414747, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1414747, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1414758, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1414758, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1417004, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1417004, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1417015, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1417015, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1419760, ["Country", "Country_source"]] = fix_country_from_text("Ireland")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1419760, ["Country", "Country_source"]] = fix_country_from_text("Ireland")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1419951, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1419951, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1420064, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1420064, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1420110, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1420110, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1420124, ["Country", "Country_source"]] = fix_country_from_text("Romania")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1420124, ["Country", "Country_source"]] = fix_country_from_text("Romania")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1420127, ["Country", "Country_source"]] = fix_country_from_text("Romania")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1420127, ["Country", "Country_source"]] = fix_country_from_text("Romania")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1420657, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1420657, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1423263, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1423263, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1424076, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1424076, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1424975, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1424975, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1426098, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1426098, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1427288, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1427288, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1427471, ["Country", "Country_source"]] = fix_country_from_text("Israel")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1427471, ["Country", "Country_source"]] = fix_country_from_text("Israel")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1430267, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1430267, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1431203, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1431203, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1431486, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1431486, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1433935, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1433935, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1433946, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1433946, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1433953, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1433953, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1434237, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1434237, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1434273, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1434273, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1312009, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1312009, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1441583, ["Country", "Country_source"]] = fix_country_from_text("Saudi Arabia")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1441583, ["Country", "Country_source"]] = fix_country_from_text("Saudi Arabia")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1592412, ["Country", "Country_source"]] = fix_country_from_text("Saudi Arabia")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1592412, ["Country", "Country_source"]] = fix_country_from_text("Saudi Arabia")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1665968, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1665968, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1700955, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1700955, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == 1963384, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == 1963384, ["Country", "Country_source"]] = fix_country_from_text("Russia")

#df_parsedX_100000_base.loc[df_parsedX_100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_100000_match_GLOBOCAN.loc[df_parsedX_100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [24]:
df_parsedX_100000_base.to_csv(DF_input_base + "parsedX_100000.csv", index = False)
df_parsedX_100000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_100000.csv", index = False)

del df_parsedX_100000_base
del df_parsedX_100000_match_GLOBOCAN

### Correcting *parsedX_1000000.csv*

In [25]:
# Read csv
df_parsedX_1000000_base = pd.read_csv(DF_input_base + "parsedX_1000000.csv")
df_parsedX_1000000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_1000000.csv")

In [26]:
## Replace found errors by true Country values

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11721110, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11721110, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11309027, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11309027, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11450918, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11450918, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11468033, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11468033, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11593153, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11593153, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11680021, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11680021, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11917379, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11917379, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11956310, ["Country", "Country_source"]] = fix_country_from_text("Ireland")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11956310, ["Country", "Country_source"]] = fix_country_from_text("Ireland")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 12016415, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 12016415, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 12058131, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 12058131, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 12108281, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 12108281, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11688359, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11688359, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11462953, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11462953, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11787374, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11787374, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11818943, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11818943, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11836591, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11836591, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 12063738, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 12063738, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11292140, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11292140, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11716229, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11716229, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 12026781, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 12026781, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 12057873, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 12057873, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11820680, ["Country", "Country_source"]] = fix_country_from_text("Denmark")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11820680, ["Country", "Country_source"]] = fix_country_from_text("Denmark")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11745825, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11745825, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11374632, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11374632, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11715595, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11715595, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11299736, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11299736, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11346461, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11346461, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11392669, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11392669, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11525795, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11525795, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11585860, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11585860, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11585862, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11585862, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11689289, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11689289, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11831358, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11831358, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11856476, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11856476, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11957087, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11957087, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11957150, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11957150, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11971643, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11971643, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 12051806, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 12051806, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 12065087, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 12065087, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11274560, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11274560, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11312188, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11312188, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11373238, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11373238, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11391258, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11391258, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11394840, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11394840, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11475067, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11475067, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11550299, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11550299, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11641151, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11641151, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11641168, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11641168, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11704885, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11704885, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11713294, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11713294, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11775520, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11775520, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11792503, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11792503, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11803655, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11803655, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11818600, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11818600, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11867778, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11867778, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11867808, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11867808, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11886351, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11886351, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11906872, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11906872, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11921866, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11921866, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 12100775, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 12100775, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11678667, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11678667, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11322142, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11322142, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11337012, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11337012, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == 11692950	, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == 11692950	, ["Country", "Country_source"]] = fix_country_from_text("Spain")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1000000_base.loc[df_parsedX_1000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1000000_match_GLOBOCAN.loc[df_parsedX_1000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [27]:
df_parsedX_1000000_base.to_csv(DF_input_base + "parsedX_1000000.csv", index = False)
df_parsedX_1000000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_1000000.csv", index = False)

del df_parsedX_1000000_base
del df_parsedX_1000000_match_GLOBOCAN

### Correcting *parsedX_1100000.csv*

In [28]:
# Read csv
df_parsedX_1100000_base = pd.read_csv(DF_input_base + "parsedX_1100000.csv")
df_parsedX_1100000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_1100000.csv")

In [29]:
## Replace found errors by true Country values

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12957547, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12957547, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12355183, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12355183, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12161378, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12161378, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12506764, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12506764, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12621853, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12621853, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12422048, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12422048, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12538984, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12538984, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12873172, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12873172, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12145848, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12145848, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12205789, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12205789, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12846424, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12846424, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12972519, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12972519, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12508755, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12508755, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12713186, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12713186, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12239967, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12239967, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12483128, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12483128, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12860824, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12860824, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12943194, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12943194, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12512967, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12512967, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12613152, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12613152, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12709653, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12709653, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12821934, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12821934, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12829030, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12829030, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12124465, ["Country", "Country_source"]] = fix_country_from_text("Sweden")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12124465, ["Country", "Country_source"]] = fix_country_from_text("Sweden")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12375037, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12375037, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12846497, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12846497, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12186347, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12186347, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12772990, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12772990, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12121834, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12121834, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12469273, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12469273, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 14508103, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 14508103, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12778306, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12778306, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12208572, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12208572, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12880155, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12880155, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12126097, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12126097, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12130459, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12130459, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12151355, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12151355, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12271438, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12271438, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12366719, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12366719, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12427434, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12427434, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12465387, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12465387, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12553031, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12553031, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12600411, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12600411, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12653256, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12653256, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12680153, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12680153, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12692952, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12692952, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12788315, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12788315, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12826309, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12826309, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12829003, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12829003, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12860469, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12860469, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12886135, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12886135, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12118378, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12118378, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12130443, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12130443, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12204674, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12204674, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12210601, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12210601, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12354987, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12354987, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12381480, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12381480, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12388495, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12388495, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12461296, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12461296, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12490981, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12490981, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12525194, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12525194, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12530055, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12530055, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12540430, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12540430, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12601184, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12601184, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12633840, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12633840, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12685660, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12685660, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12700131, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12700131, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12783605, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12783605, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12818835, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12818835, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12844367, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12844367, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12853663, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12853663, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12930079, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12930079, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12932929, ["Country", "Country_source"]] = fix_country_from_text("Israel")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12932929, ["Country", "Country_source"]] = fix_country_from_text("Israel")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12958549, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12958549, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12570061, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12570061, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12618671, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12618671, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12772187, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12772187, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12821385, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12821385, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12400953, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12400953, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12479062, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12479062, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == 12557517, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == 12557517, ["Country", "Country_source"]] = fix_country_from_text("China")

#df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1100000_base.loc[df_parsedX_1100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1100000_match_GLOBOCAN.loc[df_parsedX_1100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [30]:
df_parsedX_1100000_base.to_csv(DF_input_base + "parsedX_1100000.csv", index = False)
df_parsedX_1100000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_1100000.csv", index = False)

del df_parsedX_1100000_base
del df_parsedX_1100000_match_GLOBOCAN

### Correcting *parsedX_1200000.csv*

In [31]:
# Read csv
df_parsedX_1200000_base = pd.read_csv(DF_input_base + "parsedX_1200000.csv")
df_parsedX_1200000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_1200000.csv")

In [32]:
## Replace found errors by true Country values

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15481700, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15481700, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 14696430, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 14696430, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15067206, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15067206, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 14586258, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 14586258, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 14600589, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 14600589, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 14968015, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 14968015, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15022781, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15022781, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15027374, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15027374, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15036243, ["Country", "Country_source"]] = fix_country_from_text("Ireland")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15036243, ["Country", "Country_source"]] = fix_country_from_text("Ireland")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15039608, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15039608, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15204374, ["Country", "Country_source"]] = fix_country_from_text("Ireland")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15204374, ["Country", "Country_source"]] = fix_country_from_text("Ireland")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15204423, ["Country", "Country_source"]] = fix_country_from_text("Ireland")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15204423, ["Country", "Country_source"]] = fix_country_from_text("Ireland")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15351012, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15351012, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15357755, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15357755, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15365563, ["Country", "Country_source"]] = fix_country_from_text("Ireland")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15365563, ["Country", "Country_source"]] = fix_country_from_text("Ireland")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15380568, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15380568, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15545939, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15545939, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15052176, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15052176, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 14717137, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 14717137, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 14571433, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 14571433, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 14725654, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 14725654, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15485203, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15485203, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15535599, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15535599, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 14766271, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 14766271, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 14605625, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 14605625, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 14690004, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 14690004, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 14719847, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 14719847 , ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 14990209, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 14990209, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15166878, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15166878, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15190250, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15190250, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15542931, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15542931, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15468691, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15468691, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15472714, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15472714, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15232311, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15232311, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 14614324, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 14614324, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15534620, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15534620, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15149080, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15149080, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15293670, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15293670, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15370683, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15370683, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 14574513, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 14574513, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 14586750, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 14586750, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 14728590, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 14728590, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15052456, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15052456, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15061261, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15061261, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15063149, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15063149, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15072586, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15072586, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15110228, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15110228, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15170673, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15170673, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15178352, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15178352, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15182427, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15182427, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15183528, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15183528, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15183531, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15183531, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15195751, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15195751, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15459021, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15459021, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15471671, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15471671, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 14574001, ["Country", "Country_source"]] = fix_country_from_text("Israel")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 14574001, ["Country", "Country_source"]] = fix_country_from_text("Israel")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 14593775, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 14593775, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 14619605, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 14619605, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 14661438, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 14661438, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 14706924, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 14706924, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 14722482, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 14722482, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 14738240, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 14738240, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 14752193, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 14752193, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 14985491, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 14985491, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 14990839, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 14990839, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15006540, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15006540, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15026141, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15026141, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15044255, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15044255, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15047380, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15047380, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15082088, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15082088, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15090276, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15090276, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15143160, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15143160, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15171812, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15171812, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15219496, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15219496, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15223312, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15223312, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15238167, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15238167, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15280468, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15280468, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15289300, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15289300, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15304390, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15304390, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15310767, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15310767, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15320718, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15320718, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15367573, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15367573, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15367686, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15367686, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15456869, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15456869, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15461858, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15461858, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15474442, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15474442, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15509721, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15509721, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15537980, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15537980, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 14561262, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 14561262, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 14561263, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 14561263, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 14564245, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 14564245, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15359655, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15359655, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15010194, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15010194, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15192967, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15192967, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15262439, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15262439, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 14564821, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 14564821, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 14996395, ["Country", "Country_source"]] = fix_country_from_text("Syria")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 14996395, ["Country", "Country_source"]] = fix_country_from_text("Syria")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15087042, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15087042, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == 15250765, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == 15250765, ["Country", "Country_source"]] = fix_country_from_text("China")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1200000_base.loc[df_parsedX_1200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1200000_match_GLOBOCAN.loc[df_parsedX_1200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [33]:
df_parsedX_1200000_base.to_csv(DF_input_base + "parsedX_1200000.csv", index = False)
df_parsedX_1200000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_1200000.csv", index = False)

del df_parsedX_1200000_base
del df_parsedX_1200000_match_GLOBOCAN

### Correcting *parsedX_1300000.csv*

In [34]:
# Read csv
df_parsedX_1300000_base = pd.read_csv(DF_input_base + "parsedX_1300000.csv")
df_parsedX_1300000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_1300000.csv")

In [35]:
## Replace found errors by true Country values

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 16267798, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 16267798, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15942562, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15942562, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15692413, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15692413, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15731292, ["Country", "Country_source"]] = fix_country_from_text("Ireland")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15731292, ["Country", "Country_source"]] = fix_country_from_text("Ireland")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15734139, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15734139, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15748131, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15748131, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15808448, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15808448, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15810157, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15810157, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 16039103, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 16039103, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 16129520, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 16129520, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 16134302, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 16134302, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 16181454, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 16181454, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15771949, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15771949, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 16313961, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 16313961, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 16238194, ["Country", "Country_source"]] = fix_country_from_text("Romania")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 16238194, ["Country", "Country_source"]] = fix_country_from_text("Romania")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15693138, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15693138, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15893285, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15893285, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 16010794, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 16010794, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 16203811, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 16203811, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15565382, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15565382, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15839470, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15839470, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15883785, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15883785, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15912426, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15912426, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15925994, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15925994, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 16040004, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 16040004, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 16143088, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 16143088, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 16172016, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 16172016, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 16284766, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 16284766, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15896501, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15896501, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15907365, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15907365, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15928562, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15928562, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 16033738, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 16033738, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 16043670, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 16043670, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 16215812, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 16215812, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 16238143, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 16238143, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 16249791, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 16249791, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15907589, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15907589, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15570129, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15570129, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15680664, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15680664, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 16133534, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 16133534, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15688377, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15688377, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15723650, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15723650, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15736053, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15736053, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15751122, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15751122, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15848042, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15848042, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15892897, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15892897, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15929075, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15929075, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15991841, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15991841, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 16043954, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 16043954, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 16049979, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 16049979, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 16081511, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 16081511, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 16133359, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 16133359, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 16133366, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 16133366, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 16175472, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 16175472, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 16200895, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 16200895, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 16219633, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 16219633, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15583013, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15583013, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15593349, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15593349, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15603536, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15603536, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15728131, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15728131, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15737872, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15737872, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15742256, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15742256, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15743682, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15743682, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15743806, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15743806, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15800709, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15800709, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15831701, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15831701, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15836967, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15836967, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15840790, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15840790, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15842999, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15842999, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15857509, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15857509, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15858077, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15858077, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15870290, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15870290, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15927067, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15927067, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15939586, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15939586, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15984700, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15984700, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15987430, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15987430, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 16127634, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 16127634, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 16157293, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 16157293, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 16164925, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 16164925, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 16164933, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 16164933, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 16183933, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 16183933, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 16219545, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 16219545, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 16230047, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 16230047, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 16280041, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 16280041, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 16044687, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 16044687, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15596293, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15596293, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15713519, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15713519, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15795242, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15795242, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15888262, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15888262, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 16023763, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 16023763, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 16213362, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 16213362, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15767962, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15767962, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 16309230, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 16309230, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 16184700, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 16184700, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 15748562, ["Country", "Country_source"]] = fix_country_from_text("Syria")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 15748562, ["Country", "Country_source"]] = fix_country_from_text("Syria")

df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == 16188352, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == 16188352, ["Country", "Country_source"]] = fix_country_from_text("China")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1300000_base.loc[df_parsedX_1300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1300000_match_GLOBOCAN.loc[df_parsedX_1300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [36]:
df_parsedX_1300000_base.to_csv(DF_input_base + "parsedX_1300000.csv", index = False)
df_parsedX_1300000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_1300000.csv", index = False)

del df_parsedX_1300000_base
del df_parsedX_1300000_match_GLOBOCAN

### Correcting *parsedX_1400000.csv*

In [37]:
# Read csv
df_parsedX_1400000_base = pd.read_csv(DF_input_base + "parsedX_1400000.csv")
df_parsedX_1400000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_1400000.csv")

In [38]:
## Replace found errors by true Country values

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16715323, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16715323, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 17007025, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 17007025, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16340834, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16340834, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16482873, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16482873, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16642235, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16642235, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16697618, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16697618, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 17030388, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 17030388, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 17091098, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 17091098, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16760298, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16760298, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16388370, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] ==16388370 , ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16701496, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16701496, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 17171637, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 17171637, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16892552, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16892552, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16567022, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16567022, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16696934, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16696934, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16715760, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16715760, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16538218, ["Country", "Country_source"]] = fix_country_from_text("Sweden")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16538218, ["Country", "Country_source"]] = fix_country_from_text("Sweden")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16372487, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] ==16372487, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16410589, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] ==16410589 , ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16942641, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16942641, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 17007014, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 17007014, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16419064, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16419064, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 17010186, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 17010186, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16407624, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16407624, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 17016635, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 17016635, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16683388, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16683388, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16872799, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16872799, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16819688, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16819688, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16614876, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16614876, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16365683, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16365683, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16394290, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16394290, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16395701, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16395701, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16493531, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16493531, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16515943, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16515943, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16570353, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16570353, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16601012, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16601012, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16616664, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16616664, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16643879, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16643879, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16704527, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16704527, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16935975, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16935975, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16959434, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16959434, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 17012850, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 17012850, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 17016582, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 17016582, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 17049467, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 17049467, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 17053293, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 17053293, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 17111235, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 17111235, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16357363, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16357363, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16399897, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16399897, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16404370, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16404370, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16419187, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16419187, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16421588, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16421588, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16426843, ["Country", "Country_source"]] = fix_country_from_text("Israel")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16426843, ["Country", "Country_source"]] = fix_country_from_text("Israel")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16458195, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16458195, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16478428, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16478428, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16482447, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16482447, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16517773, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16517773, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16583981, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16583981, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16647352, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16647352, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16705174, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16705174, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16705457, ["Country", "Country_source"]] = fix_country_from_text("Egypt")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16705457, ["Country", "Country_source"]] = fix_country_from_text("Egypt")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16730930, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16730930, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16775349, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16775349, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16792817, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16792817, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16805920, ["Country", "Country_source"]] = fix_country_from_text("Ireland")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16805920, ["Country", "Country_source"]] = fix_country_from_text("Ireland")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16808852, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16808852, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16832677, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16832677, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16849592, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16849592, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16877361, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16877361, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16942611, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16942611, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16978951, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16978951, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16984691, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16984691, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16990665, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16990665, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 17018578, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 17018578, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 17036566, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 17036566, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 17066774, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 17066774, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 17071576, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 17071576, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 17102042, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 17102042, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 17102978, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 17102978, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 17114539, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 17114539, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 17131121, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 17131121, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16359366, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16359366, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16982072, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16982072, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16343481, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16343481, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16504883, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16504883, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16537699, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16537699, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16803519, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16803519, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16852046, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16852046, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16959370, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16959370, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 17022892, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 17022892, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 17042487, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 17042487, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 17112680, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 17112680, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16460443, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16460443, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16971196, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16971196, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 17139403, ["Country", "Country_source"]] = fix_country_from_text("Brunei Darussalam")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 17139403, ["Country", "Country_source"]] = fix_country_from_text("Brunei Darussalam")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16755073, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16755073, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 16989516, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 16989516, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == 17083872, ["Country", "Country_source"]] = fix_country_from_text("Syria")
df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == 17083872, ["Country", "Country_source"]] = fix_country_from_text("Syria")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1400000_base.loc[df_parsedX_1400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1400000_match_GLOBOCAN.loc[df_parsedX_1400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [39]:
df_parsedX_1400000_base.to_csv(DF_input_base + "parsedX_1400000.csv", index = False)
df_parsedX_1400000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_1400000.csv", index = False)

del df_parsedX_1400000_base
del df_parsedX_1400000_match_GLOBOCAN

### Correcting *parsedX_1500000.csv*

In [40]:
# Read csv
df_parsedX_1500000_base = pd.read_csv(DF_input_base + "parsedX_1500000.csv")
df_parsedX_1500000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_1500000.csv")

In [41]:
## Replace found errors by true Country values

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17406831, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17406831, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17927986, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17927986, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17383523, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17383523, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17389413, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17389413, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17479108, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17479108, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17596859, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17596859, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17702593, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17702593, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17932053, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17932053, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 18080419, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 18080419, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17347637, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17347637, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17240633, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17240633, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17319790, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17319790, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17394837, ["Country", "Country_source"]] = fix_country_from_text("Denmark")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17394837, ["Country", "Country_source"]] = fix_country_from_text("Denmark")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17870629, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17870629, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17955256 , ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17955256 , ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17236946, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17236946, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17430760, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17430760, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17618801, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17618801, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17414499, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17414499, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17454361, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17454361, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17998653, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17998653, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17919432, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17919432, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17481704, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17481704, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17470161, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17470161, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17589568, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17589568, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17217918, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17217918, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17235569, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17235569, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17272286, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17272286, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17331170, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17331170, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17410583, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17410583, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17417780, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17417780, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17468826, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17468826, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17506908, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17506908, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17531078, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17531078, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17557291, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17557291, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17571970, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17571970, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17599625, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17599625, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17655966, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17655966, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17657716, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17657716, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17728072, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17728072, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17976452, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17976452, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 18037575, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 18037575, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17189608, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17189608, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17261174, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17261174, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17312187, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17312187, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17317227, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17317227, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17336103, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17336103, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17376329, ["Country", "Country_source"]] = fix_country_from_text("Portugal")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17376329, ["Country", "Country_source"]] = fix_country_from_text("Portugal")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17385008, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17385008, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17561080, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17561080, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17615985, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17615985, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17631914, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17631914, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17668300, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17668300, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17692120, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17692120, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17692334, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17692334, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17694514, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17694514, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17722646, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17722646, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17867389, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17867389, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17909818, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17909818, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17937802, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17937802, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17979493, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17979493, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17980055, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17980055, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17990362, ["Country", "Country_source"]] = fix_country_from_text("Greece")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17990362, ["Country", "Country_source"]] = fix_country_from_text("Greece")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 18045465, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 18045465, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 18079680, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 18079680, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17284750, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17284750, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17254669, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17254669, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17342320, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17342320, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17896919, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17896919, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17434159, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17434159, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17668204, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17668204, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17922950, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17922950, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 18053356, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 18053356, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 18053362, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 18053362, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 18053364, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 18053364, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17315792, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17315792, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17310287, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17310287, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17554735, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17554735, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == 17722185, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == 17722185, ["Country", "Country_source"]] = fix_country_from_text("China")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1500000_base.loc[df_parsedX_1500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1500000_match_GLOBOCAN.loc[df_parsedX_1500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [42]:
df_parsedX_1500000_base.to_csv(DF_input_base + "parsedX_1500000.csv", index = False)
df_parsedX_1500000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_1500000.csv", index = False)

del df_parsedX_1500000_base
del df_parsedX_1500000_match_GLOBOCAN

### Correcting *parsedX_1600000.csv*

In [43]:
# Read csv
df_parsedX_1600000_base = pd.read_csv(DF_input_base + "parsedX_1600000.csv")
df_parsedX_1600000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_1600000.csv")

In [44]:
## Replace found errors by true Country values

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18228178, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18228178, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 19011482, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 19011482, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18365760, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18365760, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18977715, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18977715, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18408385, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18408385, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18380311, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18380311, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18456062, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18456062, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18538271, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18538271, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18557535, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18557535, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18588522, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18588522, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18604657, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18604657, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18656282, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18656282, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18807605, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18807605, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 19031932, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 19031932, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18502168, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18502168, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 19016421, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 19016421, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18603738, ["Country", "Country_source"]] = fix_country_from_text("Iran")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18603738, ["Country", "Country_source"]] = fix_country_from_text("Iran")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18990025, ["Country", "Country_source"]] = fix_country_from_text("Iran")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18990025, ["Country", "Country_source"]] = fix_country_from_text("Iran")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18945211, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18945211, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18293907, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18293907, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 19020783, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 19020783, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18803871, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18803871, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18954760, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18954760, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18793269, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18793269, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18162345, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18162345, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18358106, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18358106, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18569287, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18569287, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18604658, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18604658, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18604659, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18604659, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18824095, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18824095, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18950215, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18950215, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18666318, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18666318, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18264209, ["Country", "Country_source"]] = fix_country_from_text("New Zealand")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18264209, ["Country", "Country_source"]] = fix_country_from_text("New Zealand")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18813779, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18813779, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18363871, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18363871, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18294668, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18294668, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18397471, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18397471, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18483618, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18483618, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18547741, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18547741, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18579255, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18579255, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18704666, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18704666, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18844938, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18844938, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18853477, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18853477, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18853746, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18853746, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 19003583, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 19003583, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 19003968, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 19003968, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 19028472, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 19028472, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18087007, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18087007, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18091388, ["Country", "Country_source"]] = fix_country_from_text("Greece")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18091388, ["Country", "Country_source"]] = fix_country_from_text("Greece")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18179693, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18179693, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18186930, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18186930, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18208587, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18208587, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18211746, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18211746, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18227155, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18227155, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18279521, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18279521, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18287436, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18287436, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18373870, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18373870, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18375898, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18375898, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18378905, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18378905, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18403589, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18403589, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18430835, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18430835, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18439062, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18439062, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18440462, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18440462, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18442884, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18442884, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18461285, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18461285, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18468080, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18468080, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18489764, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18489764, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18490593, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18490593, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18510722, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18510722, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18543006, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18543006, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18550181, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18550181, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18578862, ["Country", "Country_source"]] = fix_country_from_text("Ireland")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18578862, ["Country", "Country_source"]] = fix_country_from_text("Ireland")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18645123, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18645123, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18648954, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18648954, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18683894, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18683894, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18700021, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18700021, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18710507, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18710507, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18817561, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18817561, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18819711, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18819711, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18841754, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18841754, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18842429, ["Country", "Country_source"]] = fix_country_from_text("Portugal")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18842429, ["Country", "Country_source"]] = fix_country_from_text("Portugal")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18925701, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18925701, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18974381, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18974381, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18988376, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18988376, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 19020218, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 19020218, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18192578, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18192578, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18534614, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18534614, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18178686, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18178686, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18288922, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18288922, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18442289, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18442289, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18508272, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18508272, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18207203, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18207203, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18210196, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18210196, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18647433, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18647433, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18647554, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18647554, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18766473, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18766473, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18246671, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18246671, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18570662, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18570662, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18575777, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18575777, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18702047, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18702047, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == 18710011, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == 18710011, ["Country", "Country_source"]] = fix_country_from_text("Japan")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1600000_base.loc[df_parsedX_1600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1600000_match_GLOBOCAN.loc[df_parsedX_1600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [45]:
df_parsedX_1600000_base.to_csv(DF_input_base + "parsedX_1600000.csv", index = False)
df_parsedX_1600000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_1600000.csv", index = False)

del df_parsedX_1600000_base
del df_parsedX_1600000_match_GLOBOCAN

### Correcting *parsedX_1700000.csv*

In [46]:
# Read csv
df_parsedX_1700000_base = pd.read_csv(DF_input_base + "parsedX_1700000.csv")
df_parsedX_1700000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_1700000.csv")

In [47]:
## Replace found errors by true Country values

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19046130, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19046130, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19051189, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19051189, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19061466, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19061466, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19075591, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19075591, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19126291, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19126291, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19168018, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19168018, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19178759, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19178759, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19268208, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19268208, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19326171, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19326171, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19479418, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19479418, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19647630, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19647630, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19760114, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19760114, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19801219, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19801219, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19824281, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19824281, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19683251, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19683251, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19101895, ["Country", "Country_source"]] = fix_country_from_text("Iran")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19101895, ["Country", "Country_source"]] = fix_country_from_text("Iran")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19256743, ["Country", "Country_source"]] = fix_country_from_text("Iran")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19256743, ["Country", "Country_source"]] = fix_country_from_text("Iran")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19527768, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19527768, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19793167, ["Country", "Country_source"]] = fix_country_from_text("Iran")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19793167, ["Country", "Country_source"]] = fix_country_from_text("Iran")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19332022, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19332022, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19754825, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19754825, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19135106, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19135106, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19501160, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19501160, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19559379, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19559379, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19579023, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19579023, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19186231, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19186231, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19341448, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19341448, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19539896, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19539896, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19423036, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19423036, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19132508, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19132508, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19439861, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19439861, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19460457, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19460457, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19578735, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19578735, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19288002, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19288002, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19041193, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19041193, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19140851, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19140851, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19159355, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19159355, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19212106, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19212106, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19243246, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19243246, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19332896, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19332896, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19363707, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19363707, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19491504, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19491504, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19617218, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19617218, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19624307, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19624307, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19661356, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19661356, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19688830, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19688830, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19696287, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19696287, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19830514, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19830514, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19075129, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19075129, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19083124, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19083124, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19112019, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19112019, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19113066, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19113066, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19128442, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19128442, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19135233, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19135233, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19161612, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19161612, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19162530, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19162530, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19172559, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19172559, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19204205, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19204205, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19224751, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19224751, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19226029, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19226029, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19261923, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19261923, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19295174, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19295174, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19338682, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19338682, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19351437, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19351437, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19374538, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19374538, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19380366, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19380366, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19451225, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19451225, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19456988, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19456988, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19466138, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19466138, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19513974, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19513974, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19596836, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19596836, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19605661, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19605661, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19620444, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19620444, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19620554, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19620554, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19634814, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19634814, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19640736, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19640736, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19642980, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19642980, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19667072, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19667072, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19690108, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19690108, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19819862, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19819862, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19625371, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19625371, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19184018, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 191840180, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19438579, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19438579, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19459643, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19459643, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19487245, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19487245, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19566874, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19566874, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19184018, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19184018, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19071205, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19071205, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19221528, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19221528, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19396245, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19396245, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19484502, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19484502, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19509180, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19509180, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19666296, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19666296, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19784659, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19784659, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19608330, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19608330, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19660768, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19660768, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19468811, ["Country", "Country_source"]] = fix_country_from_text("Brunei Darussalam")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19468811, ["Country", "Country_source"]] = fix_country_from_text("Brunei Darussalam")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19660212, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19660212, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19119482, ["Country", "Country_source"]] = fix_country_from_text("Türkiye")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19119482, ["Country", "Country_source"]] = fix_country_from_text("Türkiye")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19379657, ["Country", "Country_source"]] = fix_country_from_text("Syria")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19379657, ["Country", "Country_source"]] = fix_country_from_text("Syria")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19577453, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19577453, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == 19778628, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == 19778628, ["Country", "Country_source"]] = fix_country_from_text("China")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1700000_base.loc[df_parsedX_1700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1700000_match_GLOBOCAN.loc[df_parsedX_1700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [48]:
df_parsedX_1700000_base.to_csv(DF_input_base + "parsedX_1700000.csv", index = False)
df_parsedX_1700000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_1700000.csv", index = False)

del df_parsedX_1700000_base
del df_parsedX_1700000_match_GLOBOCAN

### Correcting *parsedX_1800000.csv*

In [49]:
# Read csv
df_parsedX_1800000_base = pd.read_csv(DF_input_base + "parsedX_1800000.csv")
df_parsedX_1800000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_1800000.csv")

In [50]:
## Replace found errors by true Country values

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20704980, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20704980, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20116771, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20116771, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 19858739, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 19858739, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 19858743, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 19858743, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20651972, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20651972, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20079483, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20079483, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20177103, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20177103, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20399038, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20399038, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20423029, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20423029, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20423030, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20423030, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20423034, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20423034, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20423041, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20423041, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20594856, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20594856, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20607507, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20607507, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20690526, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20690526, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20535696, ["Country", "Country_source"]] = fix_country_from_text("Iran")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20535696, ["Country", "Country_source"]] = fix_country_from_text("Iran")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20585938, ["Country", "Country_source"]] = fix_country_from_text("Sweden")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20585938, ["Country", "Country_source"]] = fix_country_from_text("Sweden")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20107748, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20107748, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20482834, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20482834, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20683915, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20683915, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 19895170, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 19895170, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 19945092, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 19945092, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20027168, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20027168, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20150629, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20150629, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20156772, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20156772, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20358379, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20358379, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20435483, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20435483, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20177111, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20177111, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 19944031, ["Country", "Country_source"]] = fix_country_from_text("Ecuador")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 19944031, ["Country", "Country_source"]] = fix_country_from_text("Ecuador")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20382204, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20382204, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20416248, ["Country", "Country_source"]] = fix_country_from_text("Cuba")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20416248, ["Country", "Country_source"]] = fix_country_from_text("Cuba")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20442517, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20442517, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20508350, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20508350, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20384524, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20384524, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20428757, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20428757, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20150637, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20150637, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20628233, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20628233, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20558993, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20558993, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 19842033, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 19842033, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 19945229, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 19945229, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 19951711, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 19951711, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20063370, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20063370, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20082301, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20082301, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20180086, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20180086, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20651352, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20651352, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20664927, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20664927, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 19838853, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 19838853, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 19917953, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 19917953, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 19920823, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 19920823, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 19932449, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 19932449, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 19938727, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 19938727, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20062554, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20062554, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20066419, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20066419, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20083694, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20083694, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20085938, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20085938, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20093504, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20093504, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20160001, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20160001, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20173136, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20173136, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20188700, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20188700, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20194622, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20194622, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20209740, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20209740, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20223023, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20223023, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20299242, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20299242, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20354138, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20354138, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20364408, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20364408, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20376532, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20376532, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20377096, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20377096, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20382846, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20382846, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20385618, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20385618, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20392998, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20392998, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20399885, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20399885, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20410506, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20410506, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20442224, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20442224, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20453920, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20453920, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20453921, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20453921, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20470778, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20470778, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20482797, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20482797, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20506860, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20506860, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20516093, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20516093, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20521089, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20521089, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20586632, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20586632, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20624308, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20624308, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20625146, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20625146, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20660826, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20660826, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20697056, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20697056, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20704751, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20704751, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20704851, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20704851, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20015556, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20015556, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 19941471, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 19941471, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20417624, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20417624, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20491997, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20491997, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20495653, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20495653, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20628822, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20628822, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20013146, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20013146, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20153564, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20153564, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20166982, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20166982, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20172682, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20172682, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20367086, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20367086, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20399003, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20399003, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20597404, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20597404, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20597418, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20597418, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20388096, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20388096, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20044761, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20044761, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20345012, ["Country", "Country_source"]] = fix_country_from_text("Malawi")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20345012, ["Country", "Country_source"]] = fix_country_from_text("Malawi")

df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == 20376385, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == 20376385, ["Country", "Country_source"]] = fix_country_from_text("China")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1800000_base.loc[df_parsedX_1800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1800000_match_GLOBOCAN.loc[df_parsedX_1800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [51]:
df_parsedX_1800000_base.to_csv(DF_input_base + "parsedX_1800000.csv", index = False)
df_parsedX_1800000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_1800000.csv", index = False)

del df_parsedX_1800000_base
del df_parsedX_1800000_match_GLOBOCAN

### Correcting *parsedX_1900000.csv*

In [52]:
# Read csv
df_parsedX_1900000_base = pd.read_csv(DF_input_base + "parsedX_1900000.csv")
df_parsedX_1900000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_1900000.csv")

In [53]:
## Replace found errors by true Country values

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 20979677, ["Country", "Country_source"]] = fix_country_from_text("Ireland")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 20979677, ["Country", "Country_source"]] = fix_country_from_text("Ireland")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 20981674, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 20981674, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21353748, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21353748, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21389920, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21389920, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21513998, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21513998, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21577052, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21577052, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 20812824, ["Country", "Country_source"]] = fix_country_from_text("Iran")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 20812824, ["Country", "Country_source"]] = fix_country_from_text("Iran")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 20845290, ["Country", "Country_source"]] = fix_country_from_text("Iran")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 20845290, ["Country", "Country_source"]] = fix_country_from_text("Iran")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 20845293, ["Country", "Country_source"]] = fix_country_from_text("Iran")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 20845293, ["Country", "Country_source"]] = fix_country_from_text("Iran")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 20929551, ["Country", "Country_source"]] = fix_country_from_text("Iran")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 20929551, ["Country", "Country_source"]] = fix_country_from_text("Iran")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 20938425, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 20938425, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21134529, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21134529, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] ==  21132507, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] ==  21132507, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21147621, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21147621, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21374929, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21374929, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21381411, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21381411, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21530283, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21530283, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21570311, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21570311, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 20963646, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 20963646, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21057074, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21057074, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21078497, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21078497, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21317527, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21317527, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 20940127, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 20940127, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21188482, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21188482, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21550293, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21550293, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 20811660, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 20811660, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21342522, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21342522, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 20921213, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 20921213, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21104470, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21104470, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 20871480, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 20871480, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21042771, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21042771, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21046513, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21046513, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21166884, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21166884, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21237499, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21237499, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21274264, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21274264, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21432438, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21432438, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21515922, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21515922, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21573878, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21573878, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21573892, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21573892, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 20737365, ["Country", "Country_source"]] = fix_country_from_text("Ireland")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 20737365, ["Country", "Country_source"]] = fix_country_from_text("Ireland")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 20795794, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 20795794, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 20799942, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 20799942, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 20829508, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 20829508, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 20840098, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 20840098, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 20858520, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 20858520, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 20959667, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 20959667, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21036538, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21036538, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21054901, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21054901, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21062986, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21062986, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21084729, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21084729, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21108835, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21108835, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21111364, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21111364, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21113464, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21113464, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21173379, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21173379, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21193708, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21193708, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21237503, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21237503, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21246405, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21246405, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21264542, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21264542, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21285094, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21285094, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21290202, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21290202, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21298495, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21298495, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21352556, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21352556, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21472872, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21472872, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21472885, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21472885, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21479313, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21479313, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21485699, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21485699, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21487628, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21487628, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21603121, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21603121, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21619579, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21619579, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21129090, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21129090, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21079866, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21079866, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21094563, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21094563, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21282027, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21282027, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21542600, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21542600, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21561408, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21561408, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21258055, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21258055, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21501677, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21501677, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 20955144, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 20955144, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21341559, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21341559, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 20730388, ["Country", "Country_source"]] = fix_country_from_text("Brunei Darussalam")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 20730388, ["Country", "Country_source"]] = fix_country_from_text("Brunei Darussalam")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21103808, ["Country", "Country_source"]] = fix_country_from_text("Brunei Darussalam")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21103808, ["Country", "Country_source"]] = fix_country_from_text("Brunei Darussalam")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21187522, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21187522, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21144621, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21144621, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21266800, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21266800, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21273072, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21273072, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21327325, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21327325, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21468561, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21468561, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21475928, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21475928, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == 21531287, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == 21531287, ["Country", "Country_source"]] = fix_country_from_text("China")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_1900000_base.loc[df_parsedX_1900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_1900000_match_GLOBOCAN.loc[df_parsedX_1900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [54]:
df_parsedX_1900000_base.to_csv(DF_input_base + "parsedX_1900000.csv", index = False)
df_parsedX_1900000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_1900000.csv", index = False)

del df_parsedX_1900000_base
del df_parsedX_1900000_match_GLOBOCAN

### Correcting *parsedX_200000.csv*

In [55]:
# Read csv
df_parsedX_200000_base = pd.read_csv(DF_input_base + "parsedX_200000.csv")
df_parsedX_200000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_200000.csv")

In [56]:
## Replace found errors by true Country values

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1998100, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1998100, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2010301, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2010301, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2073005, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2073005, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2164195, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2164195, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2171180, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2171180, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2233473, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2233473, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2363615, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2363615, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2446758, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2446758, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2446759, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2446759, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2477147, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2477147, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2521813, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2521813, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2611966, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2611966, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2275578, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2275578, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2356460, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2356460, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2101452, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2101452, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2583858, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2583858, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2456177, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2456177, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2536573, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2536573, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2477086, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2477086, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2218045, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2218045, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2539611, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2539611, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2348881, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2348881, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2371934, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2371934, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2675007, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2675007, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1994889, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1994889, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1995028, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1995028, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1995073, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1995073, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1995183, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1995183, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1995191, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1995191, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1995297, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1995297, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1995312, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1995312, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1995313, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1995313, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1995319, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1995319, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1995358, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1995358, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1995363, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1995363, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1995402, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1995402, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1995445, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1995445, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1995479, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1995479, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1995486, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1995486, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1995522, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1995522, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1995526, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1995526, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1995527, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1995527, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1995801, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1995801, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1995816, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1995816, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1995877, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1995877, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1996006, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1996006, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1996087, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1996087, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1996141, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1996141, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1996142, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1996142, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1996166, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1996166, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1996167, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1996167, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1996204, ["Country", "Country_source"]] = fix_country_from_text("Hungary")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1996204, ["Country", "Country_source"]] = fix_country_from_text("Hungary")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1996321, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1996321, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1996398, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1996398, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1996404, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1996404, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1996405, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1996405, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1996441, ["Country", "Country_source"]] = fix_country_from_text("South Africa")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1996441, ["Country", "Country_source"]] = fix_country_from_text("South Africa")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1996504, ["Country", "Country_source"]] = fix_country_from_text("Denmark")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1996504, ["Country", "Country_source"]] = fix_country_from_text("Denmark")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1996526, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1996526, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1996527, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1996527, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1996591, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1996591, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1996592, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1996592, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1996616, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1996616, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1996856, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1996856, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1996859, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1996859, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1996860, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1996860, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1996871, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1996871, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1997096, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1997096, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1997106, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1997106, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1997120, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1997120, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1997145, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1997145, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1997163, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1997163, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1997169, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1997169, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1997174, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1997174, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1997198, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1997198, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1997266, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1997266, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1997285, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1997285, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1997474, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1997474, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1997532, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1997532, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1997533, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1997533, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1997541, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1997541, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1997552, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1997552, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1997554, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1997554, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1997561, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1997561, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1997589, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1997589, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1997596, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1997596, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1997612, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1997612, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1997615, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1997615, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1997616, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1997616, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1997679, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1997679, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1997686, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1997686, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1997707, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1997707, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1997727, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1997727, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1997743, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1997743, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1997744, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1997744, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1997745, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1997745, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1997753, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1997753, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1997779, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1997779, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1997835, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1997835, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1997852, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1997852, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1997853, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1997853, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1997927, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1997927, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1997934, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1997934, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1997939, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1997939, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1997942, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1997942, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1997944, ["Country", "Country_source"]] = fix_country_from_text("Croatia")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1997944, ["Country", "Country_source"]] = fix_country_from_text("Croatia")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1997974, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1997974, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1998048, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1998048, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1998094, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1998094, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1998180, ["Country", "Country_source"]] = fix_country_from_text("Norway")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1998180, ["Country", "Country_source"]] = fix_country_from_text("Norway")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1998233, ["Country", "Country_source"]] = fix_country_from_text("Denmark")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1998233, ["Country", "Country_source"]] = fix_country_from_text("Denmark")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1998257, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1998257, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1998360, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1998360, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1998367, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1998367, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1998434, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1998434, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1998472, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1998472, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1998480, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1998480, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1998499, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1998499, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1998508, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1998508, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1998509, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1998509, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1998568, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1998568, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1998570, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1998570, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1998596, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] ==1998596, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1998598, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1998598, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1998629, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1998629, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1998727, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1998727, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1998759, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1998759, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1998789, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1998789, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1998793, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1998793, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1998833, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1998833, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1998835, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1998835, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1998839, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1998839, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1998842, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1998842, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1998874, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1998874, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1998904, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1998904, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1998940, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1998940, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1998942, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1998942, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1998943, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1998943, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1998944, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1998944, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1998946, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1998946, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1998947, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1998947, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1998952, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1998952, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1998954, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1998954, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1998956, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1998956, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1998965, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1998965, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1998972, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1998972, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1998982, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1998982, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1998986, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1998986, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1998992, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1998992, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1998994, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1998994, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1999004, ["Country", "Country_source"]] = fix_country_from_text("Croatia")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1999004, ["Country", "Country_source"]] = fix_country_from_text("Croatia")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1999005, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1999005, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1999065, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1999065, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1999124, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1999124, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1999137, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1999137, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1999182, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1999182, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1999221, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1999221, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1999305, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1999305, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1999307, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1999307, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1999308, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1999308, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1999484, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1999484, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1999505, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1999505, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1999515, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1999515, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1999548, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1999548, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1999647, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1999647, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1999669, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1999669, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1999710, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1999710, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1999718, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1999718, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1999720, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1999720, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1999768, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1999768, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1999784, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1999784, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1999820, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1999820, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1999837, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1999837, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1999879, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1999879, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1999890, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1999890, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1999957, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1999957, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1999969, ["Country", "Country_source"]] = fix_country_from_text("Sweden")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1999969, ["Country", "Country_source"]] = fix_country_from_text("Sweden")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1999970, ["Country", "Country_source"]] = fix_country_from_text("Sweden")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1999970, ["Country", "Country_source"]] = fix_country_from_text("Sweden")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 1999990, ["Country", "Country_source"]] = fix_country_from_text("Sweden")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 1999990, ["Country", "Country_source"]] = fix_country_from_text("Sweden")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2000042, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2000042, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2000154, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2000154, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2000162, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2000162, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2000165, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2000165, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2000218, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2000218, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2000323, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2000323, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2000391, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2000391, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2000403, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2000403, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2000459, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2000459, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2000504, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2000504, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2000525, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2000525, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2000546, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2000546, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2000550, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2000550, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2000551, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2000551, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2000553, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2000553, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2000565, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2000565, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2000566, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2000566, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2000585, ["Country", "Country_source"]] = fix_country_from_text("Norway")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2000585, ["Country", "Country_source"]] = fix_country_from_text("Norway")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2000610, ["Country", "Country_source"]] = fix_country_from_text("Norway")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2000610, ["Country", "Country_source"]] = fix_country_from_text("Norway")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2000611, ["Country", "Country_source"]] = fix_country_from_text("Norway")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2000611, ["Country", "Country_source"]] = fix_country_from_text("Norway")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2000616, ["Country", "Country_source"]] = fix_country_from_text("Norway")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2000616, ["Country", "Country_source"]] = fix_country_from_text("Norway")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2000643, ["Country", "Country_source"]] = fix_country_from_text("Denmark")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2000643, ["Country", "Country_source"]] = fix_country_from_text("Denmark")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2000645, ["Country", "Country_source"]] = fix_country_from_text("Denmark")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2000645, ["Country", "Country_source"]] = fix_country_from_text("Denmark")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2000646, ["Country", "Country_source"]] = fix_country_from_text("Denmark")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2000646, ["Country", "Country_source"]] = fix_country_from_text("Denmark")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2000647, ["Country", "Country_source"]] = fix_country_from_text("Denmark")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2000647, ["Country", "Country_source"]] = fix_country_from_text("Denmark")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2000663, ["Country", "Country_source"]] = fix_country_from_text("Denmark")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2000663, ["Country", "Country_source"]] = fix_country_from_text("Denmark")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2000664, ["Country", "Country_source"]] = fix_country_from_text("Denmark")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2000664, ["Country", "Country_source"]] = fix_country_from_text("Denmark")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2000669, ["Country", "Country_source"]] = fix_country_from_text("Denmark")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2000669, ["Country", "Country_source"]] = fix_country_from_text("Denmark")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2000671, ["Country", "Country_source"]] = fix_country_from_text("Denmark")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2000671, ["Country", "Country_source"]] = fix_country_from_text("Denmark")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2000687, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2000687, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2000823, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2000823, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2000838, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2000838, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2000840, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2000840, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2000841, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2000841, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2000889, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2000889, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2000890, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2000890, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2000932, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2000932, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2000944, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2000944, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2001102, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2001102, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2001155, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2001155, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2001167, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2001167, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2001170, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2001170, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2001331, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2001331, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2001334, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2001334, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2001341, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2001341, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2001345, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2001345, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2001370, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2001370, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2001417, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2001417, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2001420, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2001420, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2001536, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2001536, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2001543, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2001543, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2001559, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2001559, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2001561, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2001561, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2001565, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2001565, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2001568, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2001568, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2001571, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2001571, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2001573, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2001573, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2001577, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2001577, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2001580, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2001580, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2001582, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2001582, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2001584, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2001584, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2001585, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2001585, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2001649, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2001649, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2001651, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2001651, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2001652, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2001652, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2001724, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2001724, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2001874, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2001874, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2001905, ["Country", "Country_source"]] = fix_country_from_text("Guatemala")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2001905, ["Country", "Country_source"]] = fix_country_from_text("Guatemala")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2001913, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2001913, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2001987, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2001987, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2002051, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2002051, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2002063, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2002063, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2002098, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2002098, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2002118, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2002118, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2002160, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2002160, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2002194, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2002194, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2002250, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2002250, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2002258, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2002258, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2002267, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2002267, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2002268, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2002268, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2002270, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2002270, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2002366, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2002366, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2002418, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2002418, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2002421, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2002421, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2002424, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2002424, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2002426, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2002426, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2002446, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2002446, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2002477, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2002477, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2002702, ["Country", "Country_source"]] = fix_country_from_text("Sweden")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2002702, ["Country", "Country_source"]] = fix_country_from_text("Sweden")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2002704, ["Country", "Country_source"]] = fix_country_from_text("Sweden")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2002704, ["Country", "Country_source"]] = fix_country_from_text("Sweden")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2002713, ["Country", "Country_source"]] = fix_country_from_text("Sweden")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2002713, ["Country", "Country_source"]] = fix_country_from_text("Sweden")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2002715, ["Country", "Country_source"]] = fix_country_from_text("Sweden")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2002715, ["Country", "Country_source"]] = fix_country_from_text("Sweden")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2002720, ["Country", "Country_source"]] = fix_country_from_text("Sweden")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2002720, ["Country", "Country_source"]] = fix_country_from_text("Sweden")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2002732, ["Country", "Country_source"]] = fix_country_from_text("Sweden")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2002732, ["Country", "Country_source"]] = fix_country_from_text("Sweden")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2002737, ["Country", "Country_source"]] = fix_country_from_text("Sweden")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2002737, ["Country", "Country_source"]] = fix_country_from_text("Sweden")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2002801, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2002801, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2002806, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2002806, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2002819, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2002819, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2002855, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2002855, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2002963, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2002963, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2002964, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2002964, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2003006, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2003006, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2003011, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2003011, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2003012, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2003012, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2003027, ["Country", "Country_source"]] = fix_country_from_text("Hungary")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2003027, ["Country", "Country_source"]] = fix_country_from_text("Hungary")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2003033, ["Country", "Country_source"]] = fix_country_from_text("Hungary")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2003033, ["Country", "Country_source"]] = fix_country_from_text("Hungary")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2003036, ["Country", "Country_source"]] = fix_country_from_text("Hungary")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2003036, ["Country", "Country_source"]] = fix_country_from_text("Hungary")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2003038, ["Country", "Country_source"]] = fix_country_from_text("Hungary")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2003038, ["Country", "Country_source"]] = fix_country_from_text("Hungary")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2003111, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2003111, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2003115, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2003115, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2003166, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2003166, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2003182, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2003182, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2003183, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2003183, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2003224, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2003224, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2003227, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2003227, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2003231, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2003231, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2003374, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2003374, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2003395, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2003395, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2003430, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2003430, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2003431, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2003431, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2003450, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2003450, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2003521, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2003521, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2003844, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2003844, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2003845, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2003845, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2003874, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2003874, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2003918, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2003918, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2003942, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2003942, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2003980, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2003980, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2003994, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2003994, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004015, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004015, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004017, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004017, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004026, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004026, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004071, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004071, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004072, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004072, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004073, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004073, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004074, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004074, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004078, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004078, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004082, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004082, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004099, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004099, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004109, ["Country", "Country_source"]] = fix_country_from_text("Denmark")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004109, ["Country", "Country_source"]] = fix_country_from_text("Denmark")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004142, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004142, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004227, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004227, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004230, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004230, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004234, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004234, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004236, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004236, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004241, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004241, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004243, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004243, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004300, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004300, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004304, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004304, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004306, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004306, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004308, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004308, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004311, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004311, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004318, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004318, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004324, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004324, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004325, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004325, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004327, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004327, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004328, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004328, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004330, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004330, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004342, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004342, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004344, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004344, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004350, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004350, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004351, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004351, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004357, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] ==2004357, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004362, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004362, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004363, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004363, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004371, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004371, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004372, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004372, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004375, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004375, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004391, ["Country", "Country_source"]] = fix_country_from_text("Czechia")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004391, ["Country", "Country_source"]] = fix_country_from_text("Czechia")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004413, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004413, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004443, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004443, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004494, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004494, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004506, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004506, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004521, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004521, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004569, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004569, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004588, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004588, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004591, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004591, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004668, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004668, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004706, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004706, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004795, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004795, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004798, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004798, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004859, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004859, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004941, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004941, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004952, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004952, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004958, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004958, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004959, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004959, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004963, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004963, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004966, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004966, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004985, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004985, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004993, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004993, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2004996, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2004996, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2005087, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2005087, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2005248, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2005248, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2005329, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2005329, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2005391, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2005391, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2005405, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2005405, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2005408, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2005408, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2005472, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2005472, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2005483, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2005483, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2005513, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2005513, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2005537, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2005537, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2005632, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2005632, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2005653, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2005653, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2005704, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2005704, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2005780, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2005780, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2005841, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2005841, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2005879, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2005879, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2005881, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2005881, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2005896, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2005896, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2005908, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2005908, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2006090, ["Country", "Country_source"]] = fix_country_from_text("Norway")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2006090, ["Country", "Country_source"]] = fix_country_from_text("Norway")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2006142, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2006142, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2006145, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2006145, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2006167, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2006167, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2006214, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2006214, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2006260, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2006260, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2006266, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2006266, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2006300, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2006300, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2006317, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2006317, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2006320, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2006320, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2006321, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2006321, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2006323, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2006323, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2006328, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2006328, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2006329, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2006329, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2006330, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2006330, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2006331, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2006331, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2006337, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2006337, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2006339, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2006339, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2006341, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2006341, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2006342, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2006342, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2006343, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2006343, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2006357, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2006357, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2006358, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2006358, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2006375, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2006375, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2006376, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2006376, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2006381, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2006381, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2006410, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2006410, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2006444, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2006444, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2006483, ["Country", "Country_source"]] = fix_country_from_text("Norway")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2006483, ["Country", "Country_source"]] = fix_country_from_text("Norway")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2006492, ["Country", "Country_source"]] = fix_country_from_text("Norway")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2006492, ["Country", "Country_source"]] = fix_country_from_text("Norway")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2006497, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2006497, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2006560, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2006560, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2006642, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2006642, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2006649, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2006649, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2006789, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2006789, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2006829, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2006829, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2006854, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2006854, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2006855, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2006855, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2006862, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2006862, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2006877, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2006877, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2006884, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2006884, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2006885, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2006885, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2006886, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2006886, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2006920, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2006920, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2007008, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2007008, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2007018, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2007018, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2007045, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2007045, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2007094, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2007094, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2007207, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2007207, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2007324, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2007324, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2007379, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2007379, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2007492, ["Country", "Country_source"]] = fix_country_from_text("Israel")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2007492, ["Country", "Country_source"]] = fix_country_from_text("Israel")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2007524, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2007524, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2007537, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2007537, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2007611, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2007611, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2007618, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2007618, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2007622, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2007622, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2007759, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2007759, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2007801, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2007801, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2007806, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2007806, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2007822, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2007822, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2007826, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2007826, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2007829, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2007829, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2008052, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2008052, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2008057, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2008057, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2008087, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2008087, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2008133, ["Country", "Country_source"]] = fix_country_from_text("Sweden")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2008133, ["Country", "Country_source"]] = fix_country_from_text("Sweden")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2008137, ["Country", "Country_source"]] = fix_country_from_text("Sweden")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2008137, ["Country", "Country_source"]] = fix_country_from_text("Sweden")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2008151, ["Country", "Country_source"]] = fix_country_from_text("Sweden")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2008151, ["Country", "Country_source"]] = fix_country_from_text("Sweden")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2008174, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2008174, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2008261, ["Country", "Country_source"]] = fix_country_from_text("New Zealand")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2008261, ["Country", "Country_source"]] = fix_country_from_text("New Zealand")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2008300, ["Country", "Country_source"]] = fix_country_from_text("Hungary")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2008300, ["Country", "Country_source"]] = fix_country_from_text("Hungary")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2008303, ["Country", "Country_source"]] = fix_country_from_text("Hungary")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2008303, ["Country", "Country_source"]] = fix_country_from_text("Hungary")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2008382, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2008382, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2008419, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2008419, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2008423, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2008423, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2008425, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2008425, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2008427, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2008427, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2008428, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2008428, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2008430, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2008430, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2008439, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2008439, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2008447, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2008447, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2008470, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2008470, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2008479, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2008479, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2008506, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2008506, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2008572, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2008572, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2008573, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2008573, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2008597, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2008597, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2008599, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2008599, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2008633, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2008633, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2008647, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2008647, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2008658, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2008658, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2008672, ["Country", "Country_source"]] = fix_country_from_text("Norway")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2008672, ["Country", "Country_source"]] = fix_country_from_text("Norway")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2008673, ["Country", "Country_source"]] = fix_country_from_text("Norway")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2008673, ["Country", "Country_source"]] = fix_country_from_text("Norway")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2008684, ["Country", "Country_source"]] = fix_country_from_text("Norway")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2008684, ["Country", "Country_source"]] = fix_country_from_text("Norway")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2008704, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2008704, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2008706, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2008706, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2008712, ["Country", "Country_source"]] = fix_country_from_text("Denmark")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2008712, ["Country", "Country_source"]] = fix_country_from_text("Denmark")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2008724, ["Country", "Country_source"]] = fix_country_from_text("Denmark")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2008724, ["Country", "Country_source"]] = fix_country_from_text("Denmark")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2008727, ["Country", "Country_source"]] = fix_country_from_text("Denmark")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2008727, ["Country", "Country_source"]] = fix_country_from_text("Denmark")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2008732, ["Country", "Country_source"]] = fix_country_from_text("Denmark")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2008732, ["Country", "Country_source"]] = fix_country_from_text("Denmark")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2008747, ["Country", "Country_source"]] = fix_country_from_text("Denmark")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2008747, ["Country", "Country_source"]] = fix_country_from_text("Denmark")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2008748, ["Country", "Country_source"]] = fix_country_from_text("Denmark")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2008748, ["Country", "Country_source"]] = fix_country_from_text("Denmark")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2008816, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2008816, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2008922, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2008922, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2008974, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2008974, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2009056, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2009056, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2009059, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2009059, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2009061, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2009061, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2009064, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2009064, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2009129, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2009129, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2009130, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2009130, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2009132, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2009132, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2009134, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2009134, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2009135, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2009135, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2009136, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2009136, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2009183, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2009183, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2009189, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2009189, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2009213, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2009213, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2009233, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2009233, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2009234, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2009234, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2009399, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2009399, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2009498, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2009498, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2009502, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2009502, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2009503, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2009503, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2009506, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2009506, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2009513, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2009513, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2009519, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2009519, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2009521, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2009521, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2009522, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2009522, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2009525, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2009525, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2009526, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2009526, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2009531, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2009531, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2009532, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2009532, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2009535, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2009535, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2009538, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2009538, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2009540, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2009540, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2009597, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2009597, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2009598, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2009598, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2009737, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2009737, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2009740, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2009740, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2009867, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2009867, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2009870, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2009870, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2009899, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2009899, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2010030, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2010030, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2010067, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2010067, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2010068, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2010068, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2010069, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2010069, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2010071, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2010071, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2010137, ["Country", "Country_source"]] = fix_country_from_text("Israel")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2010137, ["Country", "Country_source"]] = fix_country_from_text("Israel")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2010178, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2010178, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2010179, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2010179, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2010228, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2010228, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2010283, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2010283, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2010286, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2010286, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2010288, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2010288, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2010296, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2010296, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2010309, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2010309, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2010310, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2010310, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2010312, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2010312, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2010337, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2010337, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2010345, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2010345, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2010568, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2010568, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2010666, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2010666, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2010876, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2010876, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2010890, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2010890, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2010901, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2010901, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2011015, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2011015, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2011022, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2011022, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2011025, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2011025, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2011136, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2011136, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2011205, ["Country", "Country_source"]] = fix_country_from_text("Czechia")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2011205, ["Country", "Country_source"]] = fix_country_from_text("Czechia")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2011222, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2011222, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2011289, ["Country", "Country_source"]] = fix_country_from_text("New Zealand")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2011289, ["Country", "Country_source"]] = fix_country_from_text("New Zealand")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2011349, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2011349, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2011354, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2011354, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2011364, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2011364, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2011387, ["Country", "Country_source"]] = fix_country_from_text("Hungary")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2011387, ["Country", "Country_source"]] = fix_country_from_text("Hungary")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2011388, ["Country", "Country_source"]] = fix_country_from_text("Hungary")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2011388, ["Country", "Country_source"]] = fix_country_from_text("Hungary")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2011389, ["Country", "Country_source"]] = fix_country_from_text("Hungary")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2011389, ["Country", "Country_source"]] = fix_country_from_text("Hungary")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2011478, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2011478, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2011537, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2011537, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2011600, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2011600, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2011654, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2011654, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2011657, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2011657, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2011688, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2011688, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2011707, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2011707, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2011720, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2011720, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2011801, ["Country", "Country_source"]] = fix_country_from_text("South Africa")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2011801, ["Country", "Country_source"]] = fix_country_from_text("South Africa")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2011808, ["Country", "Country_source"]] = fix_country_from_text("South Africa")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2011808, ["Country", "Country_source"]] = fix_country_from_text("South Africa")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2011969, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2011969, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2011974, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2011974, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2012067, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2012067, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2155622, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2155622, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2184346, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2184346, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2186928, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2186928, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2207371, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2207371, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2209643, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2209643, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2224919, ["Country", "Country_source"]] = fix_country_from_text("Israel")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2224919, ["Country", "Country_source"]] = fix_country_from_text("Israel")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2541015, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2541015, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2569811, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2569811, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == 2576239, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == 2576239, ["Country", "Country_source"]] = fix_country_from_text("Russia")

#df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_200000_base.loc[df_parsedX_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_200000_match_GLOBOCAN.loc[df_parsedX_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [57]:
df_parsedX_200000_base.to_csv(DF_input_base + "parsedX_200000.csv", index = False)
df_parsedX_200000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_200000.csv", index = False)

del df_parsedX_200000_base
del df_parsedX_200000_match_GLOBOCAN

### Correcting *parsedX_2000000.csv*

In [58]:
# Read csv
df_parsedX_2000000_base = pd.read_csv(DF_input_base + "parsedX_2000000.csv")
df_parsedX_2000000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_2000000.csv")

In [59]:
## Replace found errors by true Country values

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21738117, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21738117, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21955498, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21955498, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 22008273, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 22008273, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 22079516, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 22079516, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 22347959, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 22347959, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21768603, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21768603, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21845203, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21845203, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21696673, ["Country", "Country_source"]] = fix_country_from_text("Denmark")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21696673, ["Country", "Country_source"]] = fix_country_from_text("Denmark")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21975185, ["Country", "Country_source"]] = fix_country_from_text("Denmark")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21975185, ["Country", "Country_source"]] = fix_country_from_text("Denmark")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21667238, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21667238, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 22248273, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 22248273, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 22271404, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 22271404, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 22306424 , ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 22306424 , ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21886899, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21886899, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21767710, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21767710, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 22229958, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 22229958, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 22233946, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 22233946, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21749397, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21749397, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21986113, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21986113, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 22295896, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 22295896, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21623259, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21623259, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 22126249	, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 22126249	, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21737655, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21737655, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21818589, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21818589, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21826671, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21826671, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 22038670, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 22038670, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 22041920, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 22041920, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 22287757, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 22287757, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 22288737, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 22288737, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 22358611, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 22358611, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21697760, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21697760, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21723792, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21723792, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21732047, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21732047, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21741707, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21741707, ["Country", "Country_source"]] = fix_country_from_text("United States")

# Convert to Taiwan in the base dataset and to China in the matching Globocan dataset because, in the latter, there is no Taiwan (added to China)
df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21793155, ["Country", "Country_source"]] = fix_country_from_text("Taiwan")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21793155, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21800071, ["Country", "Country_source"]] = fix_country_from_text("Israel")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21800071, ["Country", "Country_source"]] = fix_country_from_text("Israel")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21803154, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21803154, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21812970, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21812970, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21844577, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21844577, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21848691, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21848691, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21849090, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21849090, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21864884, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21864884, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21867523, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21867523, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21871986, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21871986, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21894461, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21894461, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21899746, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21899746, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21906355, ["Country", "Country_source"]] = fix_country_from_text("Ireland")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21906355, ["Country", "Country_source"]] = fix_country_from_text("Ireland")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21921791, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21921791, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21922019, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21922019, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21962474, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21962474, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21965458, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21965458, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 22023734, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 22023734, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 22028409, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 22028409, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 22071020, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 22071020, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 22076479, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 22076479, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 22079760, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 22079760, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 22112838, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 22112838, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 22142690, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 22142690, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 22158716, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 22158716, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 22174434, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 22174434, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 22206853, ["Country", "Country_source"]] = fix_country_from_text("Greece")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 22206853, ["Country", "Country_source"]] = fix_country_from_text("Greece")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 22208390, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 22208390, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 22210429, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 22210429, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 22214514, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 22214514, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 22232519, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 22232519, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 22259051, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 22259051, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 22265879, ["Country", "Country_source"]] = fix_country_from_text("Portugal")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 22265879, ["Country", "Country_source"]] = fix_country_from_text("Portugal")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 22276017, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 22276017, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 22278770, ["Country", "Country_source"]] = fix_country_from_text("Israel")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 22278770, ["Country", "Country_source"]] = fix_country_from_text("Israel")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 22296784, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 22296784, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 22300659, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 22300659, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 22373079, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 22373079, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21755950, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21755950, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21792328, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21792328, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21944546, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21944546, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21954959, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21954959, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21967832, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21967832, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 22085129, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 22085129, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 22285059, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 22285059, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21686802, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21686802, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21714437, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21714437, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21779758, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21779758, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21990213, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21990213, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 22046564, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 22046564, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 22067260, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 22067260, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 22155745, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 22155745, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 22293942, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 22293942, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21934690, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21934690, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 22283649, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 22283649, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 22352749, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 22352749, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21656067, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21656067, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21721159, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21721159, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21843939, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21843939, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 21956458, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 21956458, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == 22269649, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == 22269649, ["Country", "Country_source"]] = fix_country_from_text("China")

#df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2000000_base.loc[df_parsedX_2000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2000000_match_GLOBOCAN.loc[df_parsedX_2000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [60]:
df_parsedX_2000000_base.to_csv(DF_input_base + "parsedX_2000000.csv", index = False)
df_parsedX_2000000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_2000000.csv", index = False)

del df_parsedX_2000000_base
del df_parsedX_2000000_match_GLOBOCAN

### Correcting *parsedX_2100000.csv*

In [64]:
# Read csv
df_parsedX_2100000_base = pd.read_csv(DF_input_base + "parsedX_2100000.csv")
df_parsedX_2100000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_2100000.csv")

In [65]:
## Replace found errors by true Country values

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22504321, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22504321, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 23150848, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 23150848, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22492023, ["Country", "Country_source"]] = fix_country_from_text("Ireland")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22492023, ["Country", "Country_source"]] = fix_country_from_text("Ireland")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22498191, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22498191, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22697326, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22697326, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22760819, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22760819, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22409891, ["Country", "Country_source"]] = fix_country_from_text("Denmark")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22409891, ["Country", "Country_source"]] = fix_country_from_text("Denmark")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22649741, ["Country", "Country_source"]] = fix_country_from_text("Bulgaria")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22649741, ["Country", "Country_source"]] = fix_country_from_text("Bulgaria")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 23085864, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 23085864, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22640890, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22640890, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22385244, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22385244, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22508812, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22508812, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22544068, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22544068, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22753729, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22753729, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22804241, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22804241, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22393102, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22393102, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22696254, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22696254, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22407418, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22407418, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22879551, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22879551, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22437116, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22437116, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22488622, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22488622, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22644852, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22644852, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22796160, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22796160, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22809646, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22809646, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22782591, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22782591, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22808570, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22808570, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22905907, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22905907, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22537748, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22537748, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22614238, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22614238, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22615049, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22615049, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22798169, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22798169, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22827846, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22827846, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22853690, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22853690, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22891146, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22891146, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22947631, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22947631, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22961650, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22961650, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 23022228, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 23022228, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 23054846, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 23054846, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 23054846, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 23054846, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 23117294, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 23117294, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22402380, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22402380, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22438764, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22438764, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22449145, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22449145, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22455486, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22455486, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22459781, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22459781, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22471922, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22471922, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22472335, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22472335, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22523611, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22523611, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22653380, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22653380, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22692591, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22692591, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22709648, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22709648, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22733221, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] ==22733221 , ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22749210, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22749210, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22829042, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22829042, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22831463, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22831463, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22834842, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22834842, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22841728, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22841728, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22854225, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22854225, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22863609, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22863609, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22882222, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22882222, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22889306, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22889306, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22926903, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22926903, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22977854, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22977854, ["Country", "Country_source"]] = fix_country_from_text("Russia")

# Convert to Taiwan in the base dataset and to China in the matching Globocan dataset because, in the latter, there is no Taiwan (added to China)
df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22978603, ["Country", "Country_source"]] = fix_country_from_text("Taiwan")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22978603, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 23032615, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] ==23032615 , ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 23036231, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 23036231, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 23065162, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 23065162, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 23065174, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 23065174, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 23065228, ["Country", "Country_source"]] = fix_country_from_text("Iran")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 23065228, ["Country", "Country_source"]] = fix_country_from_text("Iran")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 23065820, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 23065820, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 23066136, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 23066136, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 23072453, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 23072453, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 23143011, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 23143011, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 23152406, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 23152406, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 23170963, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 23170963, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22506638, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22506638, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22537682, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22537682, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22545419, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22545419, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22549684, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22549684, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22642942, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22642942, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22749804, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22749804, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22846704, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22846704, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 23100366, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 23100366, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22472691, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22472691, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22495270, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22495270, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22594476, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22594476, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22826625, ["Country", "Country_source"]] = fix_country_from_text("Brunei Darussalam")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22826625, ["Country", "Country_source"]] = fix_country_from_text("Brunei Darussalam")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22846855, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22846855, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22898313, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22898313, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22918687, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22918687, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 23023913, ["Country", "Country_source"]] = fix_country_from_text("Brunei Darussalam")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 23023913, ["Country", "Country_source"]] = fix_country_from_text("Brunei Darussalam")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22392578, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22392578, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22517588, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22517588, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 23119200, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 23119200, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22389143, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22389143, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22430115, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22430115, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22489084, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22489084, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22550337, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22550337, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22554587, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22554587, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22628229, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22628229, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22749192, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22749192, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22853296, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22853296, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22901464, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22901464, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 22966358, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 22966358, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 23044540, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 23044540, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == 23124769, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == 23124769, ["Country", "Country_source"]] = fix_country_from_text("China")

#df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2100000_base.loc[df_parsedX_2100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2100000_match_GLOBOCAN.loc[df_parsedX_2100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [66]:
df_parsedX_2100000_base.to_csv(DF_input_base + "parsedX_2100000.csv", index = False)
df_parsedX_2100000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_2100000.csv", index = False)

del df_parsedX_2100000_base
del df_parsedX_2100000_match_GLOBOCAN

### Correcting *parsedX_2200000.csv*

In [67]:
# Read csv
df_parsedX_2200000_base = pd.read_csv(DF_input_base + "parsedX_2200000.csv")
df_parsedX_2200000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_2200000.csv")

In [68]:
## Replace found errors by true Country values

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23199318, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23199318, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23466149, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23466149, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23567228, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23567228, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23568704, ["Country", "Country_source"]] = fix_country_from_text("Ireland")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23568704, ["Country", "Country_source"]] = fix_country_from_text("Ireland")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23642892, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23642892, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23850038, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23850038, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23852704, ["Country", "Country_source"]] = fix_country_from_text("Ireland")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23852704, ["Country", "Country_source"]] = fix_country_from_text("Ireland")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23878888, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23878888, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23410091, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23410091, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23453489, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23453489, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23591915, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23591915, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23677538, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23677538, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23737135, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23737135, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23810007, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23810007, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23814805, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23814805, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23728991, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23728991, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23541856, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23541856, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23576902, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23576902, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23745557, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23745557, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23549732, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23549732, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23531907, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23531907, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23801227, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23801227, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23880302, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23880302, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23250087, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23250087, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23225448, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23225448, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23232495, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23232495, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23242821, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23242821, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23264066, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23264066, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23280118, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23280118, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23371017, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23371017, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23379230, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23379230, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23460370, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23460370, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23587603, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23587603, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23720159, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23720159, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23861712, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23861712, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23180516, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23180516, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23231516, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23231516, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23241366, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23241366, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23250450, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23250450, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23263697, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23263697, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23304651, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23304651, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23326271, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23326271, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23328810, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23328810, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23345359, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23345359, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23345384, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23345384, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23355268, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23355268, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23379852, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23379852, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23397776, ["Country", "Country_source"]] = fix_country_from_text("Croatia")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23397776, ["Country", "Country_source"]] = fix_country_from_text("Croatia")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23451615, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23451615, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23471296, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23471296, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23520191, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23520191, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23522447, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23522447, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23566155, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23566155, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23569735, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23569735, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23571611, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23571611, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23591925, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23591925, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23664318, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23664318, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23701117, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23701117, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23718873, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23718873, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23748174, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23748174, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23773794, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23773794, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23777766, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23777766, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23793983, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23793983, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23794150, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23794150, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23810058, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23810058, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23848276, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23848276, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23866263, ["Country", "Country_source"]] = fix_country_from_text("South Korea")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23866263, ["Country", "Country_source"]] = fix_country_from_text("South Korea")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23866296, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23866296, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23870712, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23870712, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23773084, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23773084, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23185422, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23185422, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23714623, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23714623, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23796995, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23796995, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23328791, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23328791, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23594910, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23594910, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23693014, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23693014, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23782518, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23782518, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23861223, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23861223, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23876539, ["Country", "Country_source"]] = fix_country_from_text("Jamaica")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23876539, ["Country", "Country_source"]] = fix_country_from_text("Jamaica")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23314854, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23314854, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23267166, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23267166, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23455428, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23455428, ["Country", "Country_source"]] = fix_country_from_text("Spain")#df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23218839, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23218839, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23246658, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23246658, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23254855, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23254855, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23314567, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23314567, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23363385, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23363385, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23402621, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23402621, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23478573, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23478573, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23511011, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23511011, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23588806, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23588806, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23599755, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23599755, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23673926, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23673926, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23792763, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23792763, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23811047, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23811047, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23824286, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23824286, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == 23839975, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == 23839975, ["Country", "Country_source"]] = fix_country_from_text("China")

#df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2200000_base.loc[df_parsedX_2200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2200000_match_GLOBOCAN.loc[df_parsedX_2200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [69]:
df_parsedX_2200000_base.to_csv(DF_input_base + "parsedX_2200000.csv", index = False)
df_parsedX_2200000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_2200000.csv", index = False)

del df_parsedX_2200000_base
del df_parsedX_2200000_match_GLOBOCAN

### Correcting *parsedX_2300000.csv*

In [70]:
# Read csv
df_parsedX_2300000_base = pd.read_csv(DF_input_base + "parsedX_2300000.csv")
df_parsedX_2300000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_2300000.csv")

In [71]:
## Replace found errors by true Country values

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24198623, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24198623, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24363999, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24363999, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24461094, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24461094, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24635957, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24635957, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24183806, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24183806, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24284972, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24284972, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24515771, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24515771, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24355447, ["Country", "Country_source"]] = fix_country_from_text("Denmark")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24355447, ["Country", "Country_source"]] = fix_country_from_text("Denmark")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24388681, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24388681, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 23915744, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 23915744, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 23944616, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 23944616, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24068440, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24068440, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24443268, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24443268, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24477172, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24477172, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24529771, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24529771, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24027420, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24027420, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24322649, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24322649, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24235966, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24235966, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24419921, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24419921, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 23933175, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 23933175, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24073795, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24073795, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24431104, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24431104, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 23911595, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 23911595, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 23911636, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 23911636, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 23912027, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 23912027, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 23912362, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 23912362, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24519169, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24519169, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 23905912, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 23905912, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 23908602, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 23908602, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 23912793, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 23912793, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 23915951, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 23915951, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 23917393, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 23917393, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 23920228, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 23920228, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 23922721, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 23922721, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 23928999, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 23928999, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 23929086, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 23929086, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 23932339, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 23932339, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 23933084, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 23933084, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 23933406, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 23933406, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 23933437, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 23933437, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 23934226, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 23934226, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 23934840, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 23934840, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24021898, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24021898, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24101096, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24101096, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24212624, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24212624, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24282571, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24282571, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24288667, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24288667, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24343741, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24343741, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24355909, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24355909, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24371455, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24371455, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24445733, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24445733, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24568474, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24568474, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 23956614, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 23956614, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24020869, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24020869, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24045542, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24045542, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24064977, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24064977, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24091718, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24091718, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24101654, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24101654, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24119165, ["Country", "Country_source"]] = fix_country_from_text("Sweden")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24119165, ["Country", "Country_source"]] = fix_country_from_text("Sweden")

# Convert to Taiwan in the base dataset and to China in the matching Globocan dataset because, in the latter, there is no Taiwan (added to China)
df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24121978, ["Country", "Country_source"]] = fix_country_from_text("Taiwan")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24121978, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24135896, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24135896, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24147471, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24147471, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24291674, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24291674, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24323185, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24323185, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24327327, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24327327, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24447304, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24447304, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24456610, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24456610, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24476678, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24476678, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24485017, ["Country", "Country_source"]] = fix_country_from_text("South Korea")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24485017, ["Country", "Country_source"]] = fix_country_from_text("South Korea")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24494720, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24494720, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24524764, ["Country", "Country_source"]] = fix_country_from_text("Israel")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24524764, ["Country", "Country_source"]] = fix_country_from_text("Israel")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24557659, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24557659, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24561676, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24561676, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24563726, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24563726, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24572574, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24572574, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24633598, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24633598, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24652560, ["Country", "Country_source"]] = fix_country_from_text("Indonesia")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24652560, ["Country", "Country_source"]] = fix_country_from_text("Indonesia")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24467876, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24467876, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 23939153, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 23939153, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24269025, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24269025, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24315836, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24315836, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24321579, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24321579, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24000898, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24000898, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24100386, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24100386, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24119925, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24119925, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24169630, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24169630, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24268397, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24268397, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24354191, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24354191, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24594798, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24594798, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24079174, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24079174, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24374273, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24374273, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24468103, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24468103, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24173126, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24173126, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 23992642, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 23992642, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24012712, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24012712, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24051223, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24051223, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24065086, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24065086, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24072772, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24072772, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24114959, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24114959, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24141191, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24141191, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24194905, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24194905, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24209690, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24209690, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24223734, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24223734, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24282590, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24282590, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24348764, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24348764, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24396443, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24396443, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24425307, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24425307, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24460346, ["Country", "Country_source"]] = fix_country_from_text("Sri Lanka")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24460346, ["Country", "Country_source"]] = fix_country_from_text("Sri Lanka")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24472554, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24472554, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24473398, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24473398, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24503899, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24503899, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24513320, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24513320, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24529623, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24529623, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24535538, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24535538, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24548933, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24548933, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24648920, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24648920, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24649006, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24649006, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == 24649027, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == 24649027, ["Country", "Country_source"]] = fix_country_from_text("China")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2300000_base.loc[df_parsedX_2300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2300000_match_GLOBOCAN.loc[df_parsedX_2300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [72]:
df_parsedX_2300000_base.to_csv(DF_input_base + "parsedX_2300000.csv", index = False)
df_parsedX_2300000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_2300000.csv", index = False)

del df_parsedX_2300000_base
del df_parsedX_2300000_match_GLOBOCAN

### Correcting *parsedX_2400000.csv*

In [6]:
# Read csv
df_parsedX_2400000_base = pd.read_csv(DF_input_base + "parsedX_2400000.csv")
df_parsedX_2400000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_2400000.csv")

In [7]:
## Replace found errors by true Country values

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 24781824, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 24781824, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25019046, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25019046, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25253429, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25253429, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 24739432, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 24739432, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25010257, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25010257, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 24739578, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 24739578, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 24888817, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 24888817, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25198032, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25198032, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25307957, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25307957, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25329865, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25329865, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25121354, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25121354, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 24960584, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 24960584, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25124196, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25124196, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25145319, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25145319, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 24731713, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 24731713, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 24949329, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 24949329, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25138130, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25138130, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25330744, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25330744, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 24675430, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 24675430, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25323390, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25323390, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 24906397, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 24906397, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 24974129, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 24974129, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 24665346, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 24665346, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 24703097, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 24703097, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 24748865, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 24748865, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 24958732, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 24958732, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 24969896, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 24969896, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25120646, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25120646, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25169535, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25169535, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25258254, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25258254, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25349214, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25349214, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 24661934, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 24661934, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 24726916, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 24726916, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 24742083, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 24742083, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 24753208, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 24753208, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 24768732, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 24768732, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 24778001, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 24778001, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 24833010, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 24833010, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 24842651, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 24842651, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 24885701, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 24885701, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 24885892, ["Country", "Country_source"]] = fix_country_from_text("Ireland")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 24885892, ["Country", "Country_source"]] = fix_country_from_text("Ireland")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 24967520, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 24967520, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25008465, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25008465, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25015688, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25015688, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25073515, ["Country", "Country_source"]] = fix_country_from_text("Egypt")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25073515, ["Country", "Country_source"]] = fix_country_from_text("Egypt")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25077882, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25077882, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25117591, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25117591, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25168387, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25168387, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25189706, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25189706, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25196248, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25196248, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25209180, ["Country", "Country_source"]] = fix_country_from_text("Bulgaria")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25209180, ["Country", "Country_source"]] = fix_country_from_text("Bulgaria")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25216851, ["Country", "Country_source"]] = fix_country_from_text("Israel")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25216851, ["Country", "Country_source"]] = fix_country_from_text("Israel")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25286960, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25286960, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25319495, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25319495, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25333035, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25333035, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25351969, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25351969, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25360374, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25360374, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25336387, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25336387, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25344426, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25344426, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 24709561, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 24709561, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 24718706, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 24718706, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 24817602, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 24817602, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25096762, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25096762, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25119587, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25119587, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25246023, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25246023, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25299496, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25299496, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25341666, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25341666, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25344197, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] ==25344197 , ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 24928688, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 24928688, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 24787292, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 24787292, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 24885890, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 24885890, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 24934165, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 24934165, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 24684754, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 24684754, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 24788770, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 24788770, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 24820496, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 24820496, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 24823992, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 24823992, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 24826815, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 24826815, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 24857291, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 24857291, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 24859810, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 24859810, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 24882675, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 24882675, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 24954107, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 24954107, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 24996144, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 24996144, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25009582, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25009582, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25013470, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25013470, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25070263, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25070263, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25086238, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25086238, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25171780, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25171780, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25176314, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25176314, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25343162, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25343162, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25343336, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25343336, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == 25356138, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == 25356138, ["Country", "Country_source"]] = fix_country_from_text("China")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2400000_base.loc[df_parsedX_2400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2400000_match_GLOBOCAN.loc[df_parsedX_2400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [8]:
df_parsedX_2400000_base.to_csv(DF_input_base + "parsedX_2400000.csv", index = False)
df_parsedX_2400000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_2400000.csv", index = False)

del df_parsedX_2400000_base
del df_parsedX_2400000_match_GLOBOCAN

### Correcting *parsedX_2500000.csv*

In [9]:
# Read csv
df_parsedX_2500000_base = pd.read_csv(DF_input_base + "parsedX_2500000.csv")
df_parsedX_2500000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_2500000.csv")

In [11]:
## Replace found errors by true Country values

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25936229, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25936229, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25575781, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25575781, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25569638, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25569638, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25420684, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25420684, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25420684, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25634551, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25537567, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25537567, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25566968, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25566968, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25749513, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25749513, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25691671, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25691671, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25462266, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25462266, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25501064, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25501064, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25634195, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25634195, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25470344, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25470344, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25634176, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25634176, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25853692, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25853692, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25394925, ["Country", "Country_source"]] = fix_country_from_text("Denmark")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25394925, ["Country", "Country_source"]] = fix_country_from_text("Denmark")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25510259, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25510259, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25828708, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25828708, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25853302, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25853302, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25926141 , ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25926141 , ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25421664, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25421664, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25569657, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25569657, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25847424, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25847424, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25568391, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25568391, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25467734, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25467734, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25482473, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25482473, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25601092, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25601092, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25630663, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25630663, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25743823, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25743823, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25674762, ["Country", "Country_source"]] = fix_country_from_text("South Korea")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25674762, ["Country", "Country_source"]] = fix_country_from_text("South Korea")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25526474, ["Country", "Country_source"]] = fix_country_from_text("South Korea")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25526474, ["Country", "Country_source"]] = fix_country_from_text("South Korea")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25700305, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25700305, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25789951, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25789951, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25832335, ["Country", "Country_source"]] = fix_country_from_text("Sweden")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25832335, ["Country", "Country_source"]] = fix_country_from_text("Sweden")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25804110, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25804110, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25380778, ["Country", "Country_source"]] = fix_country_from_text("Greece")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25380778, ["Country", "Country_source"]] = fix_country_from_text("Greece")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25519220, ["Country", "Country_source"]] = fix_country_from_text("Malaysia")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25519220, ["Country", "Country_source"]] = fix_country_from_text("Malaysia")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25455737, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25455737, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25577513, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25577513, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25663441, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25663441, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25704555, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25704555, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25829531, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25829531, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25855965, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25855965, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25893823, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25893823, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25997737, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25997737, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25364495, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25364495, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25364655, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25364655, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25367402, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25367402, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25417181, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25417181, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25488804, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25488804, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25521938, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25521938, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25532835, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25532835, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25546693, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25546693, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25553116, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25553116, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25592002, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25592002, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25604141, ["Country", "Country_source"]] = fix_country_from_text("Iran")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25604141, ["Country", "Country_source"]] = fix_country_from_text("Iran")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25642326, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25642326, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25648640, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25648640, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25738299, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25738299, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25765654, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25765654, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25787964, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25787964, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25788186, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25788186, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25845809, ["Country", "Country_source"]] = fix_country_from_text("Israel")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25845809, ["Country", "Country_source"]] = fix_country_from_text("Israel")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25865744, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25865744, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25882093, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25882093, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25890191, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25890191, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25897337, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25897337, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25905177, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25905177, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25905927, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25905927, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25943888, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25943888, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 26021563, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 26021563, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25758031, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25758031, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25658127, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25658127, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 26001395, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 26001395, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25445350, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25445350, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25528484, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25528484, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25535587, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25535587, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25542236, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25542236, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25720523, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25720523, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25795146, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25795146, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25802479, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25802479, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25837847, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25837847, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25984538, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25984538, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 26014616, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 26014616, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25501061, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25501061, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25677263, ["Country", "Country_source"]] = fix_country_from_text("Paraguay")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25677263, ["Country", "Country_source"]] = fix_country_from_text("Paraguay")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25701382, ["Country", "Country_source"]] = fix_country_from_text("Paraguay")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25701382, ["Country", "Country_source"]] = fix_country_from_text("Paraguay")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25674748, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25674748, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25827538, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25827538, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25827539, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25827539, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25907508, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25907508, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25788199, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25788199, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25405790, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25405790, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25432874, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25432874, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25438768, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25438768, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25446590, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25446590, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25498280, ["Country", "Country_source"]] = fix_country_from_text("Türkiye")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25498280, ["Country", "Country_source"]] = fix_country_from_text("Türkiye")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25573191, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25573191, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25630222, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25630222, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25640356, ["Country", "Country_source"]] = fix_country_from_text("Sri Lanka")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25640356, ["Country", "Country_source"]] = fix_country_from_text("Sri Lanka")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25698579, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25698579, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25700543, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25700543, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25709019, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25709019, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25721026, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25721026, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25732307, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25732307, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25824795, ["Country", "Country_source"]] = fix_country_from_text("Sri Lanka")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25824795, ["Country", "Country_source"]] = fix_country_from_text("Sri Lanka")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25873049, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25873049, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25874341, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25874341, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25882313, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25882313, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25925952, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] ==25925952, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25973308, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25973308, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25982451, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25982451, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == 25996781, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == 25996781, ["Country", "Country_source"]] = fix_country_from_text("China")

#df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2500000_base.loc[df_parsedX_2500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2500000_match_GLOBOCAN.loc[df_parsedX_2500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [12]:
df_parsedX_2500000_base.to_csv(DF_input_base + "parsedX_2500000.csv", index = False)
df_parsedX_2500000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_2500000.csv", index = False)

del df_parsedX_2500000_base
del df_parsedX_2500000_match_GLOBOCAN

### Correcting *parsedX_2600000.csv*

In [13]:
# Read csv
df_parsedX_2600000_base = pd.read_csv(DF_input_base + "parsedX_2600000.csv")
df_parsedX_2600000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_2600000.csv")

In [14]:
## Replace found errors by true Country values

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26047983, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26047983, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26149098, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26149098, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26078215, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26078215, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26399387, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26399387, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26414901, ["Country", "Country_source"]] = fix_country_from_text("Ireland")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26414901, ["Country", "Country_source"]] = fix_country_from_text("Ireland")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26530358, ["Country", "Country_source"]] = fix_country_from_text("Ireland")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26530358, ["Country", "Country_source"]] = fix_country_from_text("Ireland")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26683282, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26683282, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26222501, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26222501, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26448015, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26448015, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26109347, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26109347, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26210402, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26210402, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26235418, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26235418, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26340057, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26340057, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26405550, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26405550, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26482183, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26482183, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26700672, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26700672, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26511210, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26511210, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26642762, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26642762, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26125142, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26125142, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26266182, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26266182, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26678861, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26678861, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26448009, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26448009, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26469889, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26469889, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26346926, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26346926, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26435889, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26435889, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26106210, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26106210, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26304895, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26304895, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26151503, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26151503, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26173602, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26173602, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26275804, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26275804, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26304819, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26304819, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26434861, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26434861, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26437854, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26437854, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26049699, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26049699, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26082811, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26082811, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26125223, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26125223, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26156521, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26156521, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26160057, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26160057, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26166665, ["Country", "Country_source"]] = fix_country_from_text("Portugal")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26166665, ["Country", "Country_source"]] = fix_country_from_text("Portugal")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26181546, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26181546, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26190888, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26190888, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26221192, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26221192, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26279470, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26279470, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26287406, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26287406, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26311478, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26311478, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26316877, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26316877, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26343244, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26343244, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26381428, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26381428, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26385356, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26385356, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26415497, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26415497, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26446460, ["Country", "Country_source"]] = fix_country_from_text("Iran")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26446460, ["Country", "Country_source"]] = fix_country_from_text("Iran")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26466668, ["Country", "Country_source"]] = fix_country_from_text("Greece")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26466668, ["Country", "Country_source"]] = fix_country_from_text("Greece")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26522776, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26522776, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26611382, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26611382, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26631394, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26631394, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26656323, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26656323, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26419685, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26419685, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26681384, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26681384, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26073129, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26073129, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26112485, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26112485, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26213910, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26213910, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26217805, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26217805, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26295406, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26295406, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26316883, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26316883, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26335153, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26335153, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26343499, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26343499, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26496413, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26496413, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26549463, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26549463, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26631442, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26631442, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26706364, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26706364, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26421962, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26421962, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26706169, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26706169, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26090299, ["Country", "Country_source"]] = fix_country_from_text("Paraguay")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26090299, ["Country", "Country_source"]] = fix_country_from_text("Paraguay")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26188121, ["Country", "Country_source"]] = fix_country_from_text("Paraguay")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26188121, ["Country", "Country_source"]] = fix_country_from_text("Paraguay")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26329327, ["Country", "Country_source"]] = fix_country_from_text("Paraguay")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26329327, ["Country", "Country_source"]] = fix_country_from_text("Paraguay")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26632735, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26632735, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26096294, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26096294, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26411564, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26411564, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26560753, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26560753, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26224287, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26224287, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26035636, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26035636, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26122771, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26122771, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26124805, ["Country", "Country_source"]] = fix_country_from_text("Sudan")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26124805, ["Country", "Country_source"]] = fix_country_from_text("Sudan")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26126943, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26126943, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26169763, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26169763, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26261599, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26261599, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26276860, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26276860, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26280346, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26280346, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26324182, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26324182, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26344589, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26344589, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26350232, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26350232, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26352402, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26352402, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26369691, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26369691, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26432997, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26432997, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26451770, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26451770, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26459575, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26459575, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26482853, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26482853, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26574634, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26574634, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26654950, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26654950, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26668648, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26668648, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == 26699322, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == 26699322, ["Country", "Country_source"]] = fix_country_from_text("Canada")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2600000_base.loc[df_parsedX_2600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2600000_match_GLOBOCAN.loc[df_parsedX_2600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [15]:
df_parsedX_2600000_base.to_csv(DF_input_base + "parsedX_2600000.csv", index = False)
df_parsedX_2600000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_2600000.csv", index = False)

del df_parsedX_2600000_base
del df_parsedX_2600000_match_GLOBOCAN

### Correcting *parsedX_2700000.csv*

In [16]:
# Read csv
df_parsedX_2700000_base = pd.read_csv(DF_input_base + "parsedX_2700000.csv")
df_parsedX_2700000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_2700000.csv")

In [18]:
## Replace found errors by true Country values

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 26974049, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 26974049, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27292875, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27292875, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27142453, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27142453, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27207581, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27207581, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 26817902, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 26817902, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27239393, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27239393, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27369807, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27369807, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 26886624, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 26886624, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 26830275, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 26830275, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 26902155, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 26902155, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 26921575, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 26921575, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 26944049, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 26944049, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 26906470, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 26906470, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27099806, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27099806, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 26871844, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 26871844, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27013905, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27013905, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27081632, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27081632, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27131889, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27131889, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27235944, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27235944, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 26765418, ["Country", "Country_source"]] = fix_country_from_text("South Korea")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 26765418, ["Country", "Country_source"]] = fix_country_from_text("South Korea")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 26937937, ["Country", "Country_source"]] = fix_country_from_text("South Korea")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 26937937, ["Country", "Country_source"]] = fix_country_from_text("South Korea")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27374881, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27374881, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 26765477, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 26765477, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27043668, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27043668, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27057916, ["Country", "Country_source"]] = fix_country_from_text("South Korea")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27057916, ["Country", "Country_source"]] = fix_country_from_text("South Korea")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 26896570, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 26896570, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27005806, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27005806, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27015862, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27015862, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27050912, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27050912, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27271270, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27271270, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27272782, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27272782, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27342746, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27342746, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 26729349, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 26729349, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 26817896, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 26817896, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 26828843, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 26828843, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 26838800, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 26838800, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 26847449, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 26847449, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 26862179, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 26862179, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 26874618, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 26874618, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27038831, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27038831, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27073117, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27073117, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27125973, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27125973, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27146669, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27146669, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27156077, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27156077, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27179143, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27179143, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27223688, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27223688, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27230973, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27230973, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27248849, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27248849, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27251414, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27251414, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27318562, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27318562, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27357927, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27357927, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27421752, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27421752, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 26777299, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 26777299, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 26828111, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 26828111, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 26772481, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 26772481, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 26868456, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 26868456, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 26895493, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 26895493, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 26924581, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 26924581, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 26971296, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 26971296, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27286681, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27286681, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27082613, ["Country", "Country_source"]] = fix_country_from_text("South Korea")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27082613, ["Country", "Country_source"]] = fix_country_from_text("South Korea")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27100425, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27100425, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 26884668, ["Country", "Country_source"]] = fix_country_from_text("Brunei Darussalam")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 26884668, ["Country", "Country_source"]] = fix_country_from_text("Brunei Darussalam")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27167387, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27167387, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 26718650, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 26718650, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 26756217, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 26756217, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 26774454, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 26774454, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 26783042, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 26783042, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 26785717, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 26785717, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 26807546, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 26807546, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 26820712, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 26820712, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 26895099, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 26895099, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 26909923, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 26909923, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 26922048, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 26922048, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 26935047, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 26935047, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 26972577, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 26972577, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 26991938, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 26991938, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27035745, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27035745, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27050147, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27050147, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27070469, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27070469, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27070877, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27070877, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27073494, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27073494, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27121050, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27121050, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27149454, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27149454, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27149664, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27149664, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27155466, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27155466, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27175797, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27175797, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27270327, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27270327, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27278879, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27278879, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == 27347118, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == 27347118, ["Country", "Country_source"]] = fix_country_from_text("China")

#df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2700000_base.loc[df_parsedX_2700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2700000_match_GLOBOCAN.loc[df_parsedX_2700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [19]:
df_parsedX_2700000_base.to_csv(DF_input_base + "parsedX_2700000.csv", index = False)
df_parsedX_2700000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_2700000.csv", index = False)

del df_parsedX_2700000_base
del df_parsedX_2700000_match_GLOBOCAN

### Correcting *parsedX_2800000.csv*

In [20]:
# Read csv
df_parsedX_2800000_base = pd.read_csv(DF_input_base + "parsedX_2800000.csv")
df_parsedX_2800000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_2800000.csv")

In [22]:
## Replace found errors by true Country values

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 28198377, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 28198377, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27665490, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27665490, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27683017, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27683017, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27787590, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27787590, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 28033219, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 28033219, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 28050791, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 28050791, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27663926, ["Country", "Country_source"]] = fix_country_from_text("Romania")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27663926, ["Country", "Country_source"]] = fix_country_from_text("Romania")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27430869, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27430869, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27562128, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27562128, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27668530, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27668530, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27933657, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27933657, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 28050799, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 28050799, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27579604, ["Country", "Country_source"]] = fix_country_from_text("Norway")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27579604, ["Country", "Country_source"]] = fix_country_from_text("Norway")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27629291, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27629291, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27583885, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27583885, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27507083, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27507083, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27863301, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27863301, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 28151550, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 28151550, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27540011, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27540011, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 28197752, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 28197752, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27892939, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27892939, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 28060745, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 28060745, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 28138630, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 28138630, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27565505, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27565505, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27585955, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27585955, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27605336, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27605336, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27646764, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27646764, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27656876, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27656876, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27657109, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27657109, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27657319, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27657319, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27777777, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27777777, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27817104, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27817104, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27877247, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27877247, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27890941, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27890941, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27896669, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27896669, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27930560, ["Country", "Country_source"]] = fix_country_from_text("Taiwan")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27930560, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27942988, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27942988, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 28049037, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 28049037, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 28050734, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 28050734, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 28074255, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 28074255, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 28074323, ["Country", "Country_source"]] = fix_country_from_text("Israel")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 28074323, ["Country", "Country_source"]] = fix_country_from_text("Israel")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 28089377, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 28089377, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 28111515, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 28111515, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 28112410, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 28112410, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 28124990, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 28124990, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 28144830, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 28144830, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 28174341, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 28174341, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27658668, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27658668, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 28073307, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 28073307, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27455223, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27455223, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27591381, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27591381, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27639478, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27639478, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27746255, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27746255, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27752939, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27752939, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27766434, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27766434, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27889662, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27889662, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 28063068, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 28063068, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 28079012, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 28079012, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 28105425, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 28105425, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 28122528, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 28122528, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 28131812, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 28131812, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 28174065, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 28174065, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 28191467, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 28191467, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27716767, ["Country", "Country_source"]] = fix_country_from_text("Paraguay")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27716767, ["Country", "Country_source"]] = fix_country_from_text("Paraguay")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27725415, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27725415, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27446465, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27446465, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27463610, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27463610, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27490023, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27490023, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27556505, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27556505, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27571770, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27571770, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27572636, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27572636, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27683106, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27683106, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27826625, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27826625, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27841002, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27841002, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27841002, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27841002, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27917905, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27917905, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 27998774, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 27998774, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 28040262, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 28040262, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 28073187, ["Country", "Country_source"]] = fix_country_from_text("Türkiye")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 28073187, ["Country", "Country_source"]] = fix_country_from_text("Türkiye")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 28087447, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 28087447, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == 28126011, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == 28126011, ["Country", "Country_source"]] = fix_country_from_text("China")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2800000_base.loc[df_parsedX_2800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2800000_match_GLOBOCAN.loc[df_parsedX_2800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [23]:
df_parsedX_2800000_base.to_csv(DF_input_base + "parsedX_2800000.csv", index = False)
df_parsedX_2800000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_2800000.csv", index = False)

del df_parsedX_2800000_base
del df_parsedX_2800000_match_GLOBOCAN

### Correcting *parsedX_2900000.csv*

In [24]:
# Read csv
df_parsedX_2900000_base = pd.read_csv(DF_input_base + "parsedX_2900000.csv")
df_parsedX_2900000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_2900000.csv")

In [25]:
## Replace found errors by true Country values

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28499594, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28499594, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28719413, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28719413, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28431616, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28431616, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28548372, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28548372, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28655066, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28655066, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28668260, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28668260, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28674914, ["Country", "Country_source"]] = fix_country_from_text("Ireland")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28674914, ["Country", "Country_source"]] = fix_country_from_text("Ireland")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28716152, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28716152, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28814206, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28814206, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28830697, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28830697, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28738235, ["Country", "Country_source"]] = fix_country_from_text("Portugal")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28738235, ["Country", "Country_source"]] = fix_country_from_text("Portugal")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28806573, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28806573, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28259356, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28259356, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28373458, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28373458, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28407301, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28407301, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28629398, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28629398, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28653894, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28653894, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28875243, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28875243, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28763126, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28763126, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28495502, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28495502, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28702499, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28702499, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28345916, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28345916, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28441796, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28441796, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28487075, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28487075, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28271192, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28271192, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28207858, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28207858, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28219303, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28219303, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28432617, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28432617, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28522594, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28522594, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28285009, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28285009, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28320152, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28320152, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28399836, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28399836, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28443209, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28443209, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28445439, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28445439, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28467142, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28467142, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28518072, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28518072, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28643684, ["Country", "Country_source"]] = fix_country_from_text("Uruguay")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28643684, ["Country", "Country_source"]] = fix_country_from_text("Uruguay")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28645022, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28645022, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28707579, ["Country", "Country_source"]] = fix_country_from_text("Belarus")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28707579, ["Country", "Country_source"]] = fix_country_from_text("Belarus")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28797293, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28797293, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28810912, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28810912, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28853247, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28853247, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28871632, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28871632, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28448799, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28448799, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28467626, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28467626, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28561159, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28561159, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28620964, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28620964, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28627971, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28627971, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28273793, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28273793, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28289860, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28289860, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28433864, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28433864, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28558708, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28558708, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28558783, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28558783, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28746889, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28746889, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28844340, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28844340, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28242375, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28242375, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28296773, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28296773, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28356971, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28356971, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28366857, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28366857, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28372569, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28372569, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28393218, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28393218, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28442930, ["Country", "Country_source"]] = fix_country_from_text("Egypt")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28442930, ["Country", "Country_source"]] = fix_country_from_text("Egypt")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28454092, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28454092, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28454262, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28454262, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28454439, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28454439, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28498428, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28498428, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28656308, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28656308, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28670365, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28670365, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28716639, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28716639, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28789368, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28789368, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28791371, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28791371, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28800883, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28800883, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28826084, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28826084, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28849917, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == 28849917, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == 28855534, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] ==28855534, ["Country", "Country_source"]] = fix_country_from_text("China")

#df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_2900000_base.loc[df_parsedX_2900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_2900000_match_GLOBOCAN.loc[df_parsedX_2900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [26]:
df_parsedX_2900000_base.to_csv(DF_input_base + "parsedX_2900000.csv", index = False)
df_parsedX_2900000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_2900000.csv", index = False)

del df_parsedX_2900000_base
del df_parsedX_2900000_match_GLOBOCAN

### Correcting *parsedX_300000.csv*

In [27]:
# Read csv
df_parsedX_300000_base = pd.read_csv(DF_input_base + "parsedX_300000.csv")
df_parsedX_300000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_300000.csv")

In [28]:
## Replace found errors by true Country values

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 3162999, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 3162999, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 3347787, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 3347787, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 3163142, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 3163142, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 3393815, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 3393815, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 3393816, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 3393816, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 3374767, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 3374767, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 3050155, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 3050155, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 3232476, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 3232476, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 3176994, ["Country", "Country_source"]] = fix_country_from_text("Sweden")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 3176994, ["Country", "Country_source"]] = fix_country_from_text("Sweden")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 3211366, ["Country", "Country_source"]] = fix_country_from_text("Sweden")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 3211366, ["Country", "Country_source"]] = fix_country_from_text("Sweden")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 3323456, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 3323456, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2689843, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2689843, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2689884, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2689884, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2690041, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2690041, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2690131, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2690131, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2690211, ["Country", "Country_source"]] = fix_country_from_text("Tunisia")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2690211, ["Country", "Country_source"]] = fix_country_from_text("Tunisia")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2690222, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2690222, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2690224, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2690224, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2690225, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2690225, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2690226, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2690226, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2690228, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2690228, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2690232, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2690232, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2690233, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2690233, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2690258, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2690258, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2690259, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2690259, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2690260, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2690260, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2690291, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2690291, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2690296, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2690296, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2690395, ["Country", "Country_source"]] = fix_country_from_text("Czechia")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2690395, ["Country", "Country_source"]] = fix_country_from_text("Czechia")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2690405, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2690405, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2690441, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2690441, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2690543, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2690543, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2690635, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2690635, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2690637, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2690637, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2690638, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2690638, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2690729, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2690729, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2690741, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2690741, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2690853, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2690853, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2690970, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2690970, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2691001, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2691001, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2691007, ["Country", "Country_source"]] = fix_country_from_text("Ireland")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2691007, ["Country", "Country_source"]] = fix_country_from_text("Ireland")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2691008, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2691008, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2691204, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2691204, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2691206, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2691206, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2691313, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2691313, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2691405, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2691405, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2691435, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2691435, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2691436, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2691436, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2691452, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2691452, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2691458, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2691458, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2691496, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2691496, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2691578, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2691578, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2691579, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2691579, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2691584, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2691584, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2691647, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2691647, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2691716, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2691716, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2691717, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2691717, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2691718, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2691718, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2691943, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2691943, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2691947, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2691947, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2692002, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2692002, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2692003, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2692003, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2692040, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2692040, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2692104, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2692104, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2692172, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2692172, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2692271, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2692271, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2692272, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2692272, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2692311, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2692311, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2692317, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2692317, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2692327, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2692327, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2692331, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2692331, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2692404, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2692404, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2692407, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2692407, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2692424, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2692424, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2692438, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2692438, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2692439, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2692439, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2692443, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2692443, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2692444, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2692444, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2692445, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2692445, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2692446, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2692446, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2692564, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2692564, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2692565, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2692565, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2692574, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2692574, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2692578, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2692578, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2692579, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2692579, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2692605, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2692605, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2692617, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2692617, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2692629, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2692629, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2692651, ["Country", "Country_source"]] = fix_country_from_text("Norway")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2692651, ["Country", "Country_source"]] = fix_country_from_text("Norway")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2692699, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2692699, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2692703, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2692703, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2692728, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2692728, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2692776, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2692776, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2692830, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2692830, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2692866, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2692866, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2692911, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2692911, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2692912, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2692912, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2692914, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2692914, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2692916, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2692916, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2693007, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2693007, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2693024, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2693024, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2693025, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2693025, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2693026, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2693026, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2693107, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2693107, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2693227, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2693227, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2693296, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2693296, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2693300, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2693300, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2693301, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2693301, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2693302, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2693302, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2693324, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2693324, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2693417, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2693417, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2693493, ["Country", "Country_source"]] = fix_country_from_text("Ireland")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2693493, ["Country", "Country_source"]] = fix_country_from_text("Ireland")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2693519, ["Country", "Country_source"]] = fix_country_from_text("Tunisia")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2693519, ["Country", "Country_source"]] = fix_country_from_text("Tunisia")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2693521, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2693521, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2693522, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] ==2693522, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2693523, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2693523, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2693670, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2693670, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2693697, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2693697, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2693703, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2693703, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2693866, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2693866, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2693875, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2693875, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2693980, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2693980, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2693981, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2693981, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2693982, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2693982, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2693987, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2693987, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2693989, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2693989, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2694095, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2694095, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2694123, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2694123, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2694127, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2694127, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2694160, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2694160, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2694161, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2694161, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2694165, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2694165, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2694167, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2694167, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2694211, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2694211, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2694309, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2694309, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2694419, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2694419, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2694422, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2694422, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2694625, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2694625, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2694655, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2694655, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2694674, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2694674, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2694683, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2694683, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2694756, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2694756, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2694813, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2694813, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2694825, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2694825, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2694826, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2694826, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2694827, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2694827, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2694831, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2694831, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2694834, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2694834, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2694850, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2694850, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2694872, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2694872, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2694894, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2694894, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2694897, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2694897, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2694903, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2694903, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2694924, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2694924, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2694925, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2694925, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2694926, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2694926, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2694944, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2694944, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2695056, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2695056, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2695154, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2695154, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2695156, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2695156, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2695158, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2695158, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2695168, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2695168, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2695181, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2695181, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2695183, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2695183, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2695228, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2695228, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2695240, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2695240, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2695241, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2695241, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2695246, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2695246, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2695249, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2695249, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2695261, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2695261, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2695263, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2695263, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2695264, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2695264, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2695294, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2695294, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2695300, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2695300, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2695352, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2695352, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2695474, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2695474, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2695485, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2695485, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2695492, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2695492, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2695500, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2695500, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2695530, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2695530, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2695531, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2695531, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2695534, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2695534, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2695536, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2695536, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2695562, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2695562, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2695564, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2695564, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2695569, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2695569, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2695626, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2695626, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2695627, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2695627, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2695649, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2695649, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2695714, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2695714, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2695716, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2695716, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2695722, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2695722, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2695854, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2695854, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2696030, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2696030, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2696031, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2696031, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2696034, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2696034, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2696035, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2696035, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2696037, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2696037, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2696038, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2696038, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2696064, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2696064, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2696087, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2696087, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2696091, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2696091, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2696254, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2696254, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2696342, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2696342, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2696343, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2696343, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2696383, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2696383, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2696402, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2696402, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2696422, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2696422, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2696424, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2696424, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2696491, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2696491, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2696494, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2696494, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2696564, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2696564, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2696581, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2696581, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2696585, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2696585, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2696589, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2696589, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2696629, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2696629, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2696662, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2696662, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2696675, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2696675, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2696679, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2696679, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2696684, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2696684, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2696727, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2696727, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2696739, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2696739, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2696842, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2696842, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2696843, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2696843, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2696844, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2696844, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2696875, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2696875, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2696908, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2696908, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2696910, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2696910, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2696911, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2696911, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2696912, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2696912, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2696913, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2696913, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2696914, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2696914, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697029, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697029, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697054, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697054, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697181, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697181, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697198, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697198, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697216, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697216, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697324, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697324, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697329, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697329, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697330, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697330, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697331, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697331, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697357, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697357, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697358, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697358, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697389, ["Country", "Country_source"]] = fix_country_from_text("Croatia")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697389, ["Country", "Country_source"]] = fix_country_from_text("Croatia")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697390, ["Country", "Country_source"]] = fix_country_from_text("Slovenia")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697390, ["Country", "Country_source"]] = fix_country_from_text("Slovenia")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697391, ["Country", "Country_source"]] = fix_country_from_text("Croatia")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697391, ["Country", "Country_source"]] = fix_country_from_text("Croatia")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697399, ["Country", "Country_source"]] = fix_country_from_text("Croatia")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697399, ["Country", "Country_source"]] = fix_country_from_text("Croatia")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697400, ["Country", "Country_source"]] = fix_country_from_text("Croatia")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697400, ["Country", "Country_source"]] = fix_country_from_text("Croatia")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697402, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697402, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697403, ["Country", "Country_source"]] = fix_country_from_text("Slovenia")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697403, ["Country", "Country_source"]] = fix_country_from_text("Slovenia")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697413, ["Country", "Country_source"]] = fix_country_from_text("Croatia")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697413, ["Country", "Country_source"]] = fix_country_from_text("Croatia")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697414, ["Country", "Country_source"]] = fix_country_from_text("Croatia")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697414, ["Country", "Country_source"]] = fix_country_from_text("Croatia")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697415, ["Country", "Country_source"]] = fix_country_from_text("Croatia")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697415, ["Country", "Country_source"]] = fix_country_from_text("Croatia")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697416, ["Country", "Country_source"]] = fix_country_from_text("Croatia")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697416, ["Country", "Country_source"]] = fix_country_from_text("Croatia")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697428, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697428, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697430, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697430, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697435, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697435, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697441, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697441, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697448, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697448, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697452, ["Country", "Country_source"]] = fix_country_from_text("Norway")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697452, ["Country", "Country_source"]] = fix_country_from_text("Norway")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697469, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697469, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697471, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697471, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697473, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697473, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697529, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697529, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697560, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697560, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697574, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697574, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697576, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697576, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697649, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697649, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697656, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697656, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697660, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697660, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697662, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697662, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697665, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697665, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697666, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697666, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697710, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697710, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697711, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697711, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697724, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697724, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697730, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697730, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697757, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697757, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697789, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697789, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697801, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697801, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697938, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697938, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697939, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697939, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697941, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697941, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697961, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697961, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2698074, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2698074, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2698289, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2698289, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2698302, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2698302, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2698306, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2698306, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2698439, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2698439, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2698440, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2698440, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2698448, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2698448, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2698449, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2698449, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2698450, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2698450, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2698451, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2698451, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2698452, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2698452, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2698552, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2698552, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2698618, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2698618, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2698803, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2698803, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2698806, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2698806, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2698808, ["Country", "Country_source"]] = fix_country_from_text("Denmark")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2698808, ["Country", "Country_source"]] = fix_country_from_text("Denmark")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2698811, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2698811, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2698837, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2698837, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2698842, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2698842, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2698875, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2698875, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2699035, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2699035, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2699132, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2699132, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2699136, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2699136, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2699155, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2699155, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2699449, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2699449, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2699450, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2699450, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2699629, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2699629, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2699636, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2699636, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2699641, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2699641, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2699741, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2699741, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2699850, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2699850, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2700062, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2700062, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2700284, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2700284, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2700343, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2700343, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2700347, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2700347, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2700431, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2700431, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2700663, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2700663, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2700665, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2700665, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2700948, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2700948, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2700993, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2700993, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2701081, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2701081, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2701082, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2701082, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2701085, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2701085, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2701360, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2701360, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2701362, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2701362, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2701401, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2701401, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2701404, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2701404, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2701410, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2701410, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2701413, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2701413, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2701414, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2701414, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2701416, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2701416, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2701417, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2701417, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2701418, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2701418, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2701419, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2701419, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2701506, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2701506, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2701535, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2701535, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2701572, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2701572, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2701697, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2701697, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2701698, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2701698, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2701699, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2701699, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2701710, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2701710, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2701714, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2701714, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2701718, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2701718, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2701728, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2701728, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2701732, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2701732, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2701737, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2701737, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2701738, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2701738, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2701768, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2701768, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2701768, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2701768, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2701809, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2701809, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2701899, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2701899, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702038, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702038, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702039, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702039, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702040, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702040, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702075, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702075, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702359, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702359, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702378, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702378, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702381, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702381, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702383, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702383, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702386, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702386, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702388, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702388, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702390, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702390, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702395, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702395, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702397, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702397, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702405, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702405, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702406, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702406, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702411, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702411, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702418, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702418, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702423, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702423, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702430, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702430, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702433, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702433, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702434, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702434, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702439, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702439, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702442, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702442, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702444, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702444, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702456, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702456, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702505, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702505, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702571, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702571, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702576, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702576, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702581, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702581, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702589, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702589, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702593, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702593, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702599, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702599, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702614, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702614, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702619, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702619, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702624, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702624, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702635, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702635, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702643, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702643, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702646, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702646, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702670, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702670, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702679, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702679, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702707, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702707, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702721, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702721, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702722, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702722, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702725, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702725, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702727, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702727, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702750, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702750, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702815, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702815, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702935, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702935, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702983, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702983, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702992, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702992, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702995, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702995, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2702998, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2702998, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703000, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703000, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703001, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703001, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703004, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703004, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703007, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703007, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703055, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703055, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703058, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703058, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703059, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703059, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703133, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703133, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703135, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703135, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703136, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703136, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703158, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703158, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703159, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703159, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703166, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703166, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703168, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703168, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703173, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703173, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703177, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703177, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703182, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703182, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703185, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703185, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703189, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703189, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703227, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703227, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703230, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703230, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703260, ["Country", "Country_source"]] = fix_country_from_text("Czechia")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703260, ["Country", "Country_source"]] = fix_country_from_text("Czechia")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703269, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703269, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703270, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703270, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703300, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703300, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703352, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703352, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703374, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703374, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703375, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703375, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703386, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703386, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703389, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703389, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703393, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703393, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703400, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703400, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703401, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703401, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703429, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703429, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703486, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703486, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703493, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703493, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703504, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703504, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703521, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703521, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703539, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703539, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703540, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703540, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703541, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703541, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703543, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703543, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703710, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703710, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703711, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703711, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703774, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703774, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703777, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703777, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703914, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703914, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2703970, ["Country", "Country_source"]] = fix_country_from_text("Norway")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2703970, ["Country", "Country_source"]] = fix_country_from_text("Norway")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2704026, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2704026, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2704040, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2704040, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2704078, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2704078, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2704118, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2704118, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2704120, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2704120, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2704121, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2704121, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2704123, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2704123, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2704125, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2704125, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2704126, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2704126, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2704127, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2704127, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2704128, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2704128, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2704129, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2704129, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2704131, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2704131, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2704135, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2704135, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2704136, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2704136, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2704137, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2704137, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2704138, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2704138, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2704139, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2704139, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2704140, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2704140, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2704173, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2704173, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2704176, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2704176, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2704178, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2704178, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2704231, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2704231, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2704281, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2704281, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2704283, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2704283, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2704284, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2704284, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2704285, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2704285, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2704337, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2704337, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2704377, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2704377, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2704427, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2704427, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2704461, ["Country", "Country_source"]] = fix_country_from_text("New Zealand")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2704461, ["Country", "Country_source"]] = fix_country_from_text("New Zealand")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2704463, ["Country", "Country_source"]] = fix_country_from_text("New Zealand")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2704463, ["Country", "Country_source"]] = fix_country_from_text("New Zealand")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2704507, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2704507, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2704508, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2704508, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2704536, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2704536, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2704553, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2704553, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2704554, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2704554, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2704556, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2704556, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2704800, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2704800, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2704809, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2704809, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2704851, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2704851, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2704895, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2704895, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2705147, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2705147, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2705247, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2705247, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2705397, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2705397, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2705401, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2705401, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2705426, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2705426, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2705442, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2705442, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2705452, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2705452, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2705509, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2705509, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2705718, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2705718, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2705730, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2705730, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2705733, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2705733, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2705734, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2705734, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2705738, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2705738, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2705739, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2705739, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2705742, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2705742, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2705744, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2705744, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2705747, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2705747, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2705749, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2705749, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2705751, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2705751, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2705753, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2705753, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2705759, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2705759, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2705788, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2705788, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2705845, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2705845, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2705867, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2705867, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2705875, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2705875, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2705925, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2705925, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2706060, ["Country", "Country_source"]] = fix_country_from_text("Slovenia")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2706060, ["Country", "Country_source"]] = fix_country_from_text("Slovenia")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2706061, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2706061, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2706062, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2706062, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2706064, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2706064, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2706107, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2706107, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2706132, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2706132, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2706134, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2706134, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2706212, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2706212, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2706325, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2706325, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2706413, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2706413, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2706592, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2706592, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2706619, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2706619, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2706629, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2706629, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2706630, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2706630, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2706637, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2706637, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2706639, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2706639, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2706641, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2706641, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2706732, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2706732, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2706737, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2706737, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2706738, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2706738, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2706742, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2706742, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2706832, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2706832, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2706834, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2706834, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2706839, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2706839, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2706896, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2706896, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2706903, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2706903, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2706989, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2706989, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2707068, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2707068, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2707078, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2707078, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2707133, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2707133, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2707230, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2707230, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2707490, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2707490, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2707521, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2707521, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2707535, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2707535, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2707633, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2707633, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2697571, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2697571, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2770932, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2770932, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2776781, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2776781, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 2854951, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 2854951, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 3327782, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 3327782, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 3356065, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 3356065, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == 3370631, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == 3370631, ["Country", "Country_source"]] = fix_country_from_text("Russia")

#df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_300000_base.loc[df_parsedX_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_300000_match_GLOBOCAN.loc[df_parsedX_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [29]:
df_parsedX_300000_base.to_csv(DF_input_base + "parsedX_300000.csv", index = False)
df_parsedX_300000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_300000.csv", index = False)

del df_parsedX_300000_base
del df_parsedX_300000_match_GLOBOCAN

### Correcting *parsedX_3000000.csv*

In [30]:
# Read csv
df_parsedX_3000000_base = pd.read_csv(DF_input_base + "parsedX_3000000.csv")
df_parsedX_3000000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_3000000.csv")

In [31]:
## Replace found errors by true Country values

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29427708, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29427708, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 28943004, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 28943004, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29408798, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29408798, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29027024, ["Country", "Country_source"]] = fix_country_from_text("Romania")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29027024, ["Country", "Country_source"]] = fix_country_from_text("Romania")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29200002, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29200002, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29224886, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29224886, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29324312, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29324312, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29372682, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29372682, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29372683, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29372683, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29372684, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29372684, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29372685, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29372685, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29372686, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29372686, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29372687, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29372687, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29372688, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29372688, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29372689, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29372689, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29469164, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29469164, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29541403, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29541403, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 28912581, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 28912581, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29416801, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29416801, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29054708, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29054708, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29048707, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29048707, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29099538, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29099538, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29120299, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29120299, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 28929714, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 28929714, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29217900, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29217900, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29258764, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29258764, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29298444, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29298444, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29387978, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29387978, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29399968, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29399968, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29467894, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29467894, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 28923102, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 28923102, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 28955945, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 28955945, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 28988350, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 28988350, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29158608, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29158608, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29273857, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29273857, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29317059, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29317059, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29333502, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29333502, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29361927, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29361927, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29375621, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29375621, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29390360, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29390360, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29405062, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29405062, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29439667, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29439667, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29451568, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29451568, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29454062, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29454062, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29468085, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29468085, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29484561, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29484561, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29492278, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29492278, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29500477, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29500477, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29518729, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29518729, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 28941682, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 28941682, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 28973007, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 28973007, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29017520, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29017520, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 28889078, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 28889078, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 28912828, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 28912828, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 28969609, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 28969609, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29107095, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29107095, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29158887, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29158887, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29189179, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29189179, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29278371, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29278371, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29307014, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29307014, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29329808, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29329808, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29346309, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29346309, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29456457, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29456457, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29303071, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29303071, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29346040, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29346040, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29492192, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29492192, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29696467, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29696467, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 30146944, ["Country", "Country_source"]] = fix_country_from_text("South Korea")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 30146944, ["Country", "Country_source"]] = fix_country_from_text("South Korea")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29346168, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29346168, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 28935610, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 28935610, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 28939029, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 28939029, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29027863, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29027863, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29029488, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29029488, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29070526, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29070526, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29085503, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29085503, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29088841, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29088841, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29113216, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29113216, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29160924, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29160924, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29212166, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29212166, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29227691, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29227691, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29235457, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29235457, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29254221, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29254221, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29274786, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29274786, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29316827, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29316827, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29344638, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29344638, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29374055, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29374055, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29387230, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29387230, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29399200, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29399200, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29433224, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29433224, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29486953, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29486953, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29492196, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29492196, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == 29496166, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == 29496166, ["Country", "Country_source"]] = fix_country_from_text("China")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3000000_base.loc[df_parsedX_3000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3000000_match_GLOBOCAN.loc[df_parsedX_3000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [32]:
df_parsedX_3000000_base.to_csv(DF_input_base + "parsedX_3000000.csv", index = False)
df_parsedX_3000000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_3000000.csv", index = False)

del df_parsedX_3000000_base
del df_parsedX_3000000_match_GLOBOCAN

### Correcting *parsedX_3100000.csv*

In [33]:
# Read csv
df_parsedX_3100000_base = pd.read_csv(DF_input_base + "parsedX_3100000.csv")
df_parsedX_3100000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_3100000.csv")

In [34]:
## Replace found errors by true Country values

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 29856243, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 29856243, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 29653120, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 29653120, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 30229205, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 30229205, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 29851693, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 29851693, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 29936076, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 29936076, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 29848725, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 29848725, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 30174762, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 30174762, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 29707810, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 29707810, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 29688847, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 29688847, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 29797143, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 29797143, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 29845457, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 29845457, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 29854570, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 29854570, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 30004912, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 30004912, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 29649201, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 29649201, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 29753941, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 29753941, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 29864472, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 29864472, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 30086794, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 30086794, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 30228363, ["Country", "Country_source"]] = fix_country_from_text("Ecuador")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 30228363, ["Country", "Country_source"]] = fix_country_from_text("Ecuador")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 30173360, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 30173360, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 30197505, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 30197505, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 29852068, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 29852068, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 29633024, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 29633024, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 29776427, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 29776427, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 30153836, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 30153836, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 30201356, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 30201356, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 30240295, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 30240295, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 29675328, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 29675328, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 29688241, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 29688241, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 29708784, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 29708784, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 29848545, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 29848545, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 29890024, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 29890024, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 30036439, ["Country", "Country_source"]] = fix_country_from_text("Israel")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 30036439, ["Country", "Country_source"]] = fix_country_from_text("Israel")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 30083079, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 30083079, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 30126136, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 30126136, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 30149619, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 30149619, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 30172100, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 30172100, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 30187420, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 30187420, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 30238389, ["Country", "Country_source"]] = fix_country_from_text("Taiwan")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 30238389, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 30244691, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 30244691, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 30099579, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 30099579, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 29727323, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 29727323, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 29780264, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 29780264, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 30087854, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 30087854, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 29550881, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 29550881, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 29596496, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 29596496, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 29601905, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 29601905, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 29649549, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 29649549, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 29666399, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 29666399, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 29690880, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 29690880, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 29730133, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 29730133, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 29987501, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 29987501, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 30007568, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 30007568, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 30017980, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 30017980, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 30036967, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 30036967, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 30039421, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 30039421, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 30048774, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 30048774, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 30095060, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 30095060, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 30166901, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 30166901, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 29803859, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 29803859, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 30132201, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 30132201, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 29928685, ["Country", "Country_source"]] = fix_country_from_text("Paraguay")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 29928685, ["Country", "Country_source"]] = fix_country_from_text("Paraguay")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 29783729, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 29783729, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 30073433, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 30073433, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 29620291, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 29620291, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 29798266, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 29798266, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 29805620, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 29805620, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 29877972, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 29877972, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 30015930, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 30015930, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 30020771, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 30020771, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 30048440, ["Country", "Country_source"]] = fix_country_from_text("New Zealand")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 30048440, ["Country", "Country_source"]] = fix_country_from_text("New Zealand")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 30060224, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 30060224, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 30146622, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 30146622, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 30170454, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 30170454, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 30185663, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 30185663, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 30220584, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 30220584, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == 30229245, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == 30229245, ["Country", "Country_source"]] = fix_country_from_text("China")

#df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3100000_base.loc[df_parsedX_3100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3100000_match_GLOBOCAN.loc[df_parsedX_3100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [35]:
df_parsedX_3100000_base.to_csv(DF_input_base + "parsedX_3100000.csv", index = False)
df_parsedX_3100000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_3100000.csv", index = False)

del df_parsedX_3100000_base
del df_parsedX_3100000_match_GLOBOCAN

### Correcting *parsedX_3200000.csv*

In [36]:
# Read csv
df_parsedX_3200000_base = pd.read_csv(DF_input_base + "parsedX_3200000.csv")
df_parsedX_3200000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_3200000.csv")

In [37]:
## Replace found errors by true Country values

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30585274, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30585274, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30318359, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30318359, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30770107, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30770107, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30556666, ["Country", "Country_source"]] = fix_country_from_text("Ireland")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30556666, ["Country", "Country_source"]] = fix_country_from_text("Ireland")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30605368, ["Country", "Country_source"]] = fix_country_from_text("Ireland")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30605368, ["Country", "Country_source"]] = fix_country_from_text("Ireland")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30660204, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30660204, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30778733, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30778733, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30267416, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30267416, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30509087, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30509087, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30509097, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30509097, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30522776, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30522776, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30538792, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30538792, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30543514, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30543514, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30881489, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30881489, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30903319, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30903319, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30281209, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30281209, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30499398, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30499398, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30830381, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30830381, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30394213, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30394213, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30588883, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30588883, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30594780, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30594780, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30817555, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30817555, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30304773, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30304773, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30368245, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30368245, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30519894, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30519894, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30584072, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30584072, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30867787, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30867787, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30267774, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30267774, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30293944, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30293944, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30301761, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30301761, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30306455, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30306455, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30316706, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30316706, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30318746, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30318746, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30366433, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30366433, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30390687, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30390687, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30499543, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30499543, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30511014, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30511014, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30539764, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30539764, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30568254, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30568254, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30576789, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30576789, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30589383, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30589383, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30642715, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30642715, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30649745, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30649745, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30701090, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30701090, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30721442, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30721442, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30841428, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30841428, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30881265, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30881265, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30712554, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30712554, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30271126, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30271126, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30345486, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30345486, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30425290, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30425290, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30515130, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30515130, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30627918, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30627918, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30739527, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30739527, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30775443, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30775443, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30835026, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30835026, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30839211, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30839211, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30904021, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30904021, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30562965, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30562965, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30631960, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30631960, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30641974, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30641974, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30721448, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30721448, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30758769, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30758769, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30813351, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30813351, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30816248, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30816248, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30832668, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30832668, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30704124, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30704124, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30901673, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30901673, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30657994, ["Country", "Country_source"]] = fix_country_from_text("Brunei Darussalam")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30657994, ["Country", "Country_source"]] = fix_country_from_text("Brunei Darussalam")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30661652, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30661652, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30711849, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30711849, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30788626, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30788626, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30719270, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30719270, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30316080, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30316080, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30414559, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30414559, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30456947, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30456947, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30500027, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30500027, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30526004, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30526004, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30543575, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30543575, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30702623, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30702623, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30747221, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30747221, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30776711, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30776711, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == 30867141, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == 30867141, ["Country", "Country_source"]] = fix_country_from_text("China")

#df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3200000_base.loc[df_parsedX_3200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3200000_match_GLOBOCAN.loc[df_parsedX_3200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [38]:
df_parsedX_3200000_base.to_csv(DF_input_base + "parsedX_3200000.csv", index = False)
df_parsedX_3200000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_3200000.csv", index = False)

del df_parsedX_3200000_base
del df_parsedX_3200000_match_GLOBOCAN

### Correcting *parsedX_3300000.csv*

In [6]:
# Read csv
df_parsedX_3300000_base = pd.read_csv(DF_input_base + "parsedX_3300000.csv")
df_parsedX_3300000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_3300000.csv")

In [7]:
## Replace found errors by true Country values

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 30954945, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 30954945, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31324932, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31324932, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31107807, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31107807, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 30981540, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 30981540, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31129938, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31129938, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31243585, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31243585, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31255836, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31255836, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31539207, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31539207, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31049795, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31049795, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31123853, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31123853, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31172643, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31172643, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31176573, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31176573, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31177128, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31177128, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31295151, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31295151, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31307200, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31307200, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31419008, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31419008, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31539170, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31539170, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31465861, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31465861, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31068071, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31068071, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31227248, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31227248, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31114735, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31114735, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31497440, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31497440, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31523585, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31523585, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31057112, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31057112, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31172982, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31172982, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31162490, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31162490, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31186023, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31186023, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 30999925, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 30999925, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31019757, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31019757, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31222819, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31222819, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31238937, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31238937, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31255178, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31255178, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31522686, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31522686, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 30919085, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 30919085, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 30934722, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 30934722, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31006017, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31006017, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31171910, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31171910, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31281694, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31281694, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31339352, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31339352, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31363464, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31363464, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31433788, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31433788, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31453005, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31453005, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31482393, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31482393, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31493190, ["Country", "Country_source"]] = fix_country_from_text("Belarus")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31493190, ["Country", "Country_source"]] = fix_country_from_text("Belarus")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31512414, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31512414, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31529263, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31529263, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31544847, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31544847, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31545410, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31545410, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31178298, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31178298, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31210389, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31210389, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31240514, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31240514, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31473959, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31473959, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31527615, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31527615, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31426795, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31426795, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31291743, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31291743, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31405063, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31405063, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 30922918, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 30922918, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31212024, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31212024, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31470629, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31470629, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 30920860, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 30920860, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 30952308, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 30952308, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 30963622, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 30963622, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31075266, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31075266, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31186689, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31186689, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31240705, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31240705, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31367249, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31367249, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31368349, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31368349, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31374044, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31374044, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31393486, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31393486, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31408429, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31408429, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31408781, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31408781, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31417635, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31417635, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31423251, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31423251, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31466809, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31466809, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == 31538504, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == 31538504, ["Country", "Country_source"]] = fix_country_from_text("China")

#df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3300000_base.loc[df_parsedX_3300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3300000_match_GLOBOCAN.loc[df_parsedX_3300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [8]:
df_parsedX_3300000_base.to_csv(DF_input_base + "parsedX_3300000.csv", index = False)
df_parsedX_3300000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_3300000.csv", index = False)

del df_parsedX_3300000_base
del df_parsedX_3300000_match_GLOBOCAN

### Correcting *parsedX_3400000.csv*

In [9]:
# Read csv
df_parsedX_3400000_base = pd.read_csv(DF_input_base + "parsedX_3400000.csv")
df_parsedX_3400000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_3400000.csv")

In [10]:
## Replace found errors by true Country values

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31889898, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31889898, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31610880, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31610880, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31806381, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31806381, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31653969 , ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31653969 , ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31800712, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31800712, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 32170155, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 32170155, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31624682, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31624682, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31694435, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31694435, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31825796, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31825796, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 32001804, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 32001804, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31927131, ["Country", "Country_source"]] = fix_country_from_text("United Arab Emirates")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31927131, ["Country", "Country_source"]] = fix_country_from_text("United Arab Emirates")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31974746, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31974746, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31977630, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31977630, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31991851, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31991851, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 32077247, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 32077247, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 32097951, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 32097951, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31557689, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31557689, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31593616, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31593616, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31645920, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31645920, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31652993, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31652993, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31660932, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31660932, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31678589, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31678589, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31694633, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31694633, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31702390, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31702390, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31719909, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31719909, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31775229, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31775229, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31827670, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31827670, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31841457, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31841457, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31841459, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31841459, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31841464, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31841464, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31841465, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31841465, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31841484, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31841484, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31841487, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31841487, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31844720, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31844720, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31861801, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31861801, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31877967, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31877967, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31913068, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31913068, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31997799, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31997799, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 32046103, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 32046103, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 32047249, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 32047249, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 32072655, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 32072655, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 32160426, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 32160426, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 32160445, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 32160445, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31824558, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31824558, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31959857, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31959857, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 32004398, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 32004398, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31719300, ["Country", "Country_source"]] = fix_country_from_text("Paraguay")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31719300, ["Country", "Country_source"]] = fix_country_from_text("Paraguay")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31933024, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31933024, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 32164265, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 32164265, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31936153, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31936153, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 32090092, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 32090092, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31557718, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31557718, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31592944, ["Country", "Country_source"]] = fix_country_from_text("Yemen")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31592944, ["Country", "Country_source"]] = fix_country_from_text("Yemen")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31629631, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31629631, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31651858, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31651858, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31729887, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31729887, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31731257, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31731257, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31771326, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31771326, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31772851, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31772851, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31799600, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31799600, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31810635, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31810635, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31813476, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31813476, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31843525, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31843525, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31866380, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31866380, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31897153, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31897153, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31926889, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31926889, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31972721, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31972721, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 31974627, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 31974627, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 32045269, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 32045269, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 32045670, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 32045670, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == 32061875, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == 32061875, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

#df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3400000_base.loc[df_parsedX_3400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3400000_match_GLOBOCAN.loc[df_parsedX_3400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [11]:
df_parsedX_3400000_base.to_csv(DF_input_base + "parsedX_3400000.csv", index = False)
df_parsedX_3400000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_3400000.csv", index = False)

del df_parsedX_3400000_base
del df_parsedX_3400000_match_GLOBOCAN

### Correcting *parsedX_3500000.csv*

In [12]:
# Read csv
df_parsedX_3500000_base = pd.read_csv(DF_input_base + "parsedX_3500000.csv")
df_parsedX_3500000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_3500000.csv")

In [13]:
## Replace found errors by true Country values

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32719975, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32719975, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32192726, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32192726, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32522324, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32522324, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32734609, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32734609, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32631447, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32631447, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32277441, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32277441, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32416773, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32416773, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32311274, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32311274, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32443455, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32443455, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32469226, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32469226, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32496437, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32496437, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32556328, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32556328, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32606146, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32606146, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32677927, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32677927, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32720224, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32720224, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32792225, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32792225, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32332700, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32332700, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32598532, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32598532, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32376498, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32376498, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32186248, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32186248, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32234678, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32234678, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32293439, ["Country", "Country_source"]] = fix_country_from_text("Ecuador")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32293439, ["Country", "Country_source"]] = fix_country_from_text("Ecuador")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32596391, ["Country", "Country_source"]] = fix_country_from_text("Ecuador")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32596391, ["Country", "Country_source"]] = fix_country_from_text("Ecuador")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32743726, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32743726, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32616042, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32616042, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32588349, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32588349, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32231023, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32231023, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32232692, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32232692, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32349701, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32349701, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32368321, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32368321, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32429412, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32429412, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32514625, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32514625, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32577365, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32577365, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32591964, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32591964, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32612822, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32612822, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32642067, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32642067, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32660150, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32660150, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32699600, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32699600, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32780194, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32780194, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32193734, ["Country", "Country_source"]] = fix_country_from_text("Saudi Arabia")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32193734, ["Country", "Country_source"]] = fix_country_from_text("Saudi Arabia")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32217105, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32217105, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32231055, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32231055, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32326637, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32326637, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32362086, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32362086, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32392733, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32392733, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32435917, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32435917, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32442889, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32442889, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32446127, ["Country", "Country_source"]] = fix_country_from_text("Serbia")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32446127, ["Country", "Country_source"]] = fix_country_from_text("Serbia")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32462229, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32462229, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32492520, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32492520, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32499959, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32499959, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32517362, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32517362, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32556743, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32556743, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32570010, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32570010, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32593236, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32593236, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32604802, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32604802, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32640262, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32640262, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32665581, ["Country", "Country_source"]] = fix_country_from_text("Iran")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32665581, ["Country", "Country_source"]] = fix_country_from_text("Iran")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32726945, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32726945, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32746854, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32746854, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32788923, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32788923, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32804049, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32804049, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32578139, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32578139, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32659454, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32659454, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32758457, ["Country", "Country_source"]] = fix_country_from_text("Jamaica")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32758457, ["Country", "Country_source"]] = fix_country_from_text("Jamaica")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32252576, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32252576, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32508257, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32508257, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32607742, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32607742, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32372219, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32372219, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32236611, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32236611, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32249819, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32249819, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32263468, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32263468, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32269620, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32269620, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32293507, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32293507, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32320856, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32320856, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32334322, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32334322, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32429787, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32429787, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32439835, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32439835, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32442569, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32442569, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32480172, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32480172, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32501723, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32501723, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32597562, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32597562, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32629732, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32629732, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32648567, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32648567, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32709873, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32709873, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32718270, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32718270, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32745771, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32745771, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == 32784109, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == 32784109, ["Country", "Country_source"]] = fix_country_from_text("China")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3500000_base.loc[df_parsedX_3500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3500000_match_GLOBOCAN.loc[df_parsedX_3500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [14]:
df_parsedX_3500000_base.to_csv(DF_input_base + "parsedX_3500000.csv", index = False)
df_parsedX_3500000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_3500000.csv", index = False)

del df_parsedX_3500000_base
del df_parsedX_3500000_match_GLOBOCAN

### Correcting *parsedX_3600000.csv*

In [15]:
# Read csv
df_parsedX_3600000_base = pd.read_csv(DF_input_base + "parsedX_3600000.csv")
df_parsedX_3600000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_3600000.csv")

In [16]:
## Replace found errors by true Country values

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33073332, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33073332, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33334598, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33334598, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 32840836, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 32840836, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 32860203, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 32860203, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 32917401, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 32917401, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33161422, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33161422, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33083629, ["Country", "Country_source"]] = fix_country_from_text("Ecuador")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33083629, ["Country", "Country_source"]] = fix_country_from_text("Ecuador")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33213401, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33213401, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 32853017, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 32853017, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 32948356, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 32948356, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33026351, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33026351, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33044628, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33044628, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33284505, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33284505, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33180981, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33180981, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33200027, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33200027, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33200696, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33200696, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33292387, ["Country", "Country_source"]] = fix_country_from_text("Ecuador")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33292387, ["Country", "Country_source"]] = fix_country_from_text("Ecuador")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33260045, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33260045, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33058203, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33058203, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33199152, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33199152, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 32972454, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 32972454, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 32994182, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 32994182, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33059654, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33059654, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33066747, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33066747, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33112554, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33112554, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33136258, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33136258, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33224371, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33224371, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33338879, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33338879, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33374342, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33374342, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 32812772, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 32812772, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 32842535, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 32842535, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 32878646, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 32878646, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 32992913, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 32992913, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33035134, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33035134, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33050319, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33050319, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33090259, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33090259, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33096691, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33096691, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33110006, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33110006, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33110479, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33110479, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33207760, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33207760, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33230725, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33230725, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33230804, ["Country", "Country_source"]] = fix_country_from_text("Israel")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33230804, ["Country", "Country_source"]] = fix_country_from_text("Israel")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33287305, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33287305, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33323907, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33323907, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33326760, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33326760, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33340851, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33340851, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33342103, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33342103, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33355131, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33355131, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33361833, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33361833, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33361840, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33361840, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33361841, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33361841, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33361846, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33361846, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33361851, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33361851, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33361854, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33361854, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33361856, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33361856, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33375769, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33375769, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33377057, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33377057, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33388995, ["Country", "Country_source"]] = fix_country_from_text("Cuba")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33388995, ["Country", "Country_source"]] = fix_country_from_text("Cuba")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33412409, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33412409, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33221201, ["Country", "Country_source"]] = fix_country_from_text("Peru")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33221201, ["Country", "Country_source"]] = fix_country_from_text("Peru")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33189777, ["Country", "Country_source"]] = fix_country_from_text("Bangladesh")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33189777, ["Country", "Country_source"]] = fix_country_from_text("Bangladesh")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33008752, ["Country", "Country_source"]] = fix_country_from_text("Paraguay")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33008752, ["Country", "Country_source"]] = fix_country_from_text("Paraguay")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 32819783, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 32819783, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 32825934, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 32825934, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 32831918, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 32831918, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 32836074, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 32836074, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 32840085, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 32840085, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 32871962, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 32871962, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 32952640, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 32952640, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 32983680, ["Country", "Country_source"]] = fix_country_from_text("Saudi Arabia")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 32983680, ["Country", "Country_source"]] = fix_country_from_text("Saudi Arabia")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 32994819, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 32994819, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33023539, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33023539, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33038765, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33038765, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33059953, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33059953, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33119060, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33119060, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33181461, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33181461, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33303210, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33303210, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33383046, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33383046, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33416185, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33416185, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == 33432955, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == 33432955, ["Country", "Country_source"]] = fix_country_from_text("China")

#df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3600000_base.loc[df_parsedX_3600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3600000_match_GLOBOCAN.loc[df_parsedX_3600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [17]:
df_parsedX_3600000_base.to_csv(DF_input_base + "parsedX_3600000.csv", index = False)
df_parsedX_3600000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_3600000.csv", index = False)

del df_parsedX_3600000_base
del df_parsedX_3600000_match_GLOBOCAN

### Correcting *parsedX_3700000.csv*

In [18]:
# Read csv
df_parsedX_3700000_base = pd.read_csv(DF_input_base + "parsedX_3700000.csv")
df_parsedX_3700000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_3700000.csv")

In [19]:
## Replace found errors by true Country values

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33749551, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33749551, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33848668, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33848668, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33903933, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33903933, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33616840, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33616840, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33551073, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33551073, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33463852, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33463852, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33875272, ["Country", "Country_source"]] = fix_country_from_text("South Korea")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33875272, ["Country", "Country_source"]] = fix_country_from_text("South Korea")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33969896, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33969896, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33969911, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33969911, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33999463, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33999463, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 34097764, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 34097764, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 34097765, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 34097765, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33693986, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33693986, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33474679, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33474679, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33662680, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33662680, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33685938, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33685938, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33804167, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33804167, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33880999, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33880999, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33904314, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33904314, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33948629, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33948629, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 34071401, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 34071401, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 34089500, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 34089500, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33786256, ["Country", "Country_source"]] = fix_country_from_text("Colombia")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33786256, ["Country", "Country_source"]] = fix_country_from_text("Colombia")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33509717, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33509717, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33727172, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33727172, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33863497, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33863497, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33934286, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33934286, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 34106575, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 34106575, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33515632, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33515632, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33770579, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33770579, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33900532, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33900532, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 34062227, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 34062227, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33666842, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33666842, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33745006, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33745006, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33934976, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33934976, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33794385, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33794385, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33486073, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33486073, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33502740, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33502740, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33522707, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33522707, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33623898, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33623898, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33634473, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33634473, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33662914, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33662914, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33717555, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33717555, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33794843, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33794843, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33964982, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33964982, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 34007360, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 34007360, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 34065520, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 34065520, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33503830, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33503830, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33546220, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33546220, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33549385, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33549385, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33550751, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33550751, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33559098, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33559098, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33578990, ["Country", "Country_source"]] = fix_country_from_text("Israel")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33578990, ["Country", "Country_source"]] = fix_country_from_text("Israel")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33581289, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33581289, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33778703, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33778703, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33791771, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33791771, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33812449, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33812449, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 34052775, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 34052775, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 34059243, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 34059243, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 34093104, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 34093104, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 34099850, ["Country", "Country_source"]] = fix_country_from_text("Israel")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 34099850, ["Country", "Country_source"]] = fix_country_from_text("Israel")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33458794, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33458794, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33534908, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33534908, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33559808, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33559808, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33603235, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33603235, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33912249, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33912249, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 34050359, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 34050359, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 34099896, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 34099896, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 34088814, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 34088814, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33908328, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33908328, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33539088, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33539088, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33596297, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33596297, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33618200, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33618200, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33642019, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33642019, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33871120, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33871120, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33934792, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33934792, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33967595, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33967595, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 33994055, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 33994055, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 34011786, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 34011786, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 34013366, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 34013366, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == 34022189, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == 34022189, ["Country", "Country_source"]] = fix_country_from_text("China")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3700000_base.loc[df_parsedX_3700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3700000_match_GLOBOCAN.loc[df_parsedX_3700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [20]:
df_parsedX_3700000_base.to_csv(DF_input_base + "parsedX_3700000.csv", index = False)
df_parsedX_3700000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_3700000.csv", index = False)

del df_parsedX_3700000_base
del df_parsedX_3700000_match_GLOBOCAN

### Correcting *parsedX_3800000.csv*

In [21]:
# Read csv
df_parsedX_3800000_base = pd.read_csv(DF_input_base + "parsedX_3800000.csv")
df_parsedX_3800000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_3800000.csv")

In [22]:
## Replace found errors by true Country values

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34414870, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34414870, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34374389, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34374389, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34185219, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34185219, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34324055, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34324055, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34387150, ["Country", "Country_source"]] = fix_country_from_text("Ireland")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34387150, ["Country", "Country_source"]] = fix_country_from_text("Ireland")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34618601, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34618601, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34750636, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34750636, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34224061, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34224061, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34674491, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34674491, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34277287, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34277287, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34263651, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34263651, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34550463, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34550463, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34641699, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34641699, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34571462, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34571462, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34631227, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34631227, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34162044, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34162044, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34715356, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34715356, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34116182, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34116182, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34123758, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34123758, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34260836, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34260836, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34261455, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34261455, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34285624, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34285624, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34325210, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34325210, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34391390, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34391390, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34449562, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34449562, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34496815, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34496815, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34591289, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34591289, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34123510, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34123510, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34146062, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34146062, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34149957, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34149957, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34184747, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34184747, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34197250, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34197250, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34201116, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34201116, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34218524, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34218524, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34242914, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34242914, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34245512, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34245512, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34286378, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34286378, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34300315, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34300315, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34302297, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34302297, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34313441, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34313441, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34368831, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34368831, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34402034, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34402034, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34404560, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34404560, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34441979, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34441979, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34463558, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34463558, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34473212, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34473212, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34556252, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34556252, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34572289, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34572289, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34603753, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34603753, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34638910, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34638910, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34642740, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34642740, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34652316, ["Country", "Country_source"]] = fix_country_from_text("Colombia")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34652316, ["Country", "Country_source"]] = fix_country_from_text("Colombia")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34695450, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34695450, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34702207, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34702207, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34714630, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34714630, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34744351, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34744351, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34268284, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34268284, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34437915, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34437915, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34495569, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34495569, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34600573, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34600573, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34383612, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34383612, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34645364, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34645364, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34522001, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34522001, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34302604, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34302604, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34595425, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34595425, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34254034, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34254034, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34296853, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34296853, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34314915, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34314915, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34350652, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34350652, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34426153, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34426153, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34435600, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34435600, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34534860, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34534860, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34549727, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34549727, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34622876, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34622876, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34674600, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34674600, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34714628, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34714628, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == 34729096, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == 34729096, ["Country", "Country_source"]] = fix_country_from_text("China")

#df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3800000_base.loc[df_parsedX_3800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3800000_match_GLOBOCAN.loc[df_parsedX_3800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [23]:
df_parsedX_3800000_base.to_csv(DF_input_base + "parsedX_3800000.csv", index = False)
df_parsedX_3800000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_3800000.csv", index = False)

del df_parsedX_3800000_base
del df_parsedX_3800000_match_GLOBOCAN

### Correcting *parsedX_3900000.csv*

In [24]:
# Read csv
df_parsedX_3900000_base = pd.read_csv(DF_input_base + "parsedX_3900000.csv")
df_parsedX_3900000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_3900000.csv")

In [26]:
## Replace found errors by true Country values

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 34937625, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 34937625, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 34921786, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 34921786, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 34965047, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 34965047, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35106961, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 35106961, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35301096, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 35301096, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35215276, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 35215276, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 34779383, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 34779383, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 34818028, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 34818028, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 34843006, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 34843006, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35146168, ["Country", "Country_source"]] = fix_country_from_text("Peru")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 35146168, ["Country", "Country_source"]] = fix_country_from_text("Peru")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35389573, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 35389573, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 34754837, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 34754837, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 34800790, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 34800790, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 34857768, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 34857768, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35114612, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 35114612, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35094685, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 35094685, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35186554, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 35186554, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35228941, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 35228941, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 34766229, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 34766229, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35247094, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 35247094, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 34969711, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 34969711, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 34912620, ["Country", "Country_source"]] = fix_country_from_text("Greece")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 34912620, ["Country", "Country_source"]] = fix_country_from_text("Greece")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 34789192, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 34789192, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 34972509, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 34972509, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35070303, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 35070303, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35071010, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 35071010, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35156177, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 35156177, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35196631, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 35196631, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35318539, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 35318539, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35381047, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 35381047, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 34770371, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 34770371, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 34771734, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 34771734, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 34773196, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 34773196, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 34773578, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 34773578, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 34804641, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 34804641, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 34830512, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 34830512, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 34830934, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 34830934, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 34833432, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 34833432, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 34839417, ["Country", "Country_source"]] = fix_country_from_text("Israel")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 34839417, ["Country", "Country_source"]] = fix_country_from_text("Israel")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 34846810, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 34846810, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 34857829, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 34857829, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 34873574, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 34873574, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 34945219, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 34945219, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 34965541, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 34965541, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 34974881, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 34974881, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35000753, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 35000753, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35008362, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] ==35008362, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35060018, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 35060018, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35127319, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 35127319, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35158944, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 35158944, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35159016, ["Country", "Country_source"]] = fix_country_from_text("Israel")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 35159016, ["Country", "Country_source"]] = fix_country_from_text("Israel")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35181413, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 35181413, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35246046, ["Country", "Country_source"]] = fix_country_from_text("Taiwan")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 35246046, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35265362, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 35265362, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35317277, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 35317277, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35331510, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 35331510, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35342397, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 35342397, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35369402, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 35369402, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35382184, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 35382184, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35390128, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 35390128, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35390131, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 35390131, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 34919824, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 34919824, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35260808, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 35260808, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 34945704, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 34945704, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35158182, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 35158182, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 34779435, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 34779435, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 34812806, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 34812806, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 34823109, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 34823109, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 34895419, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 34895419, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 34900449, ["Country", "Country_source"]] = fix_country_from_text("Saudi Arabia")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 34900449, ["Country", "Country_source"]] = fix_country_from_text("Saudi Arabia")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 34931824, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 34931824, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 34968048, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 34968048, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 34968834, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 34968834, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35006696, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 35006696, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35014472, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 35014472, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35021408, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 35021408, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35026926, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 35026926, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35038566, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 35038566, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35044159, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 35044159, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35121826, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 35121826, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35157958, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 35157958, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35211759, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 35211759, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35259487, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 35259487, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35295079, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 35295079, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35316767, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 35316767, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == 35333296, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == 35333296, ["Country", "Country_source"]] = fix_country_from_text("China")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_3900000_base.loc[df_parsedX_3900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_3900000_match_GLOBOCAN.loc[df_parsedX_3900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [27]:
df_parsedX_3900000_base.to_csv(DF_input_base + "parsedX_3900000.csv", index = False)
df_parsedX_3900000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_3900000.csv", index = False)

del df_parsedX_3900000_base
del df_parsedX_3900000_match_GLOBOCAN

### Correcting *parsedX_400000.csv*

In [28]:
# Read csv
df_parsedX_400000_base = pd.read_csv(DF_input_base + "parsedX_400000.csv")
df_parsedX_400000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_400000.csv")

In [29]:
## Replace found errors by true Country values

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3432692, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3432692, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3503452, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3503452, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3671025, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3671025, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3658341, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3658341, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3658342, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3658342, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3431704, ["Country", "Country_source"]] = fix_country_from_text("Portugal")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3431704, ["Country", "Country_source"]] = fix_country_from_text("Portugal")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3477471, ["Country", "Country_source"]] = fix_country_from_text("Sweden")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3477471, ["Country", "Country_source"]] = fix_country_from_text("Sweden")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3482509, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3482509, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3426259, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3426259, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3429231, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3429231, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3429980, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3429980, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3430304, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3430304, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3430739, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3430739, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3430740, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3430740, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3430742, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3430742, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3430744, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3430744, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3430745, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3430745, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3430746, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3430746, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3431163, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3431163, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3431231, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3431231, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3431232, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3431232, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3433092, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3433092, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3433823, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3433823, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3434494, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3434494, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3436205, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3436205, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3436285, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3436285, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3436505, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3436505, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3438186, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3438186, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3438189, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3438189, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3439213, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3439213, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3440231, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3440231, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3441675, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3441675, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3442157, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3442157, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3442158, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3442158, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3442739, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3442739, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3443403, ["Country", "Country_source"]] = fix_country_from_text("Georgia")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3443403, ["Country", "Country_source"]] = fix_country_from_text("Georgia")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3443404, ["Country", "Country_source"]] = fix_country_from_text("Georgia")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3443404, ["Country", "Country_source"]] = fix_country_from_text("Georgia")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3443712, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3443712, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3443823, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3443823, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3444498, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3444498, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3444499, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3444499, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3444505, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3444505, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3444516, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3444516, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3444517, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3444517, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3447041, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3447041, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3447644, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3447644, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3447857, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3447857, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3447881, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3447881, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3451018, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3451018, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3451703, ["Country", "Country_source"]] = fix_country_from_text("Madagascar")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3451703, ["Country", "Country_source"]] = fix_country_from_text("Madagascar")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3452817, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3452817, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3455648, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3455648, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3455651, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3455651, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3477664, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3477664, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3477704, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3477704, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3477806, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3477806, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3480379, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3480379, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3480843, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3480843, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3481543, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3481543, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3483932, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3483932, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3498946, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3498946, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3498948, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3498948, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3499527, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3499527, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3499529, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3499529, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3499646, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3499646, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3499769, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3499769, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3499972, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3499972, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3500549, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3500549, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3500556, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3500556, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3501207, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3501207, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3503458, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3503458, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3504060, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3504060, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3507246, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3507246, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3518477, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3518477, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3651394, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3651394, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3652913, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3652913, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3653414, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3653414, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3654904, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3654904, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3656059, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3656059, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3657619, ["Country", "Country_source"]] = fix_country_from_text("Türkiye")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3657619, ["Country", "Country_source"]] = fix_country_from_text("Türkiye")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3658101, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3658101, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3658214, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3658214, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3660223, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3660223, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3660224, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3660224, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3660522, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3660522, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3661340, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3661340, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3664332, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3664332, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3664451, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3664451, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3664502, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3664502, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3664919, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3664919, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3664919, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3664919, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3665954, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3665954, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3666558, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3666558, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3667371, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3667371, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3667858, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3667858, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3667863, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3667863, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3669305, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3669305, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3669306, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3669306, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3669307, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3669307, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3669309, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3669309, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3669311, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3669311, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3669312, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3669312, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3669314, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3669314, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3669317, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3669317, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3669318, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3669318, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3669319, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3669319, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3669320, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3669320, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3669759, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3669759, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3670524, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3670524, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3670527, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3670527, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3670528, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3670528, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3670532, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3670532, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3670533, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3670533, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3672126, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3672126, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3673325, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3673325, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3673327, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3673327, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3673742, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3673742, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3673743, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3673743, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3674635, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3674635, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3674893, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3674893, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3677058, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3677058, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3677059, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3677059, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3677062, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3677062, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3677100, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3677100, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3677319, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3677319, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3677552, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3677552, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3678791, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3678791, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3678797, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3678797, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3679421, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3679421, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3679425, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3679425, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3679556, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3679556, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3680466, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3680466, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3682192, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3682192, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3682194, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3682194, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3682196, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3682196, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3682199, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3682199, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3682711, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3682711, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3684607, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3684607, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3684612, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3684612, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3685264, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3685264, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3685839, ["Country", "Country_source"]] = fix_country_from_text("Denmark")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3685839, ["Country", "Country_source"]] = fix_country_from_text("Denmark")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3685924, ["Country", "Country_source"]] = fix_country_from_text("Sweden")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3685924, ["Country", "Country_source"]] = fix_country_from_text("Sweden")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3687090, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3687090, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3687211, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3687211, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3687233, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] ==3687233, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3688893, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3688893, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3688895, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3688895, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3693448, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3693448, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3694794, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3694794, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3694796, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3694796, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3694797, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3694797, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3694803, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3694803, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3695270, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3695270, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3695933, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3695933, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3696750, ["Country", "Country_source"]] = fix_country_from_text("Sweden")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3696750, ["Country", "Country_source"]] = fix_country_from_text("Sweden")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3411696, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3411696, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3412743, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3412743, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3415268, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3415268, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3415271, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3415271, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3415272, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3415272, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3415275, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3415275, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3417089, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3417089, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3418195, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3418195, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3419395, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3419395, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3421814, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3421814, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3422516, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3422516, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3422551, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3422551, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3423334, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3423334, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3425163, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3425163, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3425521, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3425521, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3426073, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3426073, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3426140, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3426140, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3664950, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3664950, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == 3415435, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == 3415435, ["Country", "Country_source"]] = fix_country_from_text("Russia")

#df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_400000_base.loc[df_parsedX_400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_400000_match_GLOBOCAN.loc[df_parsedX_400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [30]:
df_parsedX_400000_base.to_csv(DF_input_base + "parsedX_400000.csv", index = False)
df_parsedX_400000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_400000.csv", index = False)

del df_parsedX_400000_base
del df_parsedX_400000_match_GLOBOCAN

### Correcting *parsedX_4000000.csv*

In [31]:
# Read csv
df_parsedX_4000000_base = pd.read_csv(DF_input_base + "parsedX_4000000.csv")
df_parsedX_4000000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_4000000.csv")

In [32]:
## Replace found errors by true Country values

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35641826, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35641826, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35953110, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35953110, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35974658, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35974658, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35455464, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35455464, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35795510, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35795510, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35606546, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35606546, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35888170, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35888170, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35447567, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35447567, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35440230, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35440230, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35967130, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35967130, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35462008, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35462008, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35579140, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35579140, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35403935, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35403935, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35992599, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35992599, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35931942, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35931942, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35868124, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35868124, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35936029, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35936029, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 36029331, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 36029331, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35461381, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35461381, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35546238, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35546238, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35568860, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35568860, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35743996, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35743996, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35868288, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35868288, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35896840, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35896840, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35944311, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35944311, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35987641, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35987641, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 36052808, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 36052808, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35397692, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35397692, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35424170, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35424170, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35442592, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35442592, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35454903, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35454903, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35544375, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35544375, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35559871, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35559871, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35606604, ["Country", "Country_source"]] = fix_country_from_text("Egypt")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35606604, ["Country", "Country_source"]] = fix_country_from_text("Egypt")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35623778, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35623778, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35625921, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35625921, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35725075, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35725075, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35741039, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35741039, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35805014, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35805014, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35837299, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35837299, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35849167, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35849167, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35864130, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35864130, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35869542, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35869542, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35874908, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35874908, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35877436, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35877436, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35884467, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35884467, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35896589, ["Country", "Country_source"]] = fix_country_from_text("Israel")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35896589, ["Country", "Country_source"]] = fix_country_from_text("Israel")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35898172, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35898172, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35923859, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35923859, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35997929, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35997929, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35998623, ["Country", "Country_source"]] = fix_country_from_text("Cuba")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35998623, ["Country", "Country_source"]] = fix_country_from_text("Cuba")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 36011153, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 36011153, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35737243, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35737243, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35658211, ["Country", "Country_source"]] = fix_country_from_text("Bangladesh")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35658211, ["Country", "Country_source"]] = fix_country_from_text("Bangladesh")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35794609, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35794609, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35832429, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35832429, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35850772, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35850772, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35951087, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35951087, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35623925, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35623925, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35694190, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35694190, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35583002, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35583002, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35524835, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35524835, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35525882, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35525882, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35937719, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35937719, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35892756, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35892756, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35412588, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35412588, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35413308, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35413308, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35414400, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35414400, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35416195, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35416195, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35424141, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35424141, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35428760, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35428760, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35481621, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35481621, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35481965, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35481965, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35561631, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35561631, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35582423, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35582423, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35609735, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35609735, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35649390, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35649390, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35667253, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35667253, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35688944, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35688944, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35700603, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35700603, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35844796, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35844796, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35917454, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35917454, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35925798, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35925798, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 35943963, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 35943963, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == 36004475, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == 36004475, ["Country", "Country_source"]] = fix_country_from_text("China")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4000000_base.loc[df_parsedX_4000000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4000000_match_GLOBOCAN.loc[df_parsedX_4000000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [33]:
df_parsedX_4000000_base.to_csv(DF_input_base + "parsedX_4000000.csv", index = False)
df_parsedX_4000000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_4000000.csv", index = False)

del df_parsedX_4000000_base
del df_parsedX_4000000_match_GLOBOCAN

### Correcting *parsedX_4100000.csv*

In [34]:
# Read csv
df_parsedX_4100000_base = pd.read_csv(DF_input_base + "parsedX_4100000.csv")
df_parsedX_4100000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_4100000.csv")

In [36]:
## Replace found errors by true Country values

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36216187, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36216187, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36521078, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36521078, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36631707, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36631707, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36207082, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36207082, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36207115, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36207115, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36207122, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36207122, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36191785, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36191785, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36290665, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36290665, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36334695, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36334695, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36724554, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36724554, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36567145, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36567145, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36648521, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36648521, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36471108, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36471108, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36307558, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36307558, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36129914, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36129914, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36165084, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36165084, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36177984, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36177984, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36342647, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36342647, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36345980, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36345980, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36384685, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36384685, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36459304, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36459304, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36482484, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36482484, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36583762, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36583762, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36595982, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36595982, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36155118, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36155118, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36209930, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36209930, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36231910, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36231910, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36233525, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36233525, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36248816, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36248816, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36292985, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36292985, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36293523, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36293523, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36371129, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36371129, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36374618, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36374618, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36499101, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36499101, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36582080, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36582080, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36582081, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36582081, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36582086, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36582086, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36582092, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36582092, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36582093	, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36582093	, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36582095, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36582095, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36582097, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36582097, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36582102, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36582102, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36598557, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36598557, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36612056, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36612056, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36648868, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36648868, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36675073, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36675073, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36683777, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36683777, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36707673, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36707673, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36740642, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36740642, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36249070, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36249070, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36075865, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36075865, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36173560, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36173560, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36600261, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36600261, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36735848, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36735848, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36579755, ["Country", "Country_source"]] = fix_country_from_text("Jamaica")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36579755, ["Country", "Country_source"]] = fix_country_from_text("Jamaica")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36427594, ["Country", "Country_source"]] = fix_country_from_text("Paraguay")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36427594, ["Country", "Country_source"]] = fix_country_from_text("Paraguay")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36112113, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36112113, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36147481, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36147481, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36162009, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36162009, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36181398, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36181398, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36198664, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36198664, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36280043, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36280043, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36330800, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36330800, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36337305, ["Country", "Country_source"]] = fix_country_from_text("Saudi Arabia")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36337305, ["Country", "Country_source"]] = fix_country_from_text("Saudi Arabia")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36397897, ["Country", "Country_source"]] = fix_country_from_text("Saudi Arabia")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36397897, ["Country", "Country_source"]] = fix_country_from_text("Saudi Arabia")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36408984, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36408984, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36409066, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36409066, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36423364, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36423364, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36427655, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36427655, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36471531, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36471531, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36574309, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36574309, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36583760, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36583760, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36619215, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36619215, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36633955, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36633955, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36638855, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36638855, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36644154, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36644154, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == 36720550, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == 36720550, ["Country", "Country_source"]] = fix_country_from_text("China")

#df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4100000_base.loc[df_parsedX_4100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4100000_match_GLOBOCAN.loc[df_parsedX_4100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [37]:
df_parsedX_4100000_base.to_csv(DF_input_base + "parsedX_4100000.csv", index = False)
df_parsedX_4100000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_4100000.csv", index = False)

del df_parsedX_4100000_base
del df_parsedX_4100000_match_GLOBOCAN

### Correcting *parsedX_4200000.csv*

In [38]:
# Read csv
df_parsedX_4200000_base = pd.read_csv(DF_input_base + "parsedX_4200000.csv")
df_parsedX_4200000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_4200000.csv")

In [39]:
## Replace found errors by true Country values

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 36994202, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 36994202, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37096110, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37096110, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 36944555 , ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 36944555 , ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37053216, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37053216, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37166740, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37166740, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37316777, ["Country", "Country_source"]] = fix_country_from_text("South Korea")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37316777, ["Country", "Country_source"]] = fix_country_from_text("South Korea")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37344790, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37344790, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 36921709, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 36921709, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 36960362, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 36960362, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37014774, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37014774, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37033601, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37033601, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37089930, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37089930, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 36816957, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 36816957, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37202298, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37202298, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 36818732, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 36818732, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 36945273, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 36945273, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37168167, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37168167, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37175984, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37175984, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37214073, ["Country", "Country_source"]] = fix_country_from_text("St. Vincent and the Grenadines")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37214073, ["Country", "Country_source"]] = fix_country_from_text("St. Vincent and the Grenadines")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37035854, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37035854, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 36947346, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 36947346, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37367167, ["Country", "Country_source"]] = fix_country_from_text("Romania")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37367167, ["Country", "Country_source"]] = fix_country_from_text("Romania")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 36747228, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 36747228, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 36879276, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 36879276, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 36941666, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 36941666, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 36994539, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 36994539, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37157037, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37157037, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37170693, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37170693, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37271041, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37271041, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37302899, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37302899, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 36778405, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 36778405, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 36812415, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 36812415, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 36827840, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 36827840, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 36835201, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 36835201, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 36848399, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 36848399, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 36870038, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 36870038, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 36922819, ["Country", "Country_source"]] = fix_country_from_text("Iran")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 36922819, ["Country", "Country_source"]] = fix_country_from_text("Iran")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 36961315, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 36961315, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 36974151, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 36974151, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 36982357, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 36982357, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 36986710, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 36986710, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37000004, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37000004, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37033209, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37033209, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37072467, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37072467, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37079983, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37079983, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37149358, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37149358, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37162735, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37162735, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37201597, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37201597, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37242276, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37242276, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37247411, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37247411, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37269794, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37269794, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37362244, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37362244, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37370680, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37370680, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37373091, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37373091, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 36912068, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 36912068, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37074322, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37074322, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37154969, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37154969, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37157003, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37157003, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37232052, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37232052, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37310026, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37310026, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 36874116, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 36874116, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37148583, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37148583, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 36764141, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 36764141, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 36841114, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 36841114, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 36842766, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 36842766, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 36949576, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 36949576, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 36968934, ["Country", "Country_source"]] = fix_country_from_text("Saudi Arabia")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 36968934, ["Country", "Country_source"]] = fix_country_from_text("Saudi Arabia")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37004439, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37004439, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37100207, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37100207, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37102693, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37102693, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37153057, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37153057, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37158247, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37158247, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37166978, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37166978, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37182946, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37182946, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37186509, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37186509, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37194358, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37194358, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37390859, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37390859, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == 37395584, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == 37395584, ["Country", "Country_source"]] = fix_country_from_text("China")

#df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4200000_base.loc[df_parsedX_4200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4200000_match_GLOBOCAN.loc[df_parsedX_4200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [40]:
df_parsedX_4200000_base.to_csv(DF_input_base + "parsedX_4200000.csv", index = False)
df_parsedX_4200000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_4200000.csv", index = False)

del df_parsedX_4200000_base
del df_parsedX_4200000_match_GLOBOCAN

### Correcting *parsedX_4300000.csv*

In [41]:
# Read csv
df_parsedX_4300000_base = pd.read_csv(DF_input_base + "parsedX_4300000.csv")
df_parsedX_4300000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_4300000.csv")

In [42]:
## Replace found errors by true Country values

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37543453, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37543453, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37668792, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37668792, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37409260, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37409260, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

# Drop PMID which was found duplicate in PubMed
df_parsedX_4300000_base.drop(df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37451991].index, inplace = True)
df_parsedX_4300000_match_GLOBOCAN.drop(df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37451991].index, inplace = True)

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37478301, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37478301, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37511913, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37511913, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37702445, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37702445, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37789780, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37789780, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37869102, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37869102, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37442929, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37442929, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37898832, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37898832, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37947061, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37947061, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37453240, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37453240, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37700808, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37700808, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37933367, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37933367, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 38021583, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 38021583, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37970929, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37970929, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37929739, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37929739, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37672053, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37672053, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37433283, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37433283, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37555876, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37555876, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37846414, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37846414, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37890269, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37890269, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37899040, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37899040, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37901221, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37901221, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37906295, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37906295, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37935158, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37935158, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37958475, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37958475, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37984347, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37984347, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 38001613, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 38001613, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37425217, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37425217, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37436823, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37436823, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37444414, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37444414, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37628927, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37628927, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37664414, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37664414, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37684264, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37684264, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37685928, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37685928, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37686562, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37686562, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37760470, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37760470, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37760489, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37760489, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37760734, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37760734, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37821747, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37821747, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37835129, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37835129, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37842231, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37842231, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37852156, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37852156, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37892173, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37892173, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 38001751, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 38001751, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 38010185, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 38010185, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 38059526, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 38059526, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37490150, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37490150, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37554328, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37554328, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37789409, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37789409, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37875579, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37875579, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37888914, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37888914, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37957910, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37957910, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37429448, ["Country", "Country_source"]] = fix_country_from_text("Paraguay")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37429448, ["Country", "Country_source"]] = fix_country_from_text("Paraguay")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37648873, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37648873, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37891577, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37891577, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37894965, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37894965, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37793470, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37793470, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37480215, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37480215, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37557178, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37557178, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37590543, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37590543, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37638260, ["Country", "Country_source"]] = fix_country_from_text("Saudi Arabia")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37638260, ["Country", "Country_source"]] = fix_country_from_text("Saudi Arabia")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37660458, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37660458, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37690647, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37690647, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37802025, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37802025, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37858554, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37858554, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37902657, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37902657, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37906252, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37906252, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37975246, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37975246, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == 37996944, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == 37996944, ["Country", "Country_source"]] = fix_country_from_text("China")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4300000_base.loc[df_parsedX_4300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4300000_match_GLOBOCAN.loc[df_parsedX_4300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [43]:
df_parsedX_4300000_base.to_csv(DF_input_base + "parsedX_4300000.csv", index = False)
df_parsedX_4300000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_4300000.csv", index = False)

del df_parsedX_4300000_base
del df_parsedX_4300000_match_GLOBOCAN

### Correcting *parsedX_4400000.csv*

In [44]:
# Read csv
df_parsedX_4400000_base = pd.read_csv(DF_input_base + "parsedX_4400000.csv")
df_parsedX_4400000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_4400000.csv")

In [45]:
## Replace found errors by true Country values

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38527596, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38527596, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38316056, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38316056, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38706720, ["Country", "Country_source"]] = fix_country_from_text("Peru")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38706720, ["Country", "Country_source"]] = fix_country_from_text("Peru")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38396679, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38396679, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38273861, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38273861, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38486718, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38486718, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38471632, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38471632, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38611078, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38611078, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38174748, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38174748, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38559889, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38559889, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38711696, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38711696, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38384041, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38384041, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38498955, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38498955, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38526738, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38526738, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38663154, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38663154, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38719217, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38719217, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38071159, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38071159, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38073389, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38073389, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38075543, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38075543, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38068818, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38068818, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38069210, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38069210, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38069408, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38069408, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38070058, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38070058, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38070139, ["Country", "Country_source"]] = fix_country_from_text("Nigeria")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38070139, ["Country", "Country_source"]] = fix_country_from_text("Nigeria")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38071286, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38071286, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38073240, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38073240, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38074100, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38074100, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38075004, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38075004, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38081888, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38081888, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38083933, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38083933, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38086438, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38086438, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38086769, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38086769, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38087365, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38087365, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38091101, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38091101, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38091199, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38091199, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38092572, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38092572, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38092697, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38092697, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38093266, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38093266, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38093355, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38093355, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38094042, ["Country", "Country_source"]] = fix_country_from_text("Nigeria")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38094042, ["Country", "Country_source"]] = fix_country_from_text("Nigeria")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38096312, ["Country", "Country_source"]] = fix_country_from_text("Nigeria")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38096312, ["Country", "Country_source"]] = fix_country_from_text("Nigeria")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38097723, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38097723, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38232717, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38232717, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38266499, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38266499, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38294618, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38294618, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38502514, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38502514, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38619098, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38619098, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38104918, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38104918, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38106185, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38106185, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38136332, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38136332, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38139300, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38139300, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38155125, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38155125, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38155128, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38155128, ["Country", "Country_source"]] = fix_country_from_text("Ukraine")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38162115, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38162115, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38197800, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38197800, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38267965, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38267965, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38343990, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38343990, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38367302, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38367302, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38382150, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38382150, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38466535, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38466535, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38473228, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38473228, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38495711, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38495711, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38551406, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38551406, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38566760, ["Country", "Country_source"]] = fix_country_from_text("Rwanda")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38566760, ["Country", "Country_source"]] = fix_country_from_text("Rwanda")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38591061, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38591061, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38612619, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38612619, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38616222, ["Country", "Country_source"]] = fix_country_from_text("Iran")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38616222, ["Country", "Country_source"]] = fix_country_from_text("Iran")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38645685, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38645685, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38692686, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38692686, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38260019, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38260019, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38265492, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38265492, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38426070, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38426070, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38465222, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38465222, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38526540, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38526540, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38654605, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38654605, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38460899, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38460899, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38278449, ["Country", "Country_source"]] = fix_country_from_text("Paraguay")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38278449, ["Country", "Country_source"]] = fix_country_from_text("Paraguay")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38203664, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38203664, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38097536, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38097536, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38099343, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38099343, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38163601, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38163601, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38184937, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38184937, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38291902, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38291902, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38331001, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38331001, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38344560, ["Country", "Country_source"]] = fix_country_from_text("Saudi Arabia")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38344560, ["Country", "Country_source"]] = fix_country_from_text("Saudi Arabia")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38381576, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38381576, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38476782, ["Country", "Country_source"]] = fix_country_from_text("Saudi Arabia")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38476782, ["Country", "Country_source"]] = fix_country_from_text("Saudi Arabia")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38487141, ["Country", "Country_source"]] = fix_country_from_text("Saudi Arabia")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38487141, ["Country", "Country_source"]] = fix_country_from_text("Saudi Arabia")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38504252, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38504252, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == 38587398, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == 38587398, ["Country", "Country_source"]] = fix_country_from_text("China")

#df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4400000_base.loc[df_parsedX_4400000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4400000_match_GLOBOCAN.loc[df_parsedX_4400000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [46]:
df_parsedX_4400000_base.to_csv(DF_input_base + "parsedX_4400000.csv", index = False)
df_parsedX_4400000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_4400000.csv", index = False)

del df_parsedX_4400000_base
del df_parsedX_4400000_match_GLOBOCAN

### Correcting *parsedX_4454000.csv*

In [47]:
# Read csv
df_parsedX_4454000_base = pd.read_csv(DF_input_base + "parsedX_4454000.csv")
df_parsedX_4454000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_4454000.csv")

In [48]:
## Replace found errors by true Country values

df_parsedX_4454000_base.loc[df_parsedX_4454000_base["PMID"] == 38750928, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_4454000_match_GLOBOCAN.loc[df_parsedX_4454000_match_GLOBOCAN["PMID"] == 38750928, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_4454000_base.loc[df_parsedX_4454000_base["PMID"] == 38903136, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4454000_match_GLOBOCAN.loc[df_parsedX_4454000_match_GLOBOCAN["PMID"] == 38903136, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4454000_base.loc[df_parsedX_4454000_base["PMID"] == 38760945, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_4454000_match_GLOBOCAN.loc[df_parsedX_4454000_match_GLOBOCAN["PMID"] == 38760945, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_4454000_base.loc[df_parsedX_4454000_base["PMID"] == 38766854, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_4454000_match_GLOBOCAN.loc[df_parsedX_4454000_match_GLOBOCAN["PMID"] == 38766854, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_4454000_base.loc[df_parsedX_4454000_base["PMID"] == 38926485, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_4454000_match_GLOBOCAN.loc[df_parsedX_4454000_match_GLOBOCAN["PMID"] == 38926485, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_4454000_base.loc[df_parsedX_4454000_base["PMID"] == 39166520, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_4454000_match_GLOBOCAN.loc[df_parsedX_4454000_match_GLOBOCAN["PMID"] == 39166520, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_4454000_base.loc[df_parsedX_4454000_base["PMID"] == 39435289, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_4454000_match_GLOBOCAN.loc[df_parsedX_4454000_match_GLOBOCAN["PMID"] == 39435289, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_4454000_base.loc[df_parsedX_4454000_base["PMID"] == 38978891, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4454000_match_GLOBOCAN.loc[df_parsedX_4454000_match_GLOBOCAN["PMID"] == 38978891, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4454000_base.loc[df_parsedX_4454000_base["PMID"] == 39278673, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_4454000_match_GLOBOCAN.loc[df_parsedX_4454000_match_GLOBOCAN["PMID"] == 39278673, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_4454000_base.loc[df_parsedX_4454000_base["PMID"] == 39420726, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_4454000_match_GLOBOCAN.loc[df_parsedX_4454000_match_GLOBOCAN["PMID"] == 39420726, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_4454000_base.loc[df_parsedX_4454000_base["PMID"] == 39678637, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_4454000_match_GLOBOCAN.loc[df_parsedX_4454000_match_GLOBOCAN["PMID"] == 39678637, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_4454000_base.loc[df_parsedX_4454000_base["PMID"] == 39280576, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4454000_match_GLOBOCAN.loc[df_parsedX_4454000_match_GLOBOCAN["PMID"] == 39280576, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4454000_base.loc[df_parsedX_4454000_base["PMID"] == 40336871, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4454000_match_GLOBOCAN.loc[df_parsedX_4454000_match_GLOBOCAN["PMID"] == 40336871, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4454000_base.loc[df_parsedX_4454000_base["PMID"] == 38785200, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4454000_match_GLOBOCAN.loc[df_parsedX_4454000_match_GLOBOCAN["PMID"] == 38785200, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4454000_base.loc[df_parsedX_4454000_base["PMID"] == 38915109, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4454000_match_GLOBOCAN.loc[df_parsedX_4454000_match_GLOBOCAN["PMID"] == 38915109, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4454000_base.loc[df_parsedX_4454000_base["PMID"] == 38936093, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4454000_match_GLOBOCAN.loc[df_parsedX_4454000_match_GLOBOCAN["PMID"] == 38936093, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4454000_base.loc[df_parsedX_4454000_base["PMID"] == 38970988, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4454000_match_GLOBOCAN.loc[df_parsedX_4454000_match_GLOBOCAN["PMID"] == 38970988, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4454000_base.loc[df_parsedX_4454000_base["PMID"] == 39483425, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_4454000_match_GLOBOCAN.loc[df_parsedX_4454000_match_GLOBOCAN["PMID"] == 39483425, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_4454000_base.loc[df_parsedX_4454000_base["PMID"] == 38893249, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4454000_match_GLOBOCAN.loc[df_parsedX_4454000_match_GLOBOCAN["PMID"] == 38893249, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4454000_base.loc[df_parsedX_4454000_base["PMID"] == 38893269, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_4454000_match_GLOBOCAN.loc[df_parsedX_4454000_match_GLOBOCAN["PMID"] == 38893269, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_4454000_base.loc[df_parsedX_4454000_base["PMID"] == 38926517, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_4454000_match_GLOBOCAN.loc[df_parsedX_4454000_match_GLOBOCAN["PMID"] == 38926517, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_4454000_base.loc[df_parsedX_4454000_base["PMID"] == 38927941, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_4454000_match_GLOBOCAN.loc[df_parsedX_4454000_match_GLOBOCAN["PMID"] == 38927941, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_4454000_base.loc[df_parsedX_4454000_base["PMID"] == 39139119, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_4454000_match_GLOBOCAN.loc[df_parsedX_4454000_match_GLOBOCAN["PMID"] == 39139119, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_4454000_base.loc[df_parsedX_4454000_base["PMID"] == 40375971, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_4454000_match_GLOBOCAN.loc[df_parsedX_4454000_match_GLOBOCAN["PMID"] == 40375971, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_4454000_base.loc[df_parsedX_4454000_base["PMID"] == 39154541, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_4454000_match_GLOBOCAN.loc[df_parsedX_4454000_match_GLOBOCAN["PMID"] == 39154541, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_4454000_base.loc[df_parsedX_4454000_base["PMID"] == 38877575, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_4454000_match_GLOBOCAN.loc[df_parsedX_4454000_match_GLOBOCAN["PMID"] == 38877575, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_4454000_base.loc[df_parsedX_4454000_base["PMID"] == 38892168, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_4454000_match_GLOBOCAN.loc[df_parsedX_4454000_match_GLOBOCAN["PMID"] == 38892168, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_4454000_base.loc[df_parsedX_4454000_base["PMID"] == 39218505, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_4454000_match_GLOBOCAN.loc[df_parsedX_4454000_match_GLOBOCAN["PMID"] == 39218505, ["Country", "Country_source"]] = fix_country_from_text("China")

#df_parsedX_4454000_base.loc[df_parsedX_4454000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4454000_match_GLOBOCAN.loc[df_parsedX_4454000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4454000_base.loc[df_parsedX_4454000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4454000_match_GLOBOCAN.loc[df_parsedX_4454000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4454000_base.loc[df_parsedX_4454000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4454000_match_GLOBOCAN.loc[df_parsedX_4454000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4454000_base.loc[df_parsedX_4454000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4454000_match_GLOBOCAN.loc[df_parsedX_4454000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4454000_base.loc[df_parsedX_4454000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4454000_match_GLOBOCAN.loc[df_parsedX_4454000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4454000_base.loc[df_parsedX_4454000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4454000_match_GLOBOCAN.loc[df_parsedX_4454000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4454000_base.loc[df_parsedX_4454000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4454000_match_GLOBOCAN.loc[df_parsedX_4454000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_4454000_base.loc[df_parsedX_4454000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_4454000_match_GLOBOCAN.loc[df_parsedX_4454000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [49]:
df_parsedX_4454000_base.to_csv(DF_input_base + "parsedX_4454000.csv", index = False)
df_parsedX_4454000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_4454000.csv", index = False)

del df_parsedX_4454000_base
del df_parsedX_4454000_match_GLOBOCAN

### Correcting *parsedX_500000.csv*

In [50]:
# Read csv
df_parsedX_500000_base = pd.read_csv(DF_input_base + "parsedX_500000.csv")
df_parsedX_500000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_500000.csv")

In [51]:
## Replace found errors by true Country values

df_parsedX_500000_base.loc[df_parsedX_500000_base["PMID"] == 7716605, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_500000_match_GLOBOCAN.loc[df_parsedX_500000_match_GLOBOCAN["PMID"] == 7716605, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_500000_base.loc[df_parsedX_500000_base["PMID"] == 7629293, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_500000_match_GLOBOCAN.loc[df_parsedX_500000_match_GLOBOCAN["PMID"] == 7629293, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_500000_base.loc[df_parsedX_500000_base["PMID"] == 7694997, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_500000_match_GLOBOCAN.loc[df_parsedX_500000_match_GLOBOCAN["PMID"] == 7694997, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_500000_base.loc[df_parsedX_500000_base["PMID"] == 7724273, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_500000_match_GLOBOCAN.loc[df_parsedX_500000_match_GLOBOCAN["PMID"] == 7724273, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_500000_base.loc[df_parsedX_500000_base["PMID"] == 7742592, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_500000_match_GLOBOCAN.loc[df_parsedX_500000_match_GLOBOCAN["PMID"] == 7742592, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_500000_base.loc[df_parsedX_500000_base["PMID"] == 7805514, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_500000_match_GLOBOCAN.loc[df_parsedX_500000_match_GLOBOCAN["PMID"] == 7805514, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_500000_base.loc[df_parsedX_500000_base["PMID"] == 7842068, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_500000_match_GLOBOCAN.loc[df_parsedX_500000_match_GLOBOCAN["PMID"] == 7842068, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_500000_base.loc[df_parsedX_500000_base["PMID"] == 7899810, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_500000_match_GLOBOCAN.loc[df_parsedX_500000_match_GLOBOCAN["PMID"] == 7899810, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_500000_base.loc[df_parsedX_500000_base["PMID"] == 8116445, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_500000_match_GLOBOCAN.loc[df_parsedX_500000_match_GLOBOCAN["PMID"] == 8116445, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_500000_base.loc[df_parsedX_500000_base["PMID"] == 8119094, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_500000_match_GLOBOCAN.loc[df_parsedX_500000_match_GLOBOCAN["PMID"] == 8119094, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_500000_base.loc[df_parsedX_500000_base["PMID"] == 7565036, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_500000_match_GLOBOCAN.loc[df_parsedX_500000_match_GLOBOCAN["PMID"] == 7565036, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_500000_base.loc[df_parsedX_500000_base["PMID"] == 7619765, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_500000_match_GLOBOCAN.loc[df_parsedX_500000_match_GLOBOCAN["PMID"] == 7619765, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_500000_base.loc[df_parsedX_500000_base["PMID"] == 7619770, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_500000_match_GLOBOCAN.loc[df_parsedX_500000_match_GLOBOCAN["PMID"] == 7619770, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_500000_base.loc[df_parsedX_500000_base["PMID"] == 8086356, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_500000_match_GLOBOCAN.loc[df_parsedX_500000_match_GLOBOCAN["PMID"] == 8086356, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_500000_base.loc[df_parsedX_500000_base["PMID"] == 8094841, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_500000_match_GLOBOCAN.loc[df_parsedX_500000_match_GLOBOCAN["PMID"] == 8094841, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_500000_base.loc[df_parsedX_500000_base["PMID"] == 7493376, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_500000_match_GLOBOCAN.loc[df_parsedX_500000_match_GLOBOCAN["PMID"] == 7493376, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_500000_base.loc[df_parsedX_500000_base["PMID"] == 7892582, ["Country", "Country_source"]] = fix_country_from_text("Denmark")
df_parsedX_500000_match_GLOBOCAN.loc[df_parsedX_500000_match_GLOBOCAN["PMID"] == 7892582, ["Country", "Country_source"]] = fix_country_from_text("Denmark")

df_parsedX_500000_base.loc[df_parsedX_500000_base["PMID"] == 7976858, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_500000_match_GLOBOCAN.loc[df_parsedX_500000_match_GLOBOCAN["PMID"] == 7976858, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_500000_base.loc[df_parsedX_500000_base["PMID"] == 8015619, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_500000_match_GLOBOCAN.loc[df_parsedX_500000_match_GLOBOCAN["PMID"] == 8015619, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_500000_base.loc[df_parsedX_500000_base["PMID"] == 8032828, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_500000_match_GLOBOCAN.loc[df_parsedX_500000_match_GLOBOCAN["PMID"] == 8032828, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_500000_base.loc[df_parsedX_500000_base["PMID"] == 8044326, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_500000_match_GLOBOCAN.loc[df_parsedX_500000_match_GLOBOCAN["PMID"] == 8044326, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_500000_base.loc[df_parsedX_500000_base["PMID"] == 7625992, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_500000_match_GLOBOCAN.loc[df_parsedX_500000_match_GLOBOCAN["PMID"] == 7625992, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_500000_base.loc[df_parsedX_500000_base["PMID"] == 7636527, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_500000_match_GLOBOCAN.loc[df_parsedX_500000_match_GLOBOCAN["PMID"] == 7636527, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_500000_base.loc[df_parsedX_500000_base["PMID"] == 7697503, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_500000_match_GLOBOCAN.loc[df_parsedX_500000_match_GLOBOCAN["PMID"] == 7697503, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_500000_base.loc[df_parsedX_500000_base["PMID"] == 7783824, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_500000_match_GLOBOCAN.loc[df_parsedX_500000_match_GLOBOCAN["PMID"] == 7783824, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_500000_base.loc[df_parsedX_500000_base["PMID"] == 7928414, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_500000_match_GLOBOCAN.loc[df_parsedX_500000_match_GLOBOCAN["PMID"] == 7928414, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_500000_base.loc[df_parsedX_500000_base["PMID"] == 8133248, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_500000_match_GLOBOCAN.loc[df_parsedX_500000_match_GLOBOCAN["PMID"] == 8133248, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_500000_base.loc[df_parsedX_500000_base["PMID"] == 8037794, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_500000_match_GLOBOCAN.loc[df_parsedX_500000_match_GLOBOCAN["PMID"] == 8037794, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_500000_base.loc[df_parsedX_500000_base["PMID"] == 7629464, ["Country", "Country_source"]] = fix_country_from_text("Lithuania")
df_parsedX_500000_match_GLOBOCAN.loc[df_parsedX_500000_match_GLOBOCAN["PMID"] == 7629464, ["Country", "Country_source"]] = fix_country_from_text("Lithuania")

df_parsedX_500000_base.loc[df_parsedX_500000_base["PMID"] == 7729936, ["Country", "Country_source"]] = fix_country_from_text("Israel")
df_parsedX_500000_match_GLOBOCAN.loc[df_parsedX_500000_match_GLOBOCAN["PMID"] == 7729936, ["Country", "Country_source"]] = fix_country_from_text("Israel")

df_parsedX_500000_base.loc[df_parsedX_500000_base["PMID"] == 7761605, ["Country", "Country_source"]] = fix_country_from_text("Israel")
df_parsedX_500000_match_GLOBOCAN.loc[df_parsedX_500000_match_GLOBOCAN["PMID"] == 7761605, ["Country", "Country_source"]] = fix_country_from_text("Israel")

#df_parsedX_500000_base.loc[df_parsedX_500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_500000_match_GLOBOCAN.loc[df_parsedX_500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_500000_base.loc[df_parsedX_500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_500000_match_GLOBOCAN.loc[df_parsedX_500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_500000_base.loc[df_parsedX_500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_500000_match_GLOBOCAN.loc[df_parsedX_500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_500000_base.loc[df_parsedX_500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_500000_match_GLOBOCAN.loc[df_parsedX_500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_500000_base.loc[df_parsedX_500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_500000_match_GLOBOCAN.loc[df_parsedX_500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_500000_base.loc[df_parsedX_500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_500000_match_GLOBOCAN.loc[df_parsedX_500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_500000_base.loc[df_parsedX_500000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_500000_match_GLOBOCAN.loc[df_parsedX_500000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [52]:
df_parsedX_500000_base.to_csv(DF_input_base + "parsedX_500000.csv", index = False)
df_parsedX_500000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_500000.csv", index = False)

del df_parsedX_500000_base
del df_parsedX_500000_match_GLOBOCAN

### Correcting *parsedX_600000.csv*

In [53]:
# Read csv
df_parsedX_600000_base = pd.read_csv(DF_input_base + "parsedX_600000.csv")
df_parsedX_600000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_600000.csv")

In [54]:
## Replace found errors by true Country values

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8671916, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8671916, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8196470, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8196470, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8330586, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8330586, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8419750, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8419750, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8571181, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8571181, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8577568, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8577568, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8643973, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8643973, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8657483, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8657483, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8692551, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8692551, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8693078, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8693078, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8761744, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8761744, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8763667, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8763667, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8785560, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8785560, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8812871, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8812871, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8572636, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8572636, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8746467, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8746467, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8806698, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8806698, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8180673, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8180673, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8334678, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8334678, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8580060, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8580060, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8656198, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8656198, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8744756, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8744756, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8217793, ["Country", "Country_source"]] = fix_country_from_text("Cuba")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8217793, ["Country", "Country_source"]] = fix_country_from_text("Cuba")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8664485, ["Country", "Country_source"]] = fix_country_from_text("Cuba")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8664485, ["Country", "Country_source"]] = fix_country_from_text("Cuba")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8355888, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8355888, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8277621, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8277621, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8320912, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8320912, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8370982, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8370982, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8454257, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8454257, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8488825, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8488825, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8579123, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8579123, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8604699, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8604699, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8605013, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8605013, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8742914, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8742914, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8797307, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8797307, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8318372, ["Country", "Country_source"]] = fix_country_from_text("Sweden")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8318372, ["Country", "Country_source"]] = fix_country_from_text("Sweden")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8395101, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8395101, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8401175, ["Country", "Country_source"]] = fix_country_from_text("Sweden")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8401175, ["Country", "Country_source"]] = fix_country_from_text("Sweden")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8490657, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8490657, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8508841, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8508841, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8564351, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8564351, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8674071, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8674071, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8710034, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8710034, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8751131, ["Country", "Country_source"]] = fix_country_from_text("Sweden")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8751131, ["Country", "Country_source"]] = fix_country_from_text("Sweden")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8775747, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8775747, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8803694, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8803694, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8448081, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8448081, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8349451, ["Country", "Country_source"]] = fix_country_from_text("Israel")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8349451, ["Country", "Country_source"]] = fix_country_from_text("Israel")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8500491, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8500491, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == 8500492, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == 8500492, ["Country", "Country_source"]] = fix_country_from_text("Russia")

#df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_600000_base.loc[df_parsedX_600000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_600000_match_GLOBOCAN.loc[df_parsedX_600000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [55]:
df_parsedX_600000_base.to_csv(DF_input_base + "parsedX_600000.csv", index = False)
df_parsedX_600000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_600000.csv", index = False)

del df_parsedX_600000_base
del df_parsedX_600000_match_GLOBOCAN

### Correcting *parsedX_700000.csv*

In [56]:
# Read csv
df_parsedX_700000_base = pd.read_csv(DF_input_base + "parsedX_700000.csv")
df_parsedX_700000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_700000.csv")

In [57]:
## Replace found errors by true Country values

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 9312254, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 9312254, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 9413693, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 9413693, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 8830876, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 8830876, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 8871702, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 8871702, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 8929371, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 8929371, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 8929372, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 8929372, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 8936067, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 8936067, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 8950615, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 8950615, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 8952911, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 8952911, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 8978585, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 8978585, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 9002755, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 9002755, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 9036065, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 9036065, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 9057598, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 9057598, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 9155520, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 9155520, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 9239266, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 9239266, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 9303940, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 9303940, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 9340970, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 9340970, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 9449448, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 9449448, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 9452361, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 9452361, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 9522674, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 9522674, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 9299251, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 9299251, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 9538179, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 9538179, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 9538180, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 9538180, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 9538181, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 9538181, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 9143825, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 9143825, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 9040183, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 9040183, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 9133295, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 9133295, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 8934052, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 8934052, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 9039737, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 9039737, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 9039813, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 9039813, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 9128655, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 9128655, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 9315394, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 9315394, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 9505836, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 9505836, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 9421937, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 9421937, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 9000124, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 9000124, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 9018102, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 9018102, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 9525276, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 9525276, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 8909239, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 8909239, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 9079612, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 9079612, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 9346222, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 9346222, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 9354229, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 9354229, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 9375033, ["Country", "Country_source"]] = fix_country_from_text("Portugal")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 9375033, ["Country", "Country_source"]] = fix_country_from_text("Portugal")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 9481037, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 9481037, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 9369055, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 9369055, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 9391661, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 9391661, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 9428491, ["Country", "Country_source"]] = fix_country_from_text("Israel")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 9428491, ["Country", "Country_source"]] = fix_country_from_text("Israel")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 9456296, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 9456296, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == 9459164, ["Country", "Country_source"]] = fix_country_from_text("Israel")
df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == 9459164, ["Country", "Country_source"]] = fix_country_from_text("Israel")

#df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_700000_base.loc[df_parsedX_700000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_700000_match_GLOBOCAN.loc[df_parsedX_700000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [58]:
df_parsedX_700000_base.to_csv(DF_input_base + "parsedX_700000.csv", index = False)
df_parsedX_700000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_700000.csv", index = False)

del df_parsedX_700000_base
del df_parsedX_700000_match_GLOBOCAN

### Correcting *parsedX_800000.csv*

In [59]:
# Read csv
df_parsedX_800000_base = pd.read_csv(DF_input_base + "parsedX_800000.csv")
df_parsedX_800000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_800000.csv")

In [60]:
## Replace found errors by true Country values

df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == 9589843, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == 9589843, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == 9714688, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == 9714688, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == 9720697, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == 9720697, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == 10084537, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == 10084537, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == 10219451, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == 10219451, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == 10235026, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == 10235026, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == 10369541, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == 10369541, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == 10495567, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == 10495567, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == 9639642, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == 9639642, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == 10474450, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == 10474450, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == 9683822, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == 9683822, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == 9919765, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == 9919765, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == 9929839, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == 9929839, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == 9637203, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == 9637203, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == 9810527, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == 9810527, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == 9875642, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == 9875642, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == 10472567, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == 10472567, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == 9635577, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == 9635577, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == 9855116, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == 9855116, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == 9870502, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == 9870502, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == 9872567, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == 9872567, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == 10030736, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == 10030736, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == 10232583, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == 10232583, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == 10408901, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == 10408901, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == 10426795, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == 10426795, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == 10478762, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == 10478762, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == 9615806, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == 9615806, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == 9676935, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == 9676935, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == 9683692, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == 9683692, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == 9815589, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == 9815589, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == 10048290, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == 10048290, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == 10070433, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == 10070433, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == 9663588, ["Country", "Country_source"]] = fix_country_from_text("Israel")
df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == 9663588, ["Country", "Country_source"]] = fix_country_from_text("Israel")

df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == 9814876, ["Country", "Country_source"]] = fix_country_from_text("Israel")
df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == 9814876, ["Country", "Country_source"]] = fix_country_from_text("Israel")

df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == 9935195, ["Country", "Country_source"]] = fix_country_from_text("Israel")
df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == 9935195, ["Country", "Country_source"]] = fix_country_from_text("Israel")

df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == 9935196, ["Country", "Country_source"]] = fix_country_from_text("Israel")
df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == 9935196, ["Country", "Country_source"]] = fix_country_from_text("Israel")

#df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_800000_base.loc[df_parsedX_800000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_800000_match_GLOBOCAN.loc[df_parsedX_800000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [61]:
df_parsedX_800000_base.to_csv(DF_input_base + "parsedX_800000.csv", index = False)
df_parsedX_800000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_800000.csv", index = False)

del df_parsedX_800000_base
del df_parsedX_800000_match_GLOBOCAN

### Correcting *parsedX_900000.csv*

In [62]:
# Read csv
df_parsedX_900000_base = pd.read_csv(DF_input_base + "parsedX_900000.csv")
df_parsedX_900000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedX_900000.csv")

In [63]:
## Replace found errors by true Country values

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 10791581, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 10791581, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 10609170, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 10609170, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 10624626, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 10624626, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 10632200, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 10632200, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 10807503, ["Country", "Country_source"]] = fix_country_from_text("Ireland")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 10807503, ["Country", "Country_source"]] = fix_country_from_text("Ireland")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 11026949, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 11026949, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 11084400, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 11084400, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 11173747, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 11173747, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 11176230, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 11176230, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 11129732, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 11129732, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 10795080, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 10795080, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 11092613, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 11092613, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 10528278, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 10528278, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 10584812, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 10584812, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 11148324, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 11148324, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 10595174, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 10595174, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 10605546, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 10605546, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 10749016, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 10749016, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 10571665, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 10571665, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 10597375, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 10597375, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 10767359, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 10767359, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 10771283, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 10771283, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 10837008, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 10837008, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 10905515, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 10905515, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 10948340, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 10948340, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 10962453, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 10962453, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 10975289, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 10975289, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 11159759, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 11159759, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 11163294, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 11163294, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 11237184, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 11237184, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 10529011, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 10529011, ["Country", "Country_source"]] = fix_country_from_text("Russia10671616")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 10671616, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 10671616, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 10746981, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 10746981, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 10835727, ["Country", "Country_source"]] = fix_country_from_text("Chile")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 10835727, ["Country", "Country_source"]] = fix_country_from_text("Chile")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 10954466, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 10954466, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 10971773, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 10971773, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 11016945, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 11016945, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 11046180, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 11046180, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 11050322, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 11050322, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 11136251, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 11136251, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 11159084, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 11159084, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 11166890, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 11166890, ["Country", "Country_source"]] = fix_country_from_text("Netherlands")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 11222188, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 11222188, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 11230668, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 11230668, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 11120332, ["Country", "Country_source"]] = fix_country_from_text("Uruguay")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 11120332, ["Country", "Country_source"]] = fix_country_from_text("Uruguay")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 11154886, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 11154886, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 10785411, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 10785411, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 11093805, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 11093805, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == 10959384, ["Country", "Country_source"]] = fix_country_from_text("Israel")
df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == 10959384, ["Country", "Country_source"]] = fix_country_from_text("Israel")

#df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_900000_base.loc[df_parsedX_900000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_900000_match_GLOBOCAN.loc[df_parsedX_900000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [64]:
df_parsedX_900000_base.to_csv(DF_input_base + "parsedX_900000.csv", index = False)
df_parsedX_900000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedX_900000.csv", index = False)

del df_parsedX_900000_base
del df_parsedX_900000_match_GLOBOCAN

### Correcting *parsedXMLs_first_pred_26400.csv*

In [13]:
# Read csv
df_parsedX_first_pred_26400_base = pd.read_csv(DF_input_base + "parsedXMLs_first_pred_26400.csv")
df_parsedX_first_pred_26400_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedXMLs_first_pred_26400.csv")

In [14]:
## Replace found errors by true Country values

df_parsedX_first_pred_26400_base.loc[df_parsedX_first_pred_26400_base["PMID"] == 6203088, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_first_pred_26400_match_GLOBOCAN.loc[df_parsedX_first_pred_26400_match_GLOBOCAN["PMID"] == 6203088, ["Country", "Country_source"]] = fix_country_from_text("Italy")

#df_parsedX_first_pred_26400_base.loc[df_parsedX_first_pred_26400_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_pred_26400_match_GLOBOCAN.loc[df_parsedX_first_pred_26400_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [15]:
df_parsedX_first_pred_26400_base.to_csv(DF_input_base + "parsedXMLs_first_pred_26400.csv", index = False)
df_parsedX_first_pred_26400_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedXMLs_first_pred_26400.csv", index = False)

del df_parsedX_first_pred_26400_base
del df_parsedX_first_pred_26400_match_GLOBOCAN

### Correcting *parsedXMLs_first_upd_100000.csv*

In [6]:
# Read csv
df_parsedX_first_upd_100000_base = pd.read_csv(DF_input_base + "parsedXMLs_first_upd_100000.csv")
df_parsedX_first_upd_100000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedXMLs_first_upd_100000.csv")

In [7]:
## Replace found errors by true Country values

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 38903136, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 38903136, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 39208927, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 39208927, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 38706720, ["Country", "Country_source"]] = fix_country_from_text("Peru")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 38706720, ["Country", "Country_source"]] = fix_country_from_text("Peru")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 38978891, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 38978891, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 39255534, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 39255534, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 39034920, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 39034920, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 38760945, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 38760945, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 39278673, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 39278673, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 38573078, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 38573078, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 38926485, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 38926485, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 39123429, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 39123429, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 39134785, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 39134785, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 39282046, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 39282046, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 38750928, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 38750928, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 39057114, ["Country", "Country_source"]] = fix_country_from_text("Aruba")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 39057114, ["Country", "Country_source"]] = fix_country_from_text("Aruba")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 38316056, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 38316056, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 38950950, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 38950950, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 38865548, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 38865548, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 36870038, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 36870038, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 38367302, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 38367302, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 38893249, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 38893249, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 38893269, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 38893269, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 38927941, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 38927941, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 38959852, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 38959852, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 39096575, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 39096575, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 39136900, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 39136900, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 39160361, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 39160361, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 39216338, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 39216338, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 39254913, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 39254913, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 38700502, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 38700502, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 39117800, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 39117800, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 39141184, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 39141184, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 38877575, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 38877575, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 38460899, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 38460899, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 38996544, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 38996544, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 38549535, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 38549535, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 39227791, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 39227791, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 38484085, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 38484085, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 38518645, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 38518645, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 38643936, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 38643936, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 38728873, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 38728873, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 38761900, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 38761900, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 38768468, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 38768468, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 38884454, ["Country", "Country_source"]] = fix_country_from_text("Kuwait")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 38884454, ["Country", "Country_source"]] = fix_country_from_text("Kuwait")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 38901571, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 38901571, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 38912794, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 38912794, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 38926723, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 38926723, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 38970677, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 38970677, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 38988824, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 38988824, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 39054958, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 39054958, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 39095323, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 39095323, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 39106105, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 39106105, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 39138413, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 39138413, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 39159685, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 39159685, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 39184733, ["Country", "Country_source"]] = fix_country_from_text("Saudi Arabia")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 39184733, ["Country", "Country_source"]] = fix_country_from_text("Saudi Arabia")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 39197151, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 39197151, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 39219256, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 39219256, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 39251847, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 39251847, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 39260797, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 39260797, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 39267774, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 39267774, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == 39268290, ["Country", "Country_source"]] = fix_country_from_text("Saudi Arabia")
df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == 39268290, ["Country", "Country_source"]] = fix_country_from_text("Saudi Arabia")

#df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_100000_base.loc[df_parsedX_first_upd_100000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_100000_match_GLOBOCAN.loc[df_parsedX_first_upd_100000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [8]:
df_parsedX_first_upd_100000_base.to_csv(DF_input_base + "parsedXMLs_first_upd_100000.csv", index = False)
df_parsedX_first_upd_100000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedXMLs_first_upd_100000.csv", index = False)

del df_parsedX_first_upd_100000_base
del df_parsedX_first_upd_100000_match_GLOBOCAN

### Correcting *parsedXMLs_first_upd_200000.csv*

In [9]:
# Read csv
df_parsedX_first_upd_200000_base = pd.read_csv(DF_input_base + "parsedXMLs_first_upd_200000.csv")
df_parsedX_first_upd_200000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedXMLs_first_upd_200000.csv")

In [10]:
## Replace found errors by true Country values

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39898787, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39898787, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39524161, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39524161, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39458985, ["Country", "Country_source"]] = fix_country_from_text("Puerto Rico")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39458985, ["Country", "Country_source"]] = fix_country_from_text("Puerto Rico")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39602041, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39602041, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39772235, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39772235, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39753058, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39753058, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39898506, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39898506, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39327927, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39327927, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39529216, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39529216, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39847205, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39847205, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39856614, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39856614, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39341758, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39341758, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39427504, ["Country", "Country_source"]] = fix_country_from_text("Peru")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39427504, ["Country", "Country_source"]] = fix_country_from_text("Peru")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39435289, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39435289, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39477439, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39477439, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39747500, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39747500, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39964812, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39964812, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39330043, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39330043, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39337432, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39337432, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39362046, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39362046, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39376718, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39376718, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39456534, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39456534, ["Country", "Country_source"]] = fix_country_from_text("United States")

# Taiwan in the base, China in the match GLOBOCAN
df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39482661, ["Country", "Country_source"]] = fix_country_from_text("Taiwan")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39482661, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39495097, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39495097, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39516395, ["Country", "Country_source"]] = fix_country_from_text("Colombia")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39516395, ["Country", "Country_source"]] = fix_country_from_text("Colombia")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39586872, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39586872, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39598488, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39598488, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39659903, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39659903, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39675307, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39675307, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39694930, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39694930, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39769179, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39769179, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39769315, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39769315, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39809982, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39809982, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39931407, ["Country", "Country_source"]] = fix_country_from_text("Vietnam")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39931407, ["Country", "Country_source"]] = fix_country_from_text("Vietnam")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39958565, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39958565, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39607898, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39607898, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39448861, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39448861, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39728071, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39728071, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39901923, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39901923, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39922814, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39922814, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39961598, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39961598, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39826875, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39826875, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39426076, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39426076, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39684769, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39684769, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39727721, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39727721, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39769259, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39769259, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39771533, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39771533, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39836000, ["Country", "Country_source"]] = fix_country_from_text("New Zealand")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39836000, ["Country", "Country_source"]] = fix_country_from_text("New Zealand")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39369486, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39369486, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39411205, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39411205, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39424105, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39424105, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39446703, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39446703, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39447623, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39447623, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39465637, ["Country", "Country_source"]] = fix_country_from_text("Türkiye")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39465637, ["Country", "Country_source"]] = fix_country_from_text("Türkiye")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39495239, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39495239, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39500063, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39500063, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39550019, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39550019, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39571850, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39571850, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39616835, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39616835, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39617174, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39617174, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39696039, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39696039, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39705934, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39705934, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39790401, ["Country", "Country_source"]] = fix_country_from_text("Saudi Arabia")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39790401, ["Country", "Country_source"]] = fix_country_from_text("Saudi Arabia")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39809284, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39809284, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39826172, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39826172, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39842501, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39842501, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39842528, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39842528, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39844194, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39844194, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39845238, ["Country", "Country_source"]] = fix_country_from_text("Jordan")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39845238, ["Country", "Country_source"]] = fix_country_from_text("Jordan")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39849461, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39849461, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == 39921064, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == 39921064, ["Country", "Country_source"]] = fix_country_from_text("China")

#df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_200000_base.loc[df_parsedX_first_upd_200000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_200000_match_GLOBOCAN.loc[df_parsedX_first_upd_200000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [11]:
df_parsedX_first_upd_200000_base.to_csv(DF_input_base + "parsedXMLs_first_upd_200000.csv", index = False)
df_parsedX_first_upd_200000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedXMLs_first_upd_200000.csv", index = False)

del df_parsedX_first_upd_200000_base
del df_parsedX_first_upd_200000_match_GLOBOCAN

### Correcting *parsedXMLs_first_upd_300000.csv*

In [12]:
# Read csv
df_parsedX_first_upd_300000_base = pd.read_csv(DF_input_base + "parsedXMLs_first_upd_300000.csv")
df_parsedX_first_upd_300000_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedXMLs_first_upd_300000.csv")

In [13]:
## Replace found errors by true Country values

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40036840, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40036840, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40220908, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40220908, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40159468, ["Country", "Country_source"]] = fix_country_from_text("Ecuador")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40159468, ["Country", "Country_source"]] = fix_country_from_text("Ecuador")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40209461, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40209461, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40225770, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40225770, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40270185, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40270185, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40481983, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40481983, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40198273, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40198273, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40625142, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40625142, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40639449, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40639449, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40544104, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40544104, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40155249, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40155249, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40167786, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40167786, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40182053, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40182053, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40304827, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40304827, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40591208, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40591208, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40370481, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40370481, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40385535, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40385535, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40479529, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40479529, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40002220, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40002220, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40076625, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40076625, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40121148, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40121148, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40141790, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40141790, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40163700, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40163700, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40205400, ["Country", "Country_source"]] = fix_country_from_text("Iran")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40205400, ["Country", "Country_source"]] = fix_country_from_text("Iran")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40218878, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40218878, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40357025, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40357025, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40403699, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40403699, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40430856, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40430856, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40443068, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40443068, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40529157, ["Country", "Country_source"]] = fix_country_from_text("Puerto Rico")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40529157, ["Country", "Country_source"]] = fix_country_from_text("Puerto Rico")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40537682, ["Country", "Country_source"]] = fix_country_from_text("Egypt")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40537682, ["Country", "Country_source"]] = fix_country_from_text("Egypt")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40561495, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40561495, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40600509, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40600509, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40603713, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40603713, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40629205, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40629205, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40647492, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40647492, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40651100, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40651100, ["Country", "Country_source"]] = fix_country_from_text("United States")

# Taiwan in base, China in match GLOBOCAN
df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40637905, ["Country", "Country_source"]] = fix_country_from_text("Taiwan")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40637905, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40266025, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40266025, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40275878, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40275878, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40412491, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40412491, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40156762, ["Country", "Country_source"]] = fix_country_from_text("Paraguay")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40156762, ["Country", "Country_source"]] = fix_country_from_text("Paraguay")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40157548, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40157548, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40588481, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40588481, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40601979, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40601979, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40020497, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40020497, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40073885, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40073885, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40082896, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40082896, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40088740, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40088740, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40227950, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40227950, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40249975, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40249975, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40258522, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40258522, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40344782, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40344782, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40398074, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40398074, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40479595, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40479595, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40523887, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40523887, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40611740, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40611740, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40627961, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40627961, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == 40633742, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == 40633742, ["Country", "Country_source"]] = fix_country_from_text("China")

#df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_300000_base.loc[df_parsedX_first_upd_300000_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_300000_match_GLOBOCAN.loc[df_parsedX_first_upd_300000_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [14]:
df_parsedX_first_upd_300000_base.to_csv(DF_input_base + "parsedXMLs_first_upd_300000.csv", index = False)
df_parsedX_first_upd_300000_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedXMLs_first_upd_300000.csv", index = False)

del df_parsedX_first_upd_300000_base
del df_parsedX_first_upd_300000_match_GLOBOCAN

### Correcting *parsedXMLs_first_upd_340800.csv*

In [15]:
# Read csv
df_parsedX_first_upd_340800_base = pd.read_csv(DF_input_base + "parsedXMLs_first_upd_340800.csv")
df_parsedX_first_upd_340800_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedXMLs_first_upd_340800.csv")

In [16]:
## Replace found errors by true Country values

df_parsedX_first_upd_340800_base.loc[df_parsedX_first_upd_340800_base["PMID"] == 40828257, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_first_upd_340800_match_GLOBOCAN.loc[df_parsedX_first_upd_340800_match_GLOBOCAN["PMID"] == 40828257, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_first_upd_340800_base.loc[df_parsedX_first_upd_340800_base["PMID"] == 40866457, ["Country", "Country_source"]] = fix_country_from_text("Peru")
df_parsedX_first_upd_340800_match_GLOBOCAN.loc[df_parsedX_first_upd_340800_match_GLOBOCAN["PMID"] == 40866457, ["Country", "Country_source"]] = fix_country_from_text("Peru")

df_parsedX_first_upd_340800_base.loc[df_parsedX_first_upd_340800_base["PMID"] == 40747175, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_first_upd_340800_match_GLOBOCAN.loc[df_parsedX_first_upd_340800_match_GLOBOCAN["PMID"] == 40747175, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_first_upd_340800_base.loc[df_parsedX_first_upd_340800_base["PMID"] == 40771078, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_first_upd_340800_match_GLOBOCAN.loc[df_parsedX_first_upd_340800_match_GLOBOCAN["PMID"] == 40771078, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_first_upd_340800_base.loc[df_parsedX_first_upd_340800_base["PMID"] == 40755327, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_first_upd_340800_match_GLOBOCAN.loc[df_parsedX_first_upd_340800_match_GLOBOCAN["PMID"] == 40755327, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_first_upd_340800_base.loc[df_parsedX_first_upd_340800_base["PMID"] == 40826981, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_parsedX_first_upd_340800_match_GLOBOCAN.loc[df_parsedX_first_upd_340800_match_GLOBOCAN["PMID"] == 40826981, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_parsedX_first_upd_340800_base.loc[df_parsedX_first_upd_340800_base["PMID"] == 40833671, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_first_upd_340800_match_GLOBOCAN.loc[df_parsedX_first_upd_340800_match_GLOBOCAN["PMID"] == 40833671, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_first_upd_340800_base.loc[df_parsedX_first_upd_340800_base["PMID"] == 40949483, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_first_upd_340800_match_GLOBOCAN.loc[df_parsedX_first_upd_340800_match_GLOBOCAN["PMID"] == 40949483, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_first_upd_340800_base.loc[df_parsedX_first_upd_340800_base["PMID"] == 40717186, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")
df_parsedX_first_upd_340800_match_GLOBOCAN.loc[df_parsedX_first_upd_340800_match_GLOBOCAN["PMID"] == 40717186, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")

df_parsedX_first_upd_340800_base.loc[df_parsedX_first_upd_340800_base["PMID"] == 40764035, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_first_upd_340800_match_GLOBOCAN.loc[df_parsedX_first_upd_340800_match_GLOBOCAN["PMID"] == 40764035, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_first_upd_340800_base.loc[df_parsedX_first_upd_340800_base["PMID"] == 40789232, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_first_upd_340800_match_GLOBOCAN.loc[df_parsedX_first_upd_340800_match_GLOBOCAN["PMID"] == 40789232, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_first_upd_340800_base.loc[df_parsedX_first_upd_340800_base["PMID"] == 40862778, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_first_upd_340800_match_GLOBOCAN.loc[df_parsedX_first_upd_340800_match_GLOBOCAN["PMID"] == 40862778, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_first_upd_340800_base.loc[df_parsedX_first_upd_340800_base["PMID"] == 40867554, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_first_upd_340800_match_GLOBOCAN.loc[df_parsedX_first_upd_340800_match_GLOBOCAN["PMID"] == 40867554, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_first_upd_340800_base.loc[df_parsedX_first_upd_340800_base["PMID"] == 40940186, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_first_upd_340800_match_GLOBOCAN.loc[df_parsedX_first_upd_340800_match_GLOBOCAN["PMID"] == 40940186, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_first_upd_340800_base.loc[df_parsedX_first_upd_340800_base["PMID"] == 40834990, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_first_upd_340800_match_GLOBOCAN.loc[df_parsedX_first_upd_340800_match_GLOBOCAN["PMID"] == 40834990, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_first_upd_340800_base.loc[df_parsedX_first_upd_340800_base["PMID"] == 40739323, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_first_upd_340800_match_GLOBOCAN.loc[df_parsedX_first_upd_340800_match_GLOBOCAN["PMID"] == 40739323, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_first_upd_340800_base.loc[df_parsedX_first_upd_340800_base["PMID"] == 40805143, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_first_upd_340800_match_GLOBOCAN.loc[df_parsedX_first_upd_340800_match_GLOBOCAN["PMID"] == 40805143, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_first_upd_340800_base.loc[df_parsedX_first_upd_340800_base["PMID"] == 40680910, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_340800_match_GLOBOCAN.loc[df_parsedX_first_upd_340800_match_GLOBOCAN["PMID"] == 40680910, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_340800_base.loc[df_parsedX_first_upd_340800_base["PMID"] == 40729503, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_340800_match_GLOBOCAN.loc[df_parsedX_first_upd_340800_match_GLOBOCAN["PMID"] == 40729503, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_first_upd_340800_base.loc[df_parsedX_first_upd_340800_base["PMID"] == 40845109, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_first_upd_340800_match_GLOBOCAN.loc[df_parsedX_first_upd_340800_match_GLOBOCAN["PMID"] == 40845109, ["Country", "Country_source"]] = fix_country_from_text("China")

#df_parsedX_first_upd_340800_base.loc[df_parsedX_first_upd_340800_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_340800_match_GLOBOCAN.loc[df_parsedX_first_upd_340800_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_340800_base.loc[df_parsedX_first_upd_340800_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_340800_match_GLOBOCAN.loc[df_parsedX_first_upd_340800_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_340800_base.loc[df_parsedX_first_upd_340800_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_340800_match_GLOBOCAN.loc[df_parsedX_first_upd_340800_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_340800_base.loc[df_parsedX_first_upd_340800_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_340800_match_GLOBOCAN.loc[df_parsedX_first_upd_340800_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_340800_base.loc[df_parsedX_first_upd_340800_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_340800_match_GLOBOCAN.loc[df_parsedX_first_upd_340800_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_340800_base.loc[df_parsedX_first_upd_340800_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_340800_match_GLOBOCAN.loc[df_parsedX_first_upd_340800_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_340800_base.loc[df_parsedX_first_upd_340800_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_340800_match_GLOBOCAN.loc[df_parsedX_first_upd_340800_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_first_upd_340800_base.loc[df_parsedX_first_upd_340800_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_first_upd_340800_match_GLOBOCAN.loc[df_parsedX_first_upd_340800_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [17]:
df_parsedX_first_upd_340800_base.to_csv(DF_input_base + "parsedXMLs_first_upd_340800.csv", index = False)
df_parsedX_first_upd_340800_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedXMLs_first_upd_340800.csv", index = False)

del df_parsedX_first_upd_340800_base
del df_parsedX_first_upd_340800_match_GLOBOCAN

### Correcting *parsedXMLs_update_2025_09_33200.csv*

In [18]:
# Read csv
df_parsedX_update_2025_09_33200_base = pd.read_csv(DF_input_base + "parsedXMLs_update_2025_09_33200.csv")
df_parsedX_update_2025_09_33200_match_GLOBOCAN = pd.read_csv(DF_input_match_GLOBOCAN + "parsedXMLs_update_2025_09_33200.csv")

In [19]:
## Replace found errors by true Country values

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40001312, ["Country", "Country_source"]] = fix_country_from_text("Iran")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40001312, ["Country", "Country_source"]] = fix_country_from_text("Iran")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40080802, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40080802, ["Country", "Country_source"]] = fix_country_from_text("France")

# Taiwan for base, China for match_GLOBOCAN
df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40261832, ["Country", "Country_source"]] = fix_country_from_text("Taiwan")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40261832, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40752979, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40752979, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40880110, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40880110, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40892064, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40892064, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40931203, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40931203, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40931204, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40931204, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40940552, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40940552, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40956239, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40956239, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40974810, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40974810, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40991256, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40991256, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 39498745, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 39498745, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 39749810, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 39749810, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 39804070, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 39804070, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40266761, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40266761, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40577104, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40577104, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40641447, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40641447, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40758676, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40758676, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40898904, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40898904, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40936470, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40936470, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40950048, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40950048, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41015251, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41015251, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41032290, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41032290, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41037003, ["Country", "Country_source"]] = fix_country_from_text("Malaysia")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41037003, ["Country", "Country_source"]] = fix_country_from_text("Malaysia")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41041573, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41041573, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41049945, ["Country", "Country_source"]] = fix_country_from_text("Pakistan")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41049945, ["Country", "Country_source"]] = fix_country_from_text("Pakistan")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41052052, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41052052, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 39042937, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 39042937, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40571778, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40571778, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40694988, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40694988, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40779734, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40779734, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40879037, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40879037, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40890503, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40890503, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40959233, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40959233, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40967533, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40967533, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40974656, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40974656, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41004210, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41004210, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41022563, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41022563, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41041581, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41041581, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41047774, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41047774, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 28809522, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 28809522, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 32119428, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 32119428, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 32644700, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 32644700, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 37843521, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 37843521, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 38319227, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 38319227, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 39621192, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 39621192, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 39689303, ["Country", "Country_source"]] = fix_country_from_text("Egypt")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 39689303, ["Country", "Country_source"]] = fix_country_from_text("Egypt")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 39760856, ["Country", "Country_source"]] = fix_country_from_text("Hong Kong")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 39760856, ["Country", "Country_source"]] = fix_country_from_text("Hong Kong")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 39804360, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 39804360, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 39869100, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 39869100, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 39999214, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 39999214, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40029825, ["Country", "Country_source"]] = fix_country_from_text("Iran")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40029825, ["Country", "Country_source"]] = fix_country_from_text("Iran")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40045501, ["Country", "Country_source"]] = fix_country_from_text("Iran")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40045501, ["Country", "Country_source"]] = fix_country_from_text("Iran")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40062818, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40062818, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40065438, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40065438, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40099668, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40099668, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40100118, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40100118, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40107179, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40107179, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40178533, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40178533, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40178869, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40178869, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40209234, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40209234, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40215620, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40215620, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")

# Taiwan in base, China in match_GLOBOCAN
df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40232818, ["Country", "Country_source"]] = fix_country_from_text("Taiwan")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40232818, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40263762, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40263762, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40263765, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40263765, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40314065, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40314065, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40327712, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40327712, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40359004, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40359004, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40392199, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40392199, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40392226, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40392226, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40439484, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40439484, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40446351, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40446351, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40460390, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40460390, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40465233, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40465233, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40465327, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40465327, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40478551, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40478551, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40482312, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40482312, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40512603, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40512603, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40513504, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40513504, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40513655, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40513655, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40525287, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40525287, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40526432, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40526432, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40540799, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40540799, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40540805, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40540805, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40542604, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40542604, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40577846, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40577846, ["Country", "Country_source"]] = fix_country_from_text("United States")

# Taiwan in base, China in match_GLOBOCAN
df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40577849, ["Country", "Country_source"]] = fix_country_from_text("Taiwan")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40577849, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40583566, ["Country", "Country_source"]] = fix_country_from_text("Türkiye")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40583566, ["Country", "Country_source"]] = fix_country_from_text("Türkiye")

# Taiwan in base, China in match_GLOBOCAN
df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40583585, ["Country", "Country_source"]] = fix_country_from_text("Taiwan")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40583585, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40583589, ["Country", "Country_source"]] = fix_country_from_text("Malaysia")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40583589, ["Country", "Country_source"]] = fix_country_from_text("Malaysia")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40583782, ["Country", "Country_source"]] = fix_country_from_text("Türkiye")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40583782, ["Country", "Country_source"]] = fix_country_from_text("Türkiye")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40590121, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40590121, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40608411, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40608411, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40613528, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40613528, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40614277, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40614277, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40621819, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40621819, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40626510, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40626510, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40638249, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40638249, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40640003, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40640003, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40641327, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40641327, ["Country", "Country_source"]] = fix_country_from_text("Japan")

# Taiwan in base, China in match_GLOBOCAN
df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40644581, ["Country", "Country_source"]] = fix_country_from_text("Taiwan")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40644581, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40644718, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40644718, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40658452, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40658452, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40660840, ["Country", "Country_source"]] = fix_country_from_text("Türkiye")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40660840, ["Country", "Country_source"]] = fix_country_from_text("Türkiye")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40663408, ["Country", "Country_source"]] = fix_country_from_text("South Korea")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40663408, ["Country", "Country_source"]] = fix_country_from_text("South Korea")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40663423, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40663423, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40671967, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40671967, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40673685, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40673685, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40674637, ["Country", "Country_source"]] = fix_country_from_text("Romania")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40674637, ["Country", "Country_source"]] = fix_country_from_text("Romania")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40692431, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40692431, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40693581, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40693581, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40698993, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40698993, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40705472, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40705472, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40726445, ["Country", "Country_source"]] = fix_country_from_text("Egypt")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40726445, ["Country", "Country_source"]] = fix_country_from_text("Egypt")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40728885, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40728885, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40729165, ["Country", "Country_source"]] = fix_country_from_text("Türkiye")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40729165, ["Country", "Country_source"]] = fix_country_from_text("Türkiye")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40734613, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40734613, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40745092, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40745092, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40753844, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40753844, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40765429, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40765429, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40775057, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40775057, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40781198, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40781198, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40789013, ["Country", "Country_source"]] = fix_country_from_text("Romania")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40789013, ["Country", "Country_source"]] = fix_country_from_text("Romania")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40810105, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40810105, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40811032, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40811032, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40826229, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40826229, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40832782, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40832782, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40841588, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40841588, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40865066, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40865066, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40891038, ["Country", "Country_source"]] = fix_country_from_text("Türkiye")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40891038, ["Country", "Country_source"]] = fix_country_from_text("Türkiye")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40892381, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40892381, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40898895, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40898895, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40898901, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40898901, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40899754, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40899754, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40899840, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40899840, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40903306, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40903306, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40904487, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40904487, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40905435, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40905435, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40906500, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40906500, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40907246, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40907246, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40907468, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40907468, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40911811, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40911811, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40916123, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40916123, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40916657, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40916657, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40918085, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40918085, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40919398, ["Country", "Country_source"]] = fix_country_from_text("Kyrgyzstan")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40919398, ["Country", "Country_source"]] = fix_country_from_text("Kyrgyzstan")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40919993, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40919993, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40920591, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40920591, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40923319, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40923319, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40923321, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40923321, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40924542, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40924542, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40925663, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40925663, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40926033, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40926033, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40926097, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40926097, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40929788, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40929788, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40930144, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40930144, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40930154, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40930154, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40931187, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40931187, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40932132, ["Country", "Country_source"]] = fix_country_from_text("Colombia")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40932132, ["Country", "Country_source"]] = fix_country_from_text("Colombia")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40934118, ["Country", "Country_source"]] = fix_country_from_text("Australia")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40934118, ["Country", "Country_source"]] = fix_country_from_text("Australia")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40936196, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40936196, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40940070, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40940070, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40940072, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40940072, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40942123, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40942123, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40943943, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40943943, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40947536, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40947536, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40948080, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40948080, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40949948, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40949948, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40952996, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40952996, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40953933, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40953933, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40955667, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40955667, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40955669, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40955669, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40957654, ["Country", "Country_source"]] = fix_country_from_text("France")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40957654, ["Country", "Country_source"]] = fix_country_from_text("France")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40959233, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40959233, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40960995, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40960995, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40963299, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40963299, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40970538, ["Country", "Country_source"]] = fix_country_from_text("Singapore")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40970538, ["Country", "Country_source"]] = fix_country_from_text("Singapore")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40973431, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40973431, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40981478, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40981478, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40983324, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40983324, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40983469, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40983469, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40983667, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40983667, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40985193, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40985193, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40986763, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40986763, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40986764, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40986764, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40986765, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40986765, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40986766, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40986766, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40986768, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40986768, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40986769, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40986769, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40986770, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40986770, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40986771, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40986771, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40986772, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40986772, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40986777, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40986777, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40986779, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40986779, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40986797, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40986797, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40986826, ["Country", "Country_source"]] = fix_country_from_text("Colombia")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40986826, ["Country", "Country_source"]] = fix_country_from_text("Colombia")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40996916, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40996916, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40996962, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40996962, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40998458, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40998458, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41000253, ["Country", "Country_source"]] = fix_country_from_text("Pakistan")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41000253, ["Country", "Country_source"]] = fix_country_from_text("Pakistan")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41000431, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41000431, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41002973, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41002973, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41006703, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41006703, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41006807, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41006807, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41014610, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41014610, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41014611, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41014611, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41014612, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41014612, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41014614, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41014614, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41014619, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41014619, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41017456, ["Country", "Country_source"]] = fix_country_from_text("Iran")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41017456, ["Country", "Country_source"]] = fix_country_from_text("Iran")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41017961, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41017961, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41022563, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41022563, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41025761, ["Country", "Country_source"]] = fix_country_from_text("Denmark")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41025761, ["Country", "Country_source"]] = fix_country_from_text("Denmark")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41027312, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41027312, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41029190, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41029190, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41029960, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41029960, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41029962, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41029962, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41029963, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41029963, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41031883, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41031883, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41031891, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41031891, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41032430, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41032430, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41032432, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41032432, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41034495, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41034495, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41041468, ["Country", "Country_source"]] = fix_country_from_text("Grenada")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41041468, ["Country", "Country_source"]] = fix_country_from_text("Grenada")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41041538, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41041538, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41047774, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41047774, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41049266, ["Country", "Country_source"]] = fix_country_from_text("Saudi Arabia")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41049266, ["Country", "Country_source"]] = fix_country_from_text("Saudi Arabia")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41049886, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41049886, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41051996, ["Country", "Country_source"]] = fix_country_from_text("China")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41051996, ["Country", "Country_source"]] = fix_country_from_text("China")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41058838, ["Country", "Country_source"]] = fix_country_from_text("Vietnam")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41058838, ["Country", "Country_source"]] = fix_country_from_text("Vietnam")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41067275, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41067275, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41067277, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41067277, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41067855, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41067855, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41072036, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41072036, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41073782, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41073782, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40789232, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40789232, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40986117, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40986117, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40834990, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40834990, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40157548, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40157548, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41035612, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41035612, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41035646, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41035646, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40890130, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40890130, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40994735, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40994735, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40987094, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40987094, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41022102, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41022102, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 39602041, ["Country", "Country_source"]] = fix_country_from_text("India")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 39602041, ["Country", "Country_source"]] = fix_country_from_text("India")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 40639449, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 40639449, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == 41016940, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == 41016940, ["Country", "Country_source"]] = fix_country_from_text("Germany")

#df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

#df_parsedX_update_2025_09_33200_base.loc[df_parsedX_update_2025_09_33200_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
#df_parsedX_update_2025_09_33200_match_GLOBOCAN.loc[df_parsedX_update_2025_09_33200_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

In [20]:
df_parsedX_update_2025_09_33200_base.to_csv(DF_input_base + "parsedXMLs_update_2025_09_33200.csv", index = False)
df_parsedX_update_2025_09_33200_match_GLOBOCAN.to_csv(DF_input_match_GLOBOCAN + "parsedXMLs_update_2025_09_33200.csv", index = False)

del df_parsedX_update_2025_09_33200_base
del df_parsedX_update_2025_09_33200_match_GLOBOCAN

# Fixing articles update

### Imports

In [66]:
DF_input_update_base = input().strip()

 C:\Users\svalb\OneDrive\Escritorio\Data_40_years_cancer_studies\temp_isna_notna_base\


In [67]:
DF_input_update_match_GLOBOCAN = input().strip()

 C:\Users\svalb\OneDrive\Escritorio\Data_40_years_cancer_studies\temp_isna_notna_match_GLOBOCAN\


In [68]:
# Import list of csvs to parse
list_csvs_update_base = []

for file in os.listdir(DF_input_update_base):
    if file[-4:] == ".csv":
        list_csvs_update_base.append(file)

n_csvs_update_base = len(list_csvs_update_base)

In [69]:
# Import list of csvs to parse
list_csvs_update_match_GLOBOCAN = []

for file in os.listdir(DF_input_update_match_GLOBOCAN):
    if file[-4:] == ".csv":
        list_csvs_update_match_GLOBOCAN.append(file)

n_csvs_update_match_GLOBOCAN = len(list_csvs_update_match_GLOBOCAN)

### Global corrections

In [10]:
# Global correction: Replace "Pitcairn" by "Italy" in Country. All assignments to Pitcairn correspond to italian research centers
# Base dataset
print("--Fixing base dataset")

for csv in list_csvs_update_base:
    print(f"Parsing csv: {csv} ({list_csvs_update_base.index(csv) + 1}/{n_csvs_update_base})")
    df = pd.read_csv(DF_input_update_base + csv)
    df.loc[df["Country"] == "Pitcairn", "Country"] = "Italy"
    df.to_csv(DF_input_update_base + csv, index = False)
    del df

print("--Fixing done for base dataset")

print("--Fixing match GLobocan dataset")

# Match Globocan dataset
for csv in list_csvs_update_match_GLOBOCAN:
    print(f"Parsing csv: {csv} ({list_csvs_update_match_GLOBOCAN.index(csv) + 1}/{n_csvs_update_match_GLOBOCAN})")
    df = pd.read_csv(DF_input_update_match_GLOBOCAN + csv)
    df.loc[df["Country"] == "Pitcairn", "Country"] = "Italy"
    df.to_csv(DF_input_update_match_GLOBOCAN + csv, index = False)
    del df

print("--Fixing done for match GLobocan dataset")

--Fixing base dataset
Parsing csv: temp_isna_notna.csv (1/1)
--Fixing done for base dataset
--Fixing match GLobocan dataset
Parsing csv: temp_isna_notna.csv (1/1)
--Fixing done for match GLobocan dataset


In [11]:
## Fix wrong Country assignment in Most_common_list_countries: China instead of Japan in records with Uchinada
fixSystematicMost_common_list_countries("base", DF_input_update_base, list_csvs_update_base, "China", "Uchinada", "Japan")
fixSystematicMost_common_list_countries("match GLobocan", DF_input_update_match_GLOBOCAN, list_csvs_update_match_GLOBOCAN, "China", "Uchinada", "Japan")

--Fixing base dataset
Parsing csv: temp_isna_notna.csv (1/1)
--Fixing done for base dataset
--Fixing match GLobocan dataset
Parsing csv: temp_isna_notna.csv (1/1)
--Fixing done for match GLobocan dataset


In [12]:
## Fix wrong Country assignment in Most_common_list_countries: Peru instead of Italy in records with Perugia
fixSystematicMost_common_list_countries("base", DF_input_update_base, list_csvs_update_base, "Peru", "Perugia", "Italy")
fixSystematicMost_common_list_countries("match GLobocan", DF_input_update_match_GLOBOCAN, list_csvs_update_match_GLOBOCAN, "Peru", "Perugia", "Italy")

--Fixing base dataset
Parsing csv: temp_isna_notna.csv (1/1)
--Fixing done for base dataset
--Fixing match GLobocan dataset
Parsing csv: temp_isna_notna.csv (1/1)
--Fixing done for match GLobocan dataset


In [13]:
## Fix wrong Country assignment in Most_common_list_countries: United Kingdom instead of China in records with Ningbo
fixSystematicMost_common_list_countries("base", DF_input_update_base, list_csvs_update_base, "United Kingdom", "Ningbo", "China")
fixSystematicMost_common_list_countries("match GLobocan", DF_input_update_match_GLOBOCAN, list_csvs_update_match_GLOBOCAN, "United Kingdom", "Ningbo", "China")

--Fixing base dataset
Parsing csv: temp_isna_notna.csv (1/1)
--Fixing done for base dataset
--Fixing match GLobocan dataset
Parsing csv: temp_isna_notna.csv (1/1)
--Fixing done for match GLobocan dataset


In [14]:
## Fix wrong Country assignment in Most_common_list_countries: United Kingdom instead of China in records with Bengbu
fixSystematicMost_common_list_countries("base", DF_input_update_base, list_csvs_update_base, "United Kingdom", "Bengbu", "China")
fixSystematicMost_common_list_countries("match GLobocan", DF_input_update_match_GLOBOCAN, list_csvs_update_match_GLOBOCAN, "United Kingdom", "Bengbu", "China")

--Fixing base dataset
Parsing csv: temp_isna_notna.csv (1/1)
--Fixing done for base dataset
--Fixing match GLobocan dataset
Parsing csv: temp_isna_notna.csv (1/1)
--Fixing done for match GLobocan dataset


In [15]:
## Fix wrong Country assignment in Most_common_list_countries: Mongolia instead of China in records with Inner Mongolia
fixSystematicMost_common_list_countries("base", DF_input_update_base, list_csvs_update_base, "Mongolia", "Inner Mongolia", "China")
fixSystematicMost_common_list_countries("match GLobocan", DF_input_update_match_GLOBOCAN, list_csvs_update_match_GLOBOCAN, "Mongolia", "Inner Mongolia", "China")

--Fixing base dataset
Parsing csv: temp_isna_notna.csv (1/1)
--Fixing done for base dataset
--Fixing match GLobocan dataset
Parsing csv: temp_isna_notna.csv (1/1)
--Fixing done for match GLobocan dataset


In [16]:
## Fix wrong Country assignment in Most_common_list_countries: Italy instead of Switzerland in records with Svizzera Italiana
fixSystematicMost_common_list_countries("base", DF_input_update_base, list_csvs_update_base, "Italy", "Svizzera Italiana", "Switzerland")
fixSystematicMost_common_list_countries("match GLobocan", DF_input_update_match_GLOBOCAN, list_csvs_update_match_GLOBOCAN, "Italy", "Svizzera Italiana", "Switzerland")

--Fixing base dataset
Parsing csv: temp_isna_notna.csv (1/1)
--Fixing done for base dataset
--Fixing match GLobocan dataset
Parsing csv: temp_isna_notna.csv (1/1)
--Fixing done for match GLobocan dataset


#### Full match

In [17]:
## Fix wrong Country assignment in Most_common_list_countries: Peru instead of Brazil in records with PE

# Important: SUBSTRINGS not taken into consideration, corrected only if el["text"] == "PE"
# Important: some records should be assinged to other countries. These are done manually below

fixSystematicMost_common_list_countries_FULL_MATCH("base", DF_input_update_base, list_csvs_update_base, "Peru", "PE", "Brazil")
fixSystematicMost_common_list_countries_FULL_MATCH("match GLobocan", DF_input_update_match_GLOBOCAN, list_csvs_update_match_GLOBOCAN, "Peru", "PE", "Brazil")

--Fixing base dataset
Parsing csv: temp_isna_notna.csv (1/1)
--Fixing done for base dataset
--Fixing match GLobocan dataset
Parsing csv: temp_isna_notna.csv (1/1)
--Fixing done for match GLobocan dataset


In [18]:
## Fix wrong Country assignment in Most_common_list_countries: United Kingdom instead of Sweden in records with Sweden

# Important: SUBSTRINGS not taken into consideration, corrected only if el["text"] == "Sweden"

fixSystematicMost_common_list_countries_FULL_MATCH("base", DF_input_update_base, list_csvs_update_base, "United Kingdom", "Sweden", "Sweden")
fixSystematicMost_common_list_countries_FULL_MATCH("match GLobocan", DF_input_update_match_GLOBOCAN, list_csvs_update_match_GLOBOCAN, "United Kingdom", "Sweden", "Sweden")

--Fixing base dataset
Parsing csv: temp_isna_notna.csv (1/1)
--Fixing done for base dataset
--Fixing match GLobocan dataset
Parsing csv: temp_isna_notna.csv (1/1)
--Fixing done for match GLobocan dataset


In [19]:
## Fix wrong Country assignment in Most_common_list_countries: Denmark instead of United Kingdom in records with Denmark Hill

# Important: SUBSTRINGS not taken into consideration, corrected only if el["text"] == "Denmark Hill"

fixSystematicMost_common_list_countries_FULL_MATCH("base", DF_input_update_base, list_csvs_update_base, "Denmark", "Denmark Hill", "United Kingdom")
fixSystematicMost_common_list_countries_FULL_MATCH("match GLobocan", DF_input_update_match_GLOBOCAN, list_csvs_update_match_GLOBOCAN, "Denmark", "Denmark Hill", "United Kingdom")

--Fixing base dataset
Parsing csv: temp_isna_notna.csv (1/1)
--Fixing done for base dataset
--Fixing match GLobocan dataset
Parsing csv: temp_isna_notna.csv (1/1)
--Fixing done for match GLobocan dataset


In [20]:
## Fix wrong Country assignment in Most_common_list_countries: Madagascar instead of Brazil in records with MG

# Important: SUBSTRINGS not taken into consideration, corrected only if el["text"] == "MG"

fixSystematicMost_common_list_countries_FULL_MATCH("base", DF_input_update_base, list_csvs_update_base, "Madagascar", "MG", "Brazil")
fixSystematicMost_common_list_countries_FULL_MATCH("match GLobocan", DF_input_update_match_GLOBOCAN, list_csvs_update_match_GLOBOCAN, "Madagascar", "MG", "Brazil")

--Fixing base dataset
Parsing csv: temp_isna_notna.csv (1/1)
--Fixing done for base dataset
--Fixing match GLobocan dataset
Parsing csv: temp_isna_notna.csv (1/1)
--Fixing done for match GLobocan dataset


In [21]:
## Fix wrong Country assignment in Most_common_list_countries: Mexico instead of Colombia in records with Col

# Important: SUBSTRINGS not taken into consideration, corrected only if el["text"] == "Col"

fixSystematicMost_common_list_countries_FULL_MATCH("base", DF_input_update_base, list_csvs_update_base, "Colombia", "Col", "Mexico")
fixSystematicMost_common_list_countries_FULL_MATCH("match GLobocan", DF_input_update_match_GLOBOCAN, list_csvs_update_match_GLOBOCAN, "Colombia", "Col", "Mexico")

--Fixing base dataset
Parsing csv: temp_isna_notna.csv (1/1)
--Fixing done for base dataset
--Fixing match GLobocan dataset
Parsing csv: temp_isna_notna.csv (1/1)
--Fixing done for match GLobocan dataset


In [22]:
## Fix wrong Country assignment in Most_common_list_countries: Jamaica instead of Colombia in records with Jamaica

# Important: SUBSTRINGS not taken into consideration, corrected only if el["text"] == "Jamaica"
# Important: some records should be assinged to other countries. These are done manually below

fixSystematicMost_common_list_countries_FULL_MATCH("base", DF_input_update_base, list_csvs_update_base, "Jamaica", "Jamaica", "United States")
fixSystematicMost_common_list_countries_FULL_MATCH("match GLobocan", DF_input_update_match_GLOBOCAN, list_csvs_update_match_GLOBOCAN, "Jamaica", "Jamaica", "United States")

--Fixing base dataset
Parsing csv: temp_isna_notna.csv (1/1)
--Fixing done for base dataset
--Fixing match GLobocan dataset
Parsing csv: temp_isna_notna.csv (1/1)
--Fixing done for match GLobocan dataset


In [23]:
## Fix wrong Country assignment in Most_common_list_countries: Spain instead of Brazil in records with ES

# Important: SUBSTRINGS not taken into consideration, corrected only if el["text"] == "ES"
# Important: some records should be assinged to other countries. These are done manually below

fixSystematicMost_common_list_countries_FULL_MATCH("base", DF_input_update_base, list_csvs_update_base, "Spain", "ES", "Brazil")
fixSystematicMost_common_list_countries_FULL_MATCH("match GLobocan", DF_input_update_match_GLOBOCAN, list_csvs_update_match_GLOBOCAN, "Spain", "ES", "Brazil")

--Fixing base dataset
Parsing csv: temp_isna_notna.csv (1/1)
--Fixing done for base dataset
--Fixing match GLobocan dataset
Parsing csv: temp_isna_notna.csv (1/1)
--Fixing done for match GLobocan dataset


In [24]:
## Fix wrong Country assignment in Most_common_list_countries: Serbia instead of Brazil in records with RS

# Important: SUBSTRINGS not taken into consideration, corrected only if el["text"] == "RS"

fixSystematicMost_common_list_countries_FULL_MATCH("base", DF_input_update_base, list_csvs_update_base, "Serbia", "RS", "Brazil")
fixSystematicMost_common_list_countries_FULL_MATCH("match GLobocan", DF_input_update_match_GLOBOCAN, list_csvs_update_match_GLOBOCAN, "Serbia", "RS", "Brazil")

--Fixing base dataset
Parsing csv: temp_isna_notna.csv (1/1)
--Fixing done for base dataset
--Fixing match GLobocan dataset
Parsing csv: temp_isna_notna.csv (1/1)
--Fixing done for match GLobocan dataset


In [25]:
## Fix wrong Country assignment in Most_common_list_countries: Paraguay instead of Argentina in records with Paraguay

# Important: SUBSTRINGS not taken into consideration, corrected only if el["text"] == "Paraguay"
# Important: some records should be assinged to other countries. These are done manually below

fixSystematicMost_common_list_countries_FULL_MATCH("base", DF_input_update_base, list_csvs_update_base, "Paraguay", "Paraguay", "Argentina")
fixSystematicMost_common_list_countries_FULL_MATCH("match GLobocan", DF_input_update_match_GLOBOCAN, list_csvs_update_match_GLOBOCAN, "Paraguay", "Paraguay", "Argentina")

--Fixing base dataset
Parsing csv: temp_isna_notna.csv (1/1)
--Fixing done for base dataset
--Fixing match GLobocan dataset
Parsing csv: temp_isna_notna.csv (1/1)
--Fixing done for match GLobocan dataset


In [26]:
## Fix wrong Country assignment in Most_common_list_countries: Slovakia instead of Canada in records with SK

# Important: SUBSTRINGS not taken into consideration, corrected only if el["text"] == "SK"
# Important: some records should be assinged to other countries. These are done manually below

fixSystematicMost_common_list_countries_FULL_MATCH("base", DF_input_update_base, list_csvs_update_base, "Slovakia", "SK", "Canada")
fixSystematicMost_common_list_countries_FULL_MATCH("match GLobocan", DF_input_update_match_GLOBOCAN, list_csvs_update_match_GLOBOCAN, "Slovakia", "SK", "Canada")

--Fixing base dataset
Parsing csv: temp_isna_notna.csv (1/1)
--Fixing done for base dataset
--Fixing match GLobocan dataset
Parsing csv: temp_isna_notna.csv (1/1)
--Fixing done for match GLobocan dataset


In [27]:
## Fix wrong Country assignment in Most_common_list_countries: Bosnia and Herzegovina instead of Brazil in records with BA

# Important: SUBSTRINGS not taken into consideration, corrected only if el["text"] == "BA"
# Important: some records should be assinged to other countries. These are done manually below

fixSystematicMost_common_list_countries_FULL_MATCH("base", DF_input_update_base, list_csvs_update_base, "Bosnia and Herzegovina", "BA", "Brazil")
fixSystematicMost_common_list_countries_FULL_MATCH("match GLobocan", DF_input_update_match_GLOBOCAN, list_csvs_update_match_GLOBOCAN, "Bosnia and Herzegovina", "BA", "Brazil")

--Fixing base dataset
Parsing csv: temp_isna_notna.csv (1/1)
--Fixing done for base dataset
--Fixing match GLobocan dataset
Parsing csv: temp_isna_notna.csv (1/1)
--Fixing done for match GLobocan dataset


In [28]:
## Fix wrong Country assignment in Most_common_list_countries: Benin instead of Nigeria in records with Benin City

# Important: SUBSTRINGS not taken into consideration, corrected only if el["text"] == "Benin City"

fixSystematicMost_common_list_countries_FULL_MATCH("base", DF_input_update_base, list_csvs_update_base, "Benin", "Benin City", "Nigeria")
fixSystematicMost_common_list_countries_FULL_MATCH("match GLobocan", DF_input_update_match_GLOBOCAN, list_csvs_update_match_GLOBOCAN, "Benin", "Benin City", "Nigeria")

--Fixing base dataset
Parsing csv: temp_isna_notna.csv (1/1)
--Fixing done for base dataset
--Fixing match GLobocan dataset
Parsing csv: temp_isna_notna.csv (1/1)
--Fixing done for match GLobocan dataset


In [29]:
## Fix wrong Country assignment in Most_common_list_countries: Bolivia instead of Italy in records with BO

# Important: SUBSTRINGS not taken into consideration, corrected only if el["text"] == "BO"

fixSystematicMost_common_list_countries_FULL_MATCH("base", DF_input_update_base, list_csvs_update_base, "Bolivia", "BO", "Italy")
fixSystematicMost_common_list_countries_FULL_MATCH("match GLobocan", DF_input_update_match_GLOBOCAN, list_csvs_update_match_GLOBOCAN, "Bolivia", "BO", "Italy")

--Fixing base dataset
Parsing csv: temp_isna_notna.csv (1/1)
--Fixing done for base dataset
--Fixing match GLobocan dataset
Parsing csv: temp_isna_notna.csv (1/1)
--Fixing done for match GLobocan dataset


In [30]:
## Fix wrong Country assignment in Most_common_list_countries: United Kingdom instead of China in records with Ningbo

# Important: SUBSTRINGS not taken into consideration, corrected only if el["text"] == "Ningbo"

fixSystematicMost_common_list_countries_FULL_MATCH("base", DF_input_update_base, list_csvs_update_base, "United Kingdom", "Ningbo", "China")
fixSystematicMost_common_list_countries_FULL_MATCH("match GLobocan", DF_input_update_match_GLOBOCAN, list_csvs_update_match_GLOBOCAN, "United Kingdom", "Ningbo", "China")

--Fixing base dataset
Parsing csv: temp_isna_notna.csv (1/1)
--Fixing done for base dataset
--Fixing match GLobocan dataset
Parsing csv: temp_isna_notna.csv (1/1)
--Fixing done for match GLobocan dataset


### Correcting *temp_isna_notna.csv*

In [70]:
# Read csv
df_temp_isna_notna_base = pd.read_csv(DF_input_update_base + "temp_isna_notna.csv")
df_temp_isna_notna_match_GLOBOCAN = pd.read_csv(DF_input_update_match_GLOBOCAN + "temp_isna_notna.csv")

In [ ]:
## Replace found errors by true Country values

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 38557413, ["Country", "Country_source"]] = fix_country_from_text("Türkiye")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 38557413, ["Country", "Country_source"]] = fix_country_from_text("Türkiye")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 38602342, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 38602342, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 38634871, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 38634871, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 38743429, ["Country", "Country_source"]] = fix_country_from_text("China")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 38743429, ["Country", "Country_source"]] = fix_country_from_text("China")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 38769640, ["Country", "Country_source"]] = fix_country_from_text("France")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 38769640, ["Country", "Country_source"]] = fix_country_from_text("France")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 38780371, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 38780371, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 38858110, ["Country", "Country_source"]] = fix_country_from_text("China")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 38858110, ["Country", "Country_source"]] = fix_country_from_text("China")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 38968592, ["Country", "Country_source"]] = fix_country_from_text("China")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 38968592, ["Country", "Country_source"]] = fix_country_from_text("China")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 38991198, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 38991198, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 39010310, ["Country", "Country_source"]] = fix_country_from_text("Israel")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 39010310, ["Country", "Country_source"]] = fix_country_from_text("Israel")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 39041894, ["Country", "Country_source"]] = fix_country_from_text("France")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 39041894, ["Country", "Country_source"]] = fix_country_from_text("France")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 39101313, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 39101313, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 39118510, ["Country", "Country_source"]] = fix_country_from_text("China")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 39118510, ["Country", "Country_source"]] = fix_country_from_text("China")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 39190399, ["Country", "Country_source"]] = fix_country_from_text("Kuwait")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 39190399, ["Country", "Country_source"]] = fix_country_from_text("Kuwait")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 39192503, ["Country", "Country_source"]] = fix_country_from_text("China")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 39192503, ["Country", "Country_source"]] = fix_country_from_text("China")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 39264582, ["Country", "Country_source"]] = fix_country_from_text("China")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 39264582, ["Country", "Country_source"]] = fix_country_from_text("China")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 39280546, ["Country", "Country_source"]] = fix_country_from_text("Saudi Arabia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 39280546, ["Country", "Country_source"]] = fix_country_from_text("Saudi Arabia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 39285489, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 39285489, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 39356018, ["Country", "Country_source"]] = fix_country_from_text("Romania")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 39356018, ["Country", "Country_source"]] = fix_country_from_text("Romania")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 39388311, ["Country", "Country_source"]] = fix_country_from_text("China")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 39388311, ["Country", "Country_source"]] = fix_country_from_text("China")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 39463531, ["Country", "Country_source"]] = fix_country_from_text("Saudi Arabia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 39463531, ["Country", "Country_source"]] = fix_country_from_text("Saudi Arabia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 39629688, ["Country", "Country_source"]] = fix_country_from_text("Israel")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 39629688, ["Country", "Country_source"]] = fix_country_from_text("Israel")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 39774161, ["Country", "Country_source"]] = fix_country_from_text("Türkiye")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 39774161, ["Country", "Country_source"]] = fix_country_from_text("Türkiye")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 39774244, ["Country", "Country_source"]] = fix_country_from_text("Türkiye")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 39774244, ["Country", "Country_source"]] = fix_country_from_text("Türkiye")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 39801191, ["Country", "Country_source"]] = fix_country_from_text("Iran")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 39801191, ["Country", "Country_source"]] = fix_country_from_text("Iran")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 39853190, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 39853190, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 40026197, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 40026197, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 40059303, ["Country", "Country_source"]] = fix_country_from_text("China")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 40059303, ["Country", "Country_source"]] = fix_country_from_text("China")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 40099373, ["Country", "Country_source"]] = fix_country_from_text("China")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 40099373, ["Country", "Country_source"]] = fix_country_from_text("China")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 40263662, ["Country", "Country_source"]] = fix_country_from_text("China")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 40263662, ["Country", "Country_source"]] = fix_country_from_text("China")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 40263981, ["Country", "Country_source"]] = fix_country_from_text("South Korea")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 40263981, ["Country", "Country_source"]] = fix_country_from_text("South Korea")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 40304267, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 40304267, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 40325961, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 40325961, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 40369806, ["Country", "Country_source"]] = fix_country_from_text("Kuwait")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 40369806, ["Country", "Country_source"]] = fix_country_from_text("Kuwait")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 40377590, ["Country", "Country_source"]] = fix_country_from_text("Malaysia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 40377590, ["Country", "Country_source"]] = fix_country_from_text("Malaysia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 40626442, ["Country", "Country_source"]] = fix_country_from_text("Türkiye")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 40626442, ["Country", "Country_source"]] = fix_country_from_text("Türkiye")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1443468, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1443468, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1464019, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1464019, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1464201, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1464201, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1476653, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1476653, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1512999, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1512999, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1513040, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1513040, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1535131, ["Country", "Country_source"]] = fix_country_from_text("France")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1535131, ["Country", "Country_source"]] = fix_country_from_text("France")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1536260, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1536260, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1557175, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1557175, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1558408, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1558408, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1578645, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1578645, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1598432, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1598432, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1604074, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1604074, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1605697, ["Country", "Country_source"]] = fix_country_from_text("Tunisia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1605697, ["Country", "Country_source"]] = fix_country_from_text("Tunisia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1629369, ["Country", "Country_source"]] = fix_country_from_text("Croatia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1629369, ["Country", "Country_source"]] = fix_country_from_text("Croatia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1630573, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1630573, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1631476, ["Country", "Country_source"]] = fix_country_from_text("France")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1631476, ["Country", "Country_source"]] = fix_country_from_text("France")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1634438, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1634438, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1644320, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1644320, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1645769, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1645769, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1648638, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1648638, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1659135, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1659135, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1683348, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1683348, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1693829, ["Country", "Country_source"]] = fix_country_from_text("France")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1693829, ["Country", "Country_source"]] = fix_country_from_text("France")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1694276, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1694276, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1697629, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1697629, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1697658, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1697658, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1698519, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1698519, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1699002, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1699002, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1701859, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1701859, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1701860, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1701860, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1709464, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1709464, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1710101, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1710101, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1710328, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1710328, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1721773, ["Country", "Country_source"]] = fix_country_from_text("France")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1721773, ["Country", "Country_source"]] = fix_country_from_text("France")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1723160, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1723160, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1738883, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1738883, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1747364, ["Country", "Country_source"]] = fix_country_from_text("France")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1747364, ["Country", "Country_source"]] = fix_country_from_text("France")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1752977, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1752977, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1753419, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1753419, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1762341, ["Country", "Country_source"]] = fix_country_from_text("Serbia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1762341, ["Country", "Country_source"]] = fix_country_from_text("Serbia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1766137, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1766137, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1790507, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1790507, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1791127, ["Country", "Country_source"]] = fix_country_from_text("Ireland")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1791127, ["Country", "Country_source"]] = fix_country_from_text("Ireland")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1792632, ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1792632, ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1792632, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1792632, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1792999, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1792999, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1809281, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1809281, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1809496, ["Country", "Country_source"]] = fix_country_from_text("France")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1809496, ["Country", "Country_source"]] = fix_country_from_text("France")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1810418, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1810418, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1811002, ["Country", "Country_source"]] = fix_country_from_text("France")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1811002, ["Country", "Country_source"]] = fix_country_from_text("France")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1811989, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1811989, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1815319, ["Country", "Country_source"]] = fix_country_from_text("Czechia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1815319, ["Country", "Country_source"]] = fix_country_from_text("Czechia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1815320, ["Country", "Country_source"]] = fix_country_from_text("Czechia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1815320, ["Country", "Country_source"]] = fix_country_from_text("Czechia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1827325, ["Country", "Country_source"]] = fix_country_from_text("France")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1827325, ["Country", "Country_source"]] = fix_country_from_text("France")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1832758, ["Country", "Country_source"]] = fix_country_from_text("France")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1832758, ["Country", "Country_source"]] = fix_country_from_text("France")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1838533, ["Country", "Country_source"]] = fix_country_from_text("China")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1838533, ["Country", "Country_source"]] = fix_country_from_text("China")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1839762, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1839762, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1842227, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1842227, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1848151, ["Country", "Country_source"]] = fix_country_from_text("China")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1848151, ["Country", "Country_source"]] = fix_country_from_text("China")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1848914, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1848914, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1851326, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1851326, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1866807, ["Country", "Country_source"]] = fix_country_from_text("Tunisia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1866807, ["Country", "Country_source"]] = fix_country_from_text("Tunisia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1871504, ["Country", "Country_source"]] = fix_country_from_text("France")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1871504, ["Country", "Country_source"]] = fix_country_from_text("France")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1911479, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1911479, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1914211, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1914211, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1920994, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1920994, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1921865, ["Country", "Country_source"]] = fix_country_from_text("Serbia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1921865, ["Country", "Country_source"]] = fix_country_from_text("Serbia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1933108, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1933108, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1942528, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1942528, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1942619, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1942619, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1959730, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1959730, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1971948, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1971948, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1975831, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1975831, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1976119, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1976119, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 1976210, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 1976210, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 11280913, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 11280913, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 11289733, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 11289733, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 11308248, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 11308248, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 11342326, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 11342326, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 11360572, ["Country", "Country_source"]] = fix_country_from_text("China")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 11360572, ["Country", "Country_source"]] = fix_country_from_text("China")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 11402899, ["Country", "Country_source"]] = fix_country_from_text("South Africa")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 11402899, ["Country", "Country_source"]] = fix_country_from_text("South Africa")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 11424393, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 11424393, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 11432224, ["Country", "Country_source"]] = fix_country_from_text("Serbia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 11432224, ["Country", "Country_source"]] = fix_country_from_text("Serbia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 11432226, ["Country", "Country_source"]] = fix_country_from_text("Serbia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 11432226, ["Country", "Country_source"]] = fix_country_from_text("Serbia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 11432249, ["Country", "Country_source"]] = fix_country_from_text("Serbia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 11432249, ["Country", "Country_source"]] = fix_country_from_text("Serbia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 11432258, ["Country", "Country_source"]] = fix_country_from_text("Serbia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 11432258, ["Country", "Country_source"]] = fix_country_from_text("Serbia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 11433604, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 11433604, ["Country", "Country_source"]] = fix_country_from_text("Slovakia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 11433740, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 11433740, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 11441469, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 11441469, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 11452488, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 11452488, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 11469092, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 11469092, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 11480015, ["Country", "Country_source"]] = fix_country_from_text("China")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 11480015, ["Country", "Country_source"]] = fix_country_from_text("China")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 11480016, ["Country", "Country_source"]] = fix_country_from_text("China")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 11480016, ["Country", "Country_source"]] = fix_country_from_text("China")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 11525036, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 11525036, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 11547468, ["Country", "Country_source"]] = fix_country_from_text("Israel")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 11547468, ["Country", "Country_source"]] = fix_country_from_text("Israel")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 11620576, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 11620576, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 11669022, ["Country", "Country_source"]] = fix_country_from_text("Czechia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 11669022, ["Country", "Country_source"]] = fix_country_from_text("Czechia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 11688067, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 11688067, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 11714449, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 11714449, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 11722421, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 11722421, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 11744953, ["Country", "Country_source"]] = fix_country_from_text("Yemen")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 11744953, ["Country", "Country_source"]] = fix_country_from_text("Yemen")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 11761999, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 11761999, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 11762044, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 11762044, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 11808117, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 11808117, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 11808216, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 11808216, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 11808289, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 11808289, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 11808290, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 11808290, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 11838113, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 11838113, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 11857974, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 11857974, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 11889989, ["Country", "Country_source"]] = fix_country_from_text("Serbia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 11889989, ["Country", "Country_source"]] = fix_country_from_text("Serbia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 11905082, ["Country", "Country_source"]] = fix_country_from_text("Israel")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 11905082, ["Country", "Country_source"]] = fix_country_from_text("Israel")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 11925893, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 11925893, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 11977307, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 11977307, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 11977338, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 11977338, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 11979737, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 11979737, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 11993788, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 11993788, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 12029721, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 12029721, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 12056041, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 12056041, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 12056072, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 12056072, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 12068994, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 12068994, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 12087713, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 12087713, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 12101761, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 12101761, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 12114668, ["Country", "Country_source"]] = fix_country_from_text("Türkiye")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 12114668, ["Country", "Country_source"]] = fix_country_from_text("Türkiye")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 12138643, ["Country", "Country_source"]] = fix_country_from_text("Hungary")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 12138643, ["Country", "Country_source"]] = fix_country_from_text("Hungary")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 12170287, ["Country", "Country_source"]] = fix_country_from_text("Peru")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 12170287, ["Country", "Country_source"]] = fix_country_from_text("Peru")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 12197544, ["Country", "Country_source"]] = fix_country_from_text("South Korea")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 12197544, ["Country", "Country_source"]] = fix_country_from_text("South Korea")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 12233224, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 12233224, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 12238122, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 12238122, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 12238123, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 12238123, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 12402435, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 12402435, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 12402786, ["Country", "Country_source"]] = fix_country_from_text("Czechia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 12402786, ["Country", "Country_source"]] = fix_country_from_text("Czechia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 12422771, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 12422771, ["Country", "Country_source"]] = fix_country_from_text("Switzerland")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 12452074, ["Country", "Country_source"]] = fix_country_from_text("South Korea")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 12452074, ["Country", "Country_source"]] = fix_country_from_text("South Korea")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 12457338, ["Country", "Country_source"]] = fix_country_from_text("Serbia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 12457338, ["Country", "Country_source"]] = fix_country_from_text("Serbia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 12541774, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 12541774, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 12552756, ["Country", "Country_source"]] = fix_country_from_text("China")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 12552756, ["Country", "Country_source"]] = fix_country_from_text("China")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 12577016, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 12577016, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 12587489, ["Country", "Country_source"]] = fix_country_from_text("Serbia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 12587489, ["Country", "Country_source"]] = fix_country_from_text("Serbia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 12587492, ["Country", "Country_source"]] = fix_country_from_text("Serbia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 12587492, ["Country", "Country_source"]] = fix_country_from_text("Serbia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 12607296, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 12607296, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 12640323, ["Country", "Country_source"]] = fix_country_from_text("France")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 12640323, ["Country", "Country_source"]] = fix_country_from_text("France")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 12649851, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 12649851, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 12669396, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 12669396, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 12704299, ["Country", "Country_source"]] = fix_country_from_text("France")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 12704299, ["Country", "Country_source"]] = fix_country_from_text("France")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 12715370, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 12715370, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 12717985, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 12717985, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 12731218, ["Country", "Country_source"]] = fix_country_from_text("Romania")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 12731218, ["Country", "Country_source"]] = fix_country_from_text("Romania")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 12756890, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 12756890, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 12759658, ["Country", "Country_source"]] = fix_country_from_text("Tunisia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 12759658, ["Country", "Country_source"]] = fix_country_from_text("Tunisia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 12768872, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 12768872, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 12833861, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 12833861, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 12869919, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 12869919, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 12879657, ["Country", "Country_source"]] = fix_country_from_text("Czechia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 12879657, ["Country", "Country_source"]] = fix_country_from_text("Czechia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 12879666, ["Country", "Country_source"]] = fix_country_from_text("Czechia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 12879666, ["Country", "Country_source"]] = fix_country_from_text("Czechia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 12882053, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 12882053, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 12898831, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 12898831, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 12898833, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 12898833, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 12898839, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 12898839, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 12898843, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 12898843, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 12898890, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 12898890, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 12931571, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 12931571, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 12934262, ["Country", "Country_source"]] = fix_country_from_text("Argentina")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 12934262, ["Country", "Country_source"]] = fix_country_from_text("Argentina")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 14502867, ["Country", "Country_source"]] = fix_country_from_text("Hungary")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 14502867, ["Country", "Country_source"]] = fix_country_from_text("Hungary")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 14503335, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 14503335, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 14518166, ["Country", "Country_source"]] = fix_country_from_text("Israel")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 14518166, ["Country", "Country_source"]] = fix_country_from_text("Israel")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 14569941, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 14569941, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 14574936, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 14574936, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 14577361, ["Country", "Country_source"]] = fix_country_from_text("Bulgaria")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 14577361, ["Country", "Country_source"]] = fix_country_from_text("Bulgaria")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 14584399, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 14584399, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 14630754, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 14630754, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 14669130, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 14669130, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 14672014, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 14672014, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 14682012, ["Country", "Country_source"]] = fix_country_from_text("Bulgaria")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 14682012, ["Country", "Country_source"]] = fix_country_from_text("Bulgaria")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 14708516, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 14708516, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 14733108, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 14733108, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 14758224, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 14758224, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 14763157, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 14763157, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 14870535, ["Country", "Country_source"]] = fix_country_from_text("Romania")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 14870535, ["Country", "Country_source"]] = fix_country_from_text("Romania")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 14964972, ["Country", "Country_source"]] = fix_country_from_text("Russia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 14964972, ["Country", "Country_source"]] = fix_country_from_text("Russia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 14994564, ["Country", "Country_source"]] = fix_country_from_text("Serbia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 14994564, ["Country", "Country_source"]] = fix_country_from_text("Serbia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 14994574, ["Country", "Country_source"]] = fix_country_from_text("Serbia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 14994574, ["Country", "Country_source"]] = fix_country_from_text("Serbia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 14994581, ["Country", "Country_source"]] = fix_country_from_text("Serbia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 14994581, ["Country", "Country_source"]] = fix_country_from_text("Serbia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 14998291, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 14998291, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15035535, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15035535, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15058640, ["Country", "Country_source"]] = fix_country_from_text("Pakistan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15058640, ["Country", "Country_source"]] = fix_country_from_text("Pakistan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15067809, ["Country", "Country_source"]] = fix_country_from_text("Bulgaria")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15067809, ["Country", "Country_source"]] = fix_country_from_text("Bulgaria")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15094586, ["Country", "Country_source"]] = fix_country_from_text("France")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15094586, ["Country", "Country_source"]] = fix_country_from_text("France")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15106341, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15106341, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15106713, ["Country", "Country_source"]] = fix_country_from_text("India")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15106713, ["Country", "Country_source"]] = fix_country_from_text("India")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15134038, ["Country", "Country_source"]] = fix_country_from_text("Czechia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15134038, ["Country", "Country_source"]] = fix_country_from_text("Czechia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15148814, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15148814, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15179755, ["Country", "Country_source"]] = fix_country_from_text("Serbia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15179755, ["Country", "Country_source"]] = fix_country_from_text("Serbia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15179756, ["Country", "Country_source"]] = fix_country_from_text("Serbia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15179756, ["Country", "Country_source"]] = fix_country_from_text("Serbia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15179759, ["Country", "Country_source"]] = fix_country_from_text("Serbia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15179759, ["Country", "Country_source"]] = fix_country_from_text("Serbia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15179765, ["Country", "Country_source"]] = fix_country_from_text("Serbia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15179765, ["Country", "Country_source"]] = fix_country_from_text("Serbia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15179771, ["Country", "Country_source"]] = fix_country_from_text("Serbia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15179771, ["Country", "Country_source"]] = fix_country_from_text("Serbia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15185603, ["Country", "Country_source"]] = fix_country_from_text("Tunisia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15185603, ["Country", "Country_source"]] = fix_country_from_text("Tunisia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15205192, ["Country", "Country_source"]] = fix_country_from_text("France")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15205192, ["Country", "Country_source"]] = fix_country_from_text("France")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15216970, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15216970, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15222107, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15222107, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15298124, ["Country", "Country_source"]] = fix_country_from_text("Bangladesh")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15298124, ["Country", "Country_source"]] = fix_country_from_text("Bangladesh")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15305160, ["Country", "Country_source"]] = fix_country_from_text("Greece")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15305160, ["Country", "Country_source"]] = fix_country_from_text("Greece")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15307496, ["Country", "Country_source"]] = fix_country_from_text("Serbia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15307496, ["Country", "Country_source"]] = fix_country_from_text("Serbia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15307510, ["Country", "Country_source"]] = fix_country_from_text("Serbia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15307510, ["Country", "Country_source"]] = fix_country_from_text("Serbia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15315013, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15315013, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15323189, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15323189, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15323196, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15323196, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15323197, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15323197, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15323223, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15323223, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15323224, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15323224, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15323229, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15323229, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15330308, ["Country", "Country_source"]] = fix_country_from_text("Serbia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15330308, ["Country", "Country_source"]] = fix_country_from_text("Serbia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15332549, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15332549, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15446564, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15446564, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15446566, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15446566, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15446567, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15446567, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15462546, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15462546, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15478652, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15478652, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15500145, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15500145, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15500157, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15500157, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15510633, ["Country", "Country_source"]] = fix_country_from_text("China")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15510633, ["Country", "Country_source"]] = fix_country_from_text("China")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15526589, ["Country", "Country_source"]] = fix_country_from_text("Serbia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15526589, ["Country", "Country_source"]] = fix_country_from_text("Serbia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15535207, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15535207, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15535222, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15535222, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15535224, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15535224, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15535241, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15535241, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15535246, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15535246, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15535255, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15535255, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15535276, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15535276, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15535305, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15535305, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15535318, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15535318, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15535531, ["Country", "Country_source"]] = fix_country_from_text("Brazil")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15535531, ["Country", "Country_source"]] = fix_country_from_text("Brazil")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15553697, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15553697, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15565998, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15565998, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15566661, ["Country", "Country_source"]] = fix_country_from_text("China")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15566661, ["Country", "Country_source"]] = fix_country_from_text("China")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15568009, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15568009, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15570926, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15570926, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15619977, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15619977, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15628764, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15628764, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15651387, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15651387, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15687948, ["Country", "Country_source"]] = fix_country_from_text("France")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15687948, ["Country", "Country_source"]] = fix_country_from_text("France")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15711911, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15711911, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15751632, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15751632, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15756782, ["Country", "Country_source"]] = fix_country_from_text("Serbia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15756782, ["Country", "Country_source"]] = fix_country_from_text("Serbia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15756787, ["Country", "Country_source"]] = fix_country_from_text("Serbia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15756787, ["Country", "Country_source"]] = fix_country_from_text("Serbia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15771281, ["Country", "Country_source"]] = fix_country_from_text("Israel")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15771281, ["Country", "Country_source"]] = fix_country_from_text("Israel")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15773318, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15773318, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15775226, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15775226, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15776651, ["Country", "Country_source"]] = fix_country_from_text("Senegal")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15776651, ["Country", "Country_source"]] = fix_country_from_text("Senegal")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15853985, ["Country", "Country_source"]] = fix_country_from_text("Brunei Darussalam")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15853985, ["Country", "Country_source"]] = fix_country_from_text("Brunei Darussalam")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15859941, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15859941, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15877249, ["Country", "Country_source"]] = fix_country_from_text("Türkiye")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15877249, ["Country", "Country_source"]] = fix_country_from_text("Türkiye")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15902260, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15902260, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15918264, ["Country", "Country_source"]] = fix_country_from_text("Czechia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15918264, ["Country", "Country_source"]] = fix_country_from_text("Czechia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15925991, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15925991, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15981630, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15981630, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15988920, ["Country", "Country_source"]] = fix_country_from_text("Hungary")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15988920, ["Country", "Country_source"]] = fix_country_from_text("Hungary")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 15999508, ["Country", "Country_source"]] = fix_country_from_text("Finland")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 15999508, ["Country", "Country_source"]] = fix_country_from_text("Finland")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16001726, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16001726, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16018372, ["Country", "Country_source"]] = fix_country_from_text("Serbia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16018372, ["Country", "Country_source"]] = fix_country_from_text("Serbia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16018405, ["Country", "Country_source"]] = fix_country_from_text("Serbia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16018405, ["Country", "Country_source"]] = fix_country_from_text("Serbia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16018409, ["Country", "Country_source"]] = fix_country_from_text("Serbia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16018409, ["Country", "Country_source"]] = fix_country_from_text("Serbia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16018414, ["Country", "Country_source"]] = fix_country_from_text("Serbia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16018414, ["Country", "Country_source"]] = fix_country_from_text("Serbia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16044973, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16044973, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16086280, ["Country", "Country_source"]] = fix_country_from_text("China")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16086280, ["Country", "Country_source"]] = fix_country_from_text("China")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16101226, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16101226, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16119312, ["Country", "Country_source"]] = fix_country_from_text("Serbia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16119312, ["Country", "Country_source"]] = fix_country_from_text("Serbia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16119315, ["Country", "Country_source"]] = fix_country_from_text("Serbia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16119315, ["Country", "Country_source"]] = fix_country_from_text("Serbia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16119317, ["Country", "Country_source"]] = fix_country_from_text("Serbia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16119317, ["Country", "Country_source"]] = fix_country_from_text("Serbia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16119319, ["Country", "Country_source"]] = fix_country_from_text("Serbia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16119319, ["Country", "Country_source"]] = fix_country_from_text("Serbia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16119321, ["Country", "Country_source"]] = fix_country_from_text("Serbia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16119321, ["Country", "Country_source"]] = fix_country_from_text("Serbia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16127359, ["Country", "Country_source"]] = fix_country_from_text("United Arab Emirates")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16127359, ["Country", "Country_source"]] = fix_country_from_text("United Arab Emirates")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16130901, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16130901, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16149453, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16149453, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16149641, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16149641, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16201592, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16201592, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16227753, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16227753, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16230292, ["Country", "Country_source"]] = fix_country_from_text("France")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16230292, ["Country", "Country_source"]] = fix_country_from_text("France")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16230407, ["Country", "Country_source"]] = fix_country_from_text("Austria")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16230407, ["Country", "Country_source"]] = fix_country_from_text("Austria")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16230931, ["Country", "Country_source"]] = fix_country_from_text("France")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16230931, ["Country", "Country_source"]] = fix_country_from_text("France")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16241098, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16241098, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16261714, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16261714, ["Country", "Country_source"]] = fix_country_from_text("The Netherlands")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16279665, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16279665, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16285591, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16285591, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16315917, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16315917, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16324434, ["Country", "Country_source"]] = fix_country_from_text("Denmark")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16324434, ["Country", "Country_source"]] = fix_country_from_text("Denmark")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16366365, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16366365, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16373403, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16373403, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16393559, ["Country", "Country_source"]] = fix_country_from_text("Denmark")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16393559, ["Country", "Country_source"]] = fix_country_from_text("Denmark")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16395494, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16395494, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16401376, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16401376, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16405236, ["Country", "Country_source"]] = fix_country_from_text("Serbia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16405236, ["Country", "Country_source"]] = fix_country_from_text("Serbia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16433466, ["Country", "Country_source"]] = fix_country_from_text("France")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16433466, ["Country", "Country_source"]] = fix_country_from_text("France")

# Taiwan in base, China in match GLOBOCAN
df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16435130, ["Country", "Country_source"]] = fix_country_from_text("Taiwan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16435130, ["Country", "Country_source"]] = fix_country_from_text("China")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16435508, ["Country", "Country_source"]] = fix_country_from_text("France")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16435508, ["Country", "Country_source"]] = fix_country_from_text("France")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16449933, ["Country", "Country_source"]] = fix_country_from_text("France")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16449933, ["Country", "Country_source"]] = fix_country_from_text("France")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16454176, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16454176, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16457252, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16457252, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16461294, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16461294, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16461307, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16461307, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16501672, ["Country", "Country_source"]] = fix_country_from_text("Jordan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16501672, ["Country", "Country_source"]] = fix_country_from_text("Jordan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16502867, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16502867, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16505780, ["Country", "Country_source"]] = fix_country_from_text("France")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16505780, ["Country", "Country_source"]] = fix_country_from_text("France")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16509415, ["Country", "Country_source"]] = fix_country_from_text("Israel")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16509415, ["Country", "Country_source"]] = fix_country_from_text("Israel")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16550875, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16550875, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16579027, ["Country", "Country_source"]] = fix_country_from_text("Canada")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16579027, ["Country", "Country_source"]] = fix_country_from_text("Canada")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16604799, ["Country", "Country_source"]] = fix_country_from_text("France")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16604799, ["Country", "Country_source"]] = fix_country_from_text("France")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16607835, ["Country", "Country_source"]] = fix_country_from_text("Romania")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16607835, ["Country", "Country_source"]] = fix_country_from_text("Romania")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16608010, ["Country", "Country_source"]] = fix_country_from_text("Belgium")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16608010, ["Country", "Country_source"]] = fix_country_from_text("Belgium")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16613159, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16613159, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16615230, ["Country", "Country_source"]] = fix_country_from_text("Ireland")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16615230, ["Country", "Country_source"]] = fix_country_from_text("Ireland")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16624392, ["Country", "Country_source"]] = fix_country_from_text("United States")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16624392, ["Country", "Country_source"]] = fix_country_from_text("United States")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16634544, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16634544, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16681248, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16681248, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16711508, ["Country", "Country_source"]] = fix_country_from_text("Denmark")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16711508, ["Country", "Country_source"]] = fix_country_from_text("Denmark")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16733462, ["Country", "Country_source"]] = fix_country_from_text("France")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16733462, ["Country", "Country_source"]] = fix_country_from_text("France")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16752888, ["Country", "Country_source"]] = fix_country_from_text("Italy")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16752888, ["Country", "Country_source"]] = fix_country_from_text("Italy")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16758801, ["Country", "Country_source"]] = fix_country_from_text("Serbia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16758801, ["Country", "Country_source"]] = fix_country_from_text("Serbia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16770096, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16770096, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16776193, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16776193, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16787624, ["Country", "Country_source"]] = fix_country_from_text("Pakistan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16787624, ["Country", "Country_source"]] = fix_country_from_text("Pakistan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16789597, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16789597, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16799210, ["Country", "Country_source"]] = fix_country_from_text("Indonesia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16799210, ["Country", "Country_source"]] = fix_country_from_text("Indonesia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16813000, ["Country", "Country_source"]] = fix_country_from_text("Serbia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16813000, ["Country", "Country_source"]] = fix_country_from_text("Serbia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16813002, ["Country", "Country_source"]] = fix_country_from_text("Serbia")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16813002, ["Country", "Country_source"]] = fix_country_from_text("Serbia")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16832782, ["Country", "Country_source"]] = fix_country_from_text("Poland")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16832782, ["Country", "Country_source"]] = fix_country_from_text("Poland")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16838646, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16838646, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16850861, ["Country", "Country_source"]] = fix_country_from_text("India")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16850861, ["Country", "Country_source"]] = fix_country_from_text("India")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16870534, ["Country", "Country_source"]] = fix_country_from_text("Spain")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16870534, ["Country", "Country_source"]] = fix_country_from_text("Spain")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16878059, ["Country", "Country_source"]] = fix_country_from_text("Mexico")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16878059, ["Country", "Country_source"]] = fix_country_from_text("Mexico")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16885864, ["Country", "Country_source"]] = fix_country_from_text("France")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16885864, ["Country", "Country_source"]] = fix_country_from_text("France")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16886805, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16886805, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16898595, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16898595, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16910490, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16910490, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16944730, ["Country", "Country_source"]] = fix_country_from_text("Bosnia and Herzegovina")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16944730, ["Country", "Country_source"]] = fix_country_from_text("Bosnia and Herzegovina")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16969028, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16969028, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 16972688, ["Country", "Country_source"]] = fix_country_from_text("Japan")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 16972688, ["Country", "Country_source"]] = fix_country_from_text("Japan")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 17016662, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 17016662, ["Country", "Country_source"]] = fix_country_from_text("United Kingdom")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == 17021975, ["Country", "Country_source"]] = fix_country_from_text("Germany")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == 17021975, ["Country", "Country_source"]] = fix_country_from_text("Germany")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")

df_temp_isna_notna_base.loc[df_temp_isna_notna_base["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")
df_temp_isna_notna_match_GLOBOCAN.loc[df_temp_isna_notna_match_GLOBOCAN["PMID"] == , ["Country", "Country_source"]] = fix_country_from_text("")


In [ ]:
df_temp_isna_notna_base.to_csv(DF_input_update_base + "temp_isna_notna.csv", index = False)
df_temp_isna_notna_match_GLOBOCAN.to_csv(DF_input_update_match_GLOBOCAN + "temp_isna_notna.csv", index = False)

del df_temp_isna_notna_base
del df_temp_isna_notna_match_GLOBOCAN